In [41]:
import pandas as pd
import nltk
datacorp = pd.read_pickle('pickles/lhv2016t2021.pkl')
datacorp['document'] = datacorp.index
testdoc = datacorp.loc[datacorp['document']== '2016_ar_en_eur_con_00.txt']
testdoc

,text,document
2016_ar_en_eur_con_00.txt,group annual indd as lhv group consolidated an...,2016_ar_en_eur_con_00.txt


In [65]:
from docsim import DocSim
import docsim

In [66]:
%%time

docsim_obj = docsim.DocSim(verbose=True)
# docsim_obj = docsim.DocSim_threaded(verbose=True)

Loading default GloVe word vector model: glove-wiki-gigaword-50
Model loaded
Wall time: 20.7 s


In [29]:
print(f'Model ready: {docsim_obj.model_ready}')

Model ready: True


In [67]:
titles = [item for item in testdoc['document']]
documents = [item for item in testdoc['text']]

print(f'{len(documents)} documents')
query_G = 'Audit and control, Board structure, Remuneration, Shareholder rights, Transparency and Performance'
query_S = 'Access to medicines, HIV/AIDs, Nutrition, Product safety,Community relations, Privacy and free expression, Security, Weak, governance zones, Diversity, Health and safety, ILO core conventions, Supply chain labor standards,Bribery and corruption, Political influence, Responsible marketing,Whistle-blowing systems,Disclosure and reporting, Governance of sustainability issues, Stakeholder engagement, UNGC compliance'
query_E = 'Biofuels, Climate change strategy,Emissions management and reporting, Access to land, Biodiversity management, Water,Environmental management, Environmental standards, Pollution control, Product opportunities,Supply chain environmental standards, Waste, recycling'

1 documents


In [68]:
%%time

similarities = docsim_obj.similarity_query(query_G, documents)


1 documents loaded into corpus
Wall time: 43.7 s


In [75]:
for idx, score in (sorted(enumerate(similarities), reverse=True, key=lambda x: x[1])[:5]):
    print(f'{idx} \t {score:0.3f} \t {titles[idx]}')

TypeError: 'float' object is not iterable

In [95]:
def interpret_soft_cosine_measure(doc1, doc2, dictionary, similarity_matrix):
    word_pair_importances = dict()
    for word1_id, word1_weight in doc1:
        for word2_id, word2_weight in doc2:
            word_similarity = similarity_matrix.matrix[word1_id, word2_id]
            word_pair_importance = word1_weight * word_similarity * word2_weight
            if word_pair_importance == 0:
                continue
            word1 = dictionary.id2token[word1_id]
            word2 = dictionary.id2token[word2_id]
            if (word1, word2) not in word_pair_importances:
                word_pair_importances[word1, word2] = 0.0
            word_pair_importances[word1, word2] += word_pair_importance
    norm = 1.0
    norm *= similarity_matrix.inner_product(doc1, doc1) or 1.0
    norm *= similarity_matrix.inner_product(doc2, doc2) or 1.0
    normalized_word_pair_importances = {
        (word1, word2): word_pair_importance / norm
        for (word1, word2), word_pair_importance
        in word_pair_importances.items()
    }
    similarity = sum(normalized_word_pair_importances.values())
    normalized_word_pair_importances = sorted(normalized_word_pair_importances.items(), key=lambda x: x[1], reverse=True)
    normalized_word_pair_importances = ' + '.join(
        '{:.02f} ({}:{})'.format(word_pair_importance, word1, word2) if word1 != word2 else '{:.02f} ({})'.format(word_pair_importance, word1)
        for (word1, word2), word_pair_importance
        in normalized_word_pair_importances
    )
    print('{:.02f} = {}'.format(similarity, normalized_word_pair_importances))

     

import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix

import nltk
from nltk.corpus import stopwords

sentence_obama = 'Audit and control, Board structure, Remuneration, Shareholder rights, Transparency and Performance'.lower().split()

sentence_president = 'group annual indd as lhv group consolidated annual report translation of the estonian original as lhv group tartu mnt tallinn info lhv ee lhv ee consolidated annual report legal name as lhv group commercial registry no legal address tartu mnt tallinn phone fax e mail lhv lhv ee main activities activities of holding companies banking security brokerage financial advisory finance lease and other lending management board erkki raasuke until madis toomsalu from supervisory board rain lõhmus andres viisemann tiina mõis heldur meerits raivo hein tauno tats sten tamkivi auditor as pricewaterhousecoopers as lhv group tartu mnt tallinn info lhv ee lhv ee contents statement of managing director management report business environment financial results governance of the group corporate governance report consolidated financial statements consolidated statements of profit or loss and other comprehensive income consolidated statements of financial position consolidated statements of cash flows consolidated statements of changes in equity notes to the consolidated financial statements note general information note summary of significant accounting policies note risk management note significant management estimates and assumptions note business segments note subsidiaries and associated companies goodwill note net interest income note net fee and commission income note operating expenses note due from central bank credit institutions and investment companies note available for sale financial assets note financial assets and liabilities at fair value through profit or loss note discontinued operations note loans and advances to customers note receivables from customers note other assets note tangible and intangible assets note deposits from customers and loans received note accounts payable and other liabilities note operating lease note subordinated debts note shareholders equity in the public limited company note assets under management note contingent assets and liabilities note transactions with related parties note basic earnings and diluted earnings per share note separate financial statements of parent company signatures of the management board to the consolidated annual report independent auditors report proposal for profit distribution signatures of the supervisory board to the annual report allocation of income according to emta classificators as lhv group tartu mnt tallinn info lhv ee lhv ee statement of managing director dear lhv investor lhv group and its subsidiaries hereinafter lhv have had another successful year most of our objectives were achieved with room to spare lhvs shares were listed on the tallinn stock exchange in may the overall staff satis faction is the highest on record and we received several important recognitions during the year the economic environment has remained stable despite the worrying underlying trends the inauguration of the new president of the united states is likely to add further to trade barriers to the detriment of most of the countrys trade partners including the european union against the backdrop of over regulation high debt burden negative demographic dynamics and radical tendencies in europe it is difficult to achieve long term broad based growth to fix these problems the governments of several major countries have decided to boost expenditure rather than maintain a budget balance with opec cutting production volumes this has fuelled expectations of inflation which can in itself be considered a positive trend in the current interest rate environment regional economic environment is characterised by a growth in wages and consumption in the baltic states a strong real estate cycle in sweden and a sluggish recovery in finnish economy investments are greatly missed in all of these markets despite a strong credit cycle growth in loan volumes is also evident in estonia although we can detect the first signs of a rise in credit pricing and growth in the number of companies in debt the greatest concern for enterprises is availability of labour and a quick growth in labour costs leading to a drop in profit nonetheless convergence with europe is unavoidable to enhance effi ciency of labour costs we therefore need to boost invest ments the record low fixed financing costs provide a good foundation for such efforts this is certainly something our enterprises should further exploit low interest rates have inflated the real estate sector trig gering a potential oversupply in rental cashflow projects lhv avoids financing projects without tenants residential developments are only financed if the corresponding condi tions serve to ensure a market balance on a positive note no other major balance shifts or risks are evident on the domestic scale lhv has done well in such an environment even though financial intermediation is among the most regulated busi ness sectors we have the opportunity to offer modern day customer tailored solutions our special talent lies in the ability to be involved and to understand the market this allows us to offer traditional products in a cutting edge dressing along with modern conditions model based products can be tailored to the customers needs we lay emphasis on speed with simplicity availability and speed having a significant bearing on the product in the competi tive digital age financial products do not have to be boring and old fashioned a product should create a positive emotional bond and serve as a well designed rational tool at the customers service lhvs results demonstrate that we have only scratched the surface with this approach certainly the year marked a milestone in corporate growth on may as lhv group launched public trading of its shares on the nasdaq tallinn stock exchange a total of more than five thousand investors decided to invest in the success of lhv contributing to our mission of boosting domestic capital the listing of the shares marked the fulfilment of a long term objective and a logical continua tion of the listing of subordinated bonds in in order to support lhvs growth we engaged a total of eur million from our investors issuing two million new shares the issue was oversubscribed three times with the earnings per share amounting to nearly in lhv posted a net profit of eur million in this constitutes a eur million increase from last year with a return on equity for the shareholders of lhv profit was supported by strong customer activity strong credit quality and the extraordinary revenue generated by the acquisition of danske capital and its managed funds busi ness volumes and profits were boosted in all core activities at the end of september the financial supervision authority issued the annual supervisory review and evaluation process srep to lhv group and lhv bank based on the rates as at the end of the financial supervision authority established the minimum total capital ratio at and the minimum tier capital ratio at for lhv group and lhv bank in view of the financial supervision authoritys evaluation and by adding the nega tive scenario buffer lhv group established the groups minimum total capital ratio at and the minimum tier capital ratio at the net profit posted by the largest business unit lhv bank amounted to eur million having grown by eur million year on year loans to customers increased by eur million and totalled eur million eur million worth of new deposits were engaged during the year with the total volume of deposits amounting to eur million by the end of the year the number of customers grew by thousand year on year amounting to thousand as at the end of the year the bank received various recognitions at the beginning of the year in january nasdaq declared lhv bank the baltic member of the year in february the survey company dive declared lhv bank the bank with the best service in estonia in the annual most valuable employer poll conducted by cv keskus lhv bank rose from last years position to the position succumbing only to estonias largest companies and ranking second among banks in lhv bank focused mainly on gaining new customers enhancing user convenience and expanding the credit product offers we launched the corporate micro loan product and laid greater emphasis on small enterprise as lhv group tartu mnt tallinn info lhv ee lhv ee financing together with the provision of the required banking services mortgage loan a product whose existence elim inates the main obstacle for a private customer wishing to change his or her bank was launched at the end of the year we have brought our cost structure to a level which allows to provide the mortgage loan product on favourable conditions we opened a new more spacious office in tartu both lhvs investment seminars and the börsihai stock exchange shark game witnessed a record number of participants we have responded to growing public interest by enhancing the flexibility of our growth account product allowing all interested customers to start regular investment with small amounts lhv bank has positioned itself the third largest local bank in estonia our home market is estonia and we wish to excel here this means ranking among the top three banks with regard to both customer numbers and business volumes as well as efficiency and profitability per employee lhv asset management posted a net profit of eur million in eur million more than last year net fee and commission income increased by eur million to eur million the expense side was affected by a depreciation of client agreements recognised through acquisition of danske capital and its managed funds in the amount of eur million the revenue of lhv asset management is bound to decrease by an average of in february when the new management fee rates for the pillar funds take effect the lowering of the fees is a benefit gained by pillar fund customers from the danske capital transaction for lhv asset management the highlight of the year was signing the acquisition of as danske capital on january immediately after conclusion of the contract and acquisition of the required permits lhv launched the merger of the two management companies the merger took effect on july with the balance sheet date of may as a result of the merger danske capital was dissolved with lhv asset management becoming its legal successor lhvs business plan also includes merger of pension funds with a similar strategy similar mandatory pension funds will be merged on may in order to finance the purchase of danske capital the share capital of lhv asset management was increased by eur million to eur million in february in view of the current profit and lowering of the capital requirements as a result of merger of the management companies lhv asset management decreased its share capital by eur million before the end of the year the acquisition of and merger with danske capital generated eur million in intangible assets with the client agreements to be depreciated over a period of several years alongside actively managed pension funds lhv asset management also launched passively managed pension funds in the second half of the year the principal feature of passive funds is that these funds are not actively managed this in turn allows to offer passive funds at lower manage ment fees passive funds are invested in markets at all times with the yield mimicking the fluctuations of global markets we witnessed several stock market declines during the year but the markets rebounded quickly actively managed pension funds avoided major investment risks with the funds easily navigating all market fluctuations our flagship lhv pension fund l ranked second in its category in terms of yield as at the end of the year among conservative funds lhv pension fund s generated the highest yield for the year the number of active customers of lhvs manda tory pension funds grew by thousand year on year amounting to thousand the pension funds acquired through the purchase of danske capital had a total of thousand customers as at the end of the year the volume of funds managed by lhv grew by eur million to eur million of which eur million can be attributed to the transaction related pension funds mokilizingas posted a net profit of eur million in a eur million increase year on year profit was positively affected by the disposal of uncollectable receivables in the amount of eur million in the loan portfolio shrank by eur million during the year totalling eur million mokilizingas return on equity was mokilizingas is the second largest hire purchase provider on the lithu anian market and is recognised for the good service and transparent pricing in cooperation with a partner in the telecommunication sector mokilizingas also started offering hire purchase services in latvia furthermore the credit card product is currently in pilot stage all in all the year was successful for lhv we follow the principle of simplicity and modernity and believe in the opportunities for further growth in all key segments retail banking will focus on modern day payment solutions and development of a futuristic mobile bank the corporate loan and consumer financing segment in estonia and lithuania will concentrate on strong credit quality to provide a good foundation for growth in the credit portfolio private banking will pursue the next level of quality asset management will focus on fond yield and expansion of the client base in addition to organic growth we will continue to search for opportunities beyond estonian and lithuanian borders we will continue to make forward looking investments with the basic principle of achieving long term capital efficiency lhvs objectives for the upcoming years we endeavour to serve as the best financial service provider to private customers and smes who wish to save and invest financial assets we endeavour to serve as an attractive and recognised employer offering progress self fulfilment and growth we endeavour to do business in such a way as to ensure a return on equity we endeavour to become estonias most valuable publicly traded company madis toomsalu as lhv group tartu mnt tallinn info lhv ee lhv ee as lhv group activities holding company as lhv pank activities banking as lhv varahaldus activities fund management oü cuber tehnology activities money transfering uab mokilizingas activities consumer finance hire purchase as lhv finance activities hire purchase consumer finance management report as lhv group hereinafter the group or lhv is the biggest domestic finance group and capital provider in estonia the main subsidiaries of as lhv group are as lhv pank hereinafter the bank and as lhv varahaldus hereinafter lhv varahaldus lhv was established in by people with long experience in investing and entrepreneurship lhv s offices for client servicing are located in tallinn tartu and vilnius over people work in lhv over customers use banking services offered by lhv and the lhvs pension funds have over clients key events in ipo and listing shares in may lhv issued million new shares and listed all its shares on nasdaq tallinn stock exchange by that extremely successful transaction lhv group fulfilled the promise given to shareholders years ago lhv bank as lhv pank launched in mortgage loan product and that filled last larger gap in product offering to retail clients in addition lhv pank was the first one to start issuing nfc cards in estonia mokilizingas mokillizingas stopped providing financial leasing services and is now focusing only on consumer loans and hire pur chase products lhv varahaldus lhv varahaldus bought danske capital and merged that into its organization in lhv varahaldus was the first to launch index funds to offer low fee based second and third pillar pension fund as lhv group tartu mnt tallinn info lhv ee lhv ee operating principles the mission of lhv is to help create local capital and to support the development of local entrepreneurship lhv concentrates on active and independent customers with an entrepreneurial mind set for private consumers lhv mainly offers services of depositing and increasing monetary funds for corporate clients lhv offers flexible and suitable financing and managing of everyday financial operations the products and services provided by lhv are simple transparent and appropriate client interaction is mainly conducted using modern electronic communication chan nels costs avoided will reach the clients of lhv through affordable cost of everyday service lhv was created as a partnership and partnership is the central way of growing lhvs business new initiatives will be implemented faster and more successfully when partners have same interests and long term support of each other lhv is public company which is listed on the tallinn stock exchange since may so all clients and partners of lhv may be owners of lhv shareholders as lhv group has common shares with par value of euro as at as lhv group had shareholders shares belonged to the members of the supervisory board and management board and to their related parties shares belonged to estonian entrepreneurs and investors and to their related parties ten biggest shareholders as at number of shares percentage name of the shareholder as lõhmus holdings rain lõhmus viisemann investment ag ambient sound investments oü oü krenno as genteel as amalfi oü kristobal sia krugmans oü merona systems as lhv group tartu mnt tallinn info lhv ee lhv ee business environment even though the final figures are yet to be revealed the world economy is likely to have decelerated in for the second year in a row according to imfs january forecast there is a good chance that the trend will be reversed with the global economic growth expected to gain momentum and reach this year the dynamics of the previous six months suggest that developed and developing markets have taken a different path the new administration in the united states foresees a growth in budgetary expenditure similarly the forecast for the chinese economy has been upwardly revised against endless government stimulus in latin america however economic recovery has remained sluggish failing all expec tations mexico is suffering from us related political uncer tainty and the indian economy from backlashes of the bank note replacement the keyword for is elections in europe the year will see elections in both germany and france while the newly inaugurated president of the united states is already revealing the first major changes in course growth in protec tionist tendencies review of international trade treaties and prolongation of the long awaited structural reforms have the potential to re shape the long term economic outlook economic growth in the euro area hovered around in with the annual change expected to remain in the same ballpark the imf forecasts a marginal slow down in growth to the return of inflation fuelled by the recovery in oil and energy prices will start exerting pressure on domestic consumption the main engine for growth so far with the pressure on wage increase remaining non existent consumption is positively affected by consistent growth in employment enhanced corporate profitability is bound to attract investments the economic sentiment indicator of estonias key trade partners remained at five year highs strongly advancing in the second half of the year a major advance in sentiment was evident in finland and lithuania in suggesting an improvement in the countries economic outlook despite overall optimism the economic growth in sweden is losing pace dropping to in this can mainly be attributed to stabilisation of domestic consumption which up to now has been stimulated by low interest rates the growth in investment volumes is also expected to slow down significantly the forecast for swedish economic growth this year is around estonias interests revolve around a strong growth in swedish import volumes and investor friendly tendencies in swedish banks finland is continuing its sluggish recovery with the economy growing in consumer confidence rose above long term average in and is nearing ten year highs the highest among eu member states the positive trend is reflected in the general economy where domestic consumption has remained the main engine for growth for quite some time the key for the upcoming year lies in the ability to boost export volumes so far this has remained quite a stumbling block similarly to sweden estonias inter ests lie in further growth in finnish import volumes the economic growth in lithuania a major destina tion market for lhv slowed down to in this can mainly be attributed to shrinking volumes of projects financed through eu assistance funds which also had an adverse effect on the construction sector strong domestic consumption remained the main engine for growth fuelled by a quick increase in wages and decrease in unemploy ment the lithuanian economic growth is expected to improve compared to the growth last year growth in the volume of eu financed projects could provide a further stimulus with the transition to the new programming period expected to contribute the estonian economy in can mainly be character ised by failure to meet the overly optimistic forecasts the estimated growth of is the slowest in the post crisis period difficulties in the oil shale industry and the energy sector mainly conditioned by external factors also had their bearing the poor investment volumes in recent years have impeded the potential growth of estonian economy the economic forecast published by the european central bank in december foresees a growth for the year income increase is expected to continue but the return of inflation will have a significant impact on growth in real wages conse quently export and investments are becoming increasingly important alongside domestic consumption government expenditure is expected to rise fuelled among other things by the eu presidency and the local government elections in the autumn of the policy measures implemented by the new coalition have caused a greater than usual uncer tainty with regard to the outlook for the upcoming year as lhv group tartu mnt tallinn info lhv ee lhv ee key economic indicators of estonia bank of estonia forecast nominal gdp eur billion gdp volume private consumption expenditures government consumption expenditures fixed capital formation exports imports cpi unemployment rate of the labour force current account of gdp budget balance of gdp numbers reported are annual rates of change in per cent if not noted otherwise actual gdp and its components are chain linked including npish the budget balance forecast considers only those measures on which sufficient information was availanble at the date of the forecast sources eesti pank eurostat statistics estonia lhv is not expecting any abrupt economic changes in estonia over the month horizon economic growth is expected to pick up pace but is unlikely to achieve its full potential the policy measures of the new coalition will be a major factor rapid growth in wages is exerting pres sure on the already dropping corporate profits corporate adaptability and successful enhancement of productivity will thus play an increasingly important role a qualitative leap in investments is required to achieve the objectives in terms of economic sectors higher than average risks are evident in the processing industry and energy related sectors which are facing wage pressures special attention should also be paid to transport and warehousing as well as the risks of the construction sector where the drop in profit figures has been significant compared to previous periods lhv has also taken a more conservative approach to the real estate market carefully monitoring the dynamics on the local and scandinavian markets rental property projects involve a significant risk of insufficient demand for absorbing the developed volumes in the near future with corrections expected in either rental prices or vacancy rates on a positive note the financing environment remains favourable saw a growth in the balance of loans taken from credit institutions the loan to deposit ratio and overdue loans have maintained their post crisis lows with the record low interest rates and interbank competition having a positive impact on local entrepreneurship the opportunities should be further exploited lhv stands ready to boldly and efficiently contribute to the search of new sources of growth offering a long term partnership to enterprises and making an effort to create additional value overdue loans share in loan portfolio overdue loans over days overdue loans days overdue loans days overdue loans up to days source bank of estonia as lhv group tartu mnt tallinn info lhv ee lhv ee financial results volume of deposits and loans received volume of net loans volume of funds net interest income net fee income net financial income net income expenses operating profit loan provisions income tax expense profit from discontinued operations profit including attributable to owners of the parent eur million the summary table of financial results does not separately show income and expenses of the finnish business operations in the annual report they are presented separately as a discontinued operations the groups pre tax profit for amounted to eur million and net profit totalled eur million pre tax profit is more than a year before eur million from continued operations net interest income grew and net fee income increased financial income grew by the end of december the total volume of the groups loan portfolio less impairments amounted to eur million december eur million the volume of portfolio increased in a year in the loan portfolio the majority is corporate loans that increased in a year to eur million eur million the portfolio of retail loans increased in a year amounting to eur million eur million the volume of groups deposits increased in a year and totalled eur million by the year end eur million the share of demand deposits of all deposits increased and reached operating expenses increased in a year by business units in consolidated figures in as lhv pank earned profit of eur million lhv varahaldus earned eur million and uab mokilizingas earned a profit of eur million lhv group itself ended the year with a loss of eur million due to the fact that it does not engage in operating business and the main expense in the group is the interest expense from issued subordinated debt liquidity and capitalization the liquidity coverage ratio lcr of the group calculated according to the definitions of the basel committee was at the end of december the group includes cash and bond portfolios into a liquidity buffer accounting for of the balance sheet total among its liquidity buffer the groups loan to deposit ratio at the end of was the groups level of own funds as at was eur million million the group is well capitalised as at the end of the reporting period with a capital adequacy level of and tier capital ratio of the group uses the standard method for the calculation of credit risk and market risk capital requirements and the basic indicator approach for the calculation of operational risk capital requirements the group has complied with all capital requirements during the financial year and in previous year as lhv group tartu mnt tallinn info lhv ee lhv ee each year an internal capital adequacy assessment process icaap is performed the goal of which is to identify potential capital needs in addition to regulatory capital requirements leverage ratio calculated according to basel committee on banking supervision requirements as at was leverage ratio is calculated as groups total tier own funds divided by group s total risk exposure measure incl risk position on assets and off the balance sheet liabilities as at the end of the reporting period the leverage ratio is arithmetic mean of the monthly leverage ratios over the last quarter key figures eur million change change change net profit net profit attributable to owners of the parent average equity return on equity roe average assets return on assets roa net interest income average interest earning assets net interest margin nim spread cost to income ratio return on equity is calculated based on lhv group net profit and equity attributable to owners of the parent and does not include non controlling interest explanations average equity attributable to owners of the parent equity of current year end equity of previous year end return on equity roe net profit attributable to owners of the parent average equity attributable to owners of the parent average assets assets of current year end assets of previous year end return on assets roa net profit average assets net interest margin nim net interest income average interest earning assets spread yield on interest earning assets cost of interest bearing liabilities yield on interest earning assets interest income average interest earning assets cost of interest bearing liabilities interest expenses average interest bearing liabilities cost to income ratio total operating expenses total income as lhv group tartu mnt tallinn info lhv ee lhv ee risk management the purpose of risk management is to identify assess manage and control all risks related to the activities of the group in order to ensure the group s reliability stability and profitability the principles of identification measuring reporting and control of risks at the group are set out in the policies and procedures approved by the supervisory board and daily risk management is organised by risk manage ment unit in the group risk management is based on three lines of defence business units as the first line of defence are responsible for taking and managing risks the second line of defence which includes the risk management unit is responsible for developing and maintenance of the risk management framework the third line of defence internal audit function carries out independent control over processes operations and control functions in the entire group the rules and procedures of risk management are regularly reviewed at least once a year and updated in case of a need detailed overview of the risks taken by the group is presented in note of the annual report paid in share capital share premium statutory reserves transferred from net profit other reserves accumulated profit deficit intangible assets subtracted net profit for accounting period dividend offer including income tax non controlling interests total tier capital subordinated debt total tier capital exceeding limitations of subordinated debts and preference shares net own funds for capital adequacy calculation capital requirements central governments and central bank under standard method credit institutions and investment companies under standard method companies under standard method retail claims under standard method public sector under standard method housing real estate under standard method overdue claims under standard method investment funds shares under standard method other assets under standard method total capital requirements for covering the credit risk and counterparty credit risk capital requirement against foreign currency risk under standard method capital requirement against interest position risk under standard method capital requirement against equity portfolio risks under standard method capital requirement against credit valuation adjustment risks under standard method capital requirement for operational risk under base method total capital requirements for adequacy calculation capital adequacy tier capital ratio audited numbers which include the dividend offer capital base in eur thousands as lhv group tartu mnt tallinn info lhv ee lhv ee overview of the groups subsidiaries in as lhv pank consolidation group as lhv pank is based on estonian capital the banks clients include private individuals small and medium sized compa nies and institutional investors the branch offices of the bank are located in tallinn tartu and vilnius will be closed riga office was closed as lhv pank employs over people more than clients use the banks services as lhv pank is one of the largest brokers on nasdaq omx baltic stock exchanges and the largest broker for baltic retail investors in international markets the bank owns of the subsidiary as lhv finance that provides hire purchase and consumer finance services business activities has been year of growth in all business segments growth of loans was according to expectation and deposits transaction volumes exceeded the budget the volume of deposits grew by year on year and reached million euros by year end million euros as a result of the overall low interest rate environ ment a change has occurred in the deposit structure as the deployment of money in term deposits is no longer as attractive the bank has used the liquidity account concept to attract more active customers during the we have significantly managed to reduce deposits concentration at the end of year there were only five depositors exceeding of total deposits in total their share was of total deposits we have managed significantly to increase the share of retail deposits the proportion of demand deposits has increased accounting for of all deposits as at the end of december one year ago the loan portfolio volume grew by year on year and amounted to eur million at the end of december eur million the majority of the loan portfolio is made up of corporate loans to legal entities which grew by year on year to eur million eur million the loan portfolio given to private individuals grew by year on year to eur million eur million as lhv pank issued a loan to the other subsidiary of as lhv group uab mokilizingas the amount of which at the year end was eur million eur million in autumn bank started offering mortgage loans clients have shown very high interest from the very first days approximately of applicants have been lhv clients and another half new clients as new developments lhv connect and new functionality in mobile were launched lhv connect offers corporate customers a possibility to link their bank account informa tion directly to their accounting system lhv connect allows making payments receive notifications and import account statements banks organizational structure is divided into three larger business lines retail banking private banking and corporate banking and three larger support functions it finance and support and risk management marketing and communica tion and human resource department are across the group during the year corporate loan portfolio was divided into two credits below thousand euros were given over to retail banking and all credits exceeding thousand euros remained in corporate banking shifting smaller loans to retail banking was important to keep corporate banking and credit analyses focus on larger and more individual approach requiring credit clients credits below thousand euros are analysed more cost efficient scoring process at the end of first quarter latvian branch was closed the banks operations in latvia were small scale in its business pursuits the bank has been governed above all by the business interests of its customers and endeavours to keep pace with the customers estonian enterprises currently need more services and support on the western european front at the end of year new office was opened in tartu in second half of year the internal personnel satisfaction survey was carried out the respondent percentage and also the marks have been all time high bank received several awards we rec eived member of the year award by nasdaq research agency dive nominated us bank with best service in baltics according to yearly poll made by cv keskus lhv pank raised to eight spot as lhv group tartu mnt tallinn info lhv ee lhv ee financial results the net profit of the bank in amounted to million euros up compared to the previous year million euros net interest income grew by and net fee and commission income increased by financial income was higher by compared to the previous year the large increase in net interest income is attributable to a growth in lending and deposit volumes among fee and commission income the greatest contri bution continues to come from security brokerage income income on cards and settlements are also showing a growth trend operating expenses grew by year on year which was expected given the current phase of high growth and new business development loan losses were higher compared to the previous year organisation the organisational structure of the bank continues to be divided in six major areas retail banking private banking business banking information technology finance and operations and risk management incl compliance statement of profit or loss and other comprehensive income eur million change net interest income net fee and commission income net gains from financial assets total net operating revenues other income operating expenses loan losses discontinued operations net income volumes eur million change loan portfolio bond portfolio deposits equity number of customers holding assets in bank as lhv group tartu mnt tallinn info lhv ee lhv ee as lhv varahaldus as lhv varahaldus hereinafter lhv asset management is involved in the management of investment funds the prin cipal units of lhv asset management are the investment unit risk management unit operations unit and sales and customer service unit for lhv asset management the highlight of the year was the acquisition of and merger with as danske capital the contract of purchase and sale was entered into with danske bank a s and danske bank a s estonia branch on january upon entry into the contract lhv started applying for the permits required for the purchase of danske capital the permits were issued in april immediately after conclusion of the transaction of purchase and sale of shares on may lhv launched the merger of the two manage ment companies the merger took effect on july with the balance sheet date of may lhv asset management served as the acquiring company in the merger as a result of the merger danske capital was dissolved with lhv asset management becoming its legal successor three funds with no customers were dissolved upon the merger in august lhv world equities fund as the acquiring fund merged with danske invest new europe fund lhv asset management s business plan also includes merger of pension funds with a similar strategy similar pillar funds will be merged on may in lhv asset manage ment set up new passively managed pillar and pillar pension funds with their operations launched on november as at the end of the year lhv asset management managed mandatory pension funds the figure will drop to after the planned fund mergers voluntary pension funds after the planned merger and ucits and offered invest ment consulting to one ucits sef lhv persian gulf fund which merged with the lhv persian gulf fund in in order to finance the purchase of danske capital the share capital of lhv asset management was increased by eur million to eur million in february in view of the current profit and lowering of the capital requirements as a result of merger of the management companies lhv asset management decreased its share capital by eur million before the end of the year lhv asset management posted a net profit of eur million in eur million more than the year before net fee and commission income increased by eur million to eur million eur million of the increase in revenue can be attributed to business volumes added as a result of merger with danske capital operating expenses increased by eur million to eur million the acqui sition of and merger with danske capital generated eur million in intangible assets with the client agreements to be depreciated over a period of several years depreciation of client agreements added eur million to expenses in as the volume goes up the management fees for pillar funds will be lowered the management fees of lhv s actively managed pension funds will thus be lowered from february when the new rates will take effect by an average of the lowering of the management fees is a benefit gained by pillar fund customers from the danske capital transaction the number of lhv asset management s mandatory pension fund customers grew by thousand year on year amounting to thousand the pension funds acquired through the purchase of danske capital had a total of thousand customers as at the end of the year the volume of funds managed by lhv grew by eur million to eur million of which eur million can be attributed to the transaction related pension funds as at the end of the year lhv asset management s pillar market share grew from to in terms of volume and from to in terms of active customer numbers in we witnessed several stock market declines during the year but the markets rebounded quickly actively managed pension funds avoided major investment risks with the funds smoothly navigating all market fluctuations our flagship lhv pension fund l ended the year by ranking second in its category among conservative funds lhv pension fund s generated the highest yield for the year in the parliament adopted the long awaited invest ment funds act regulating the functioning of management companies and funds with the entry into force of the act on january the capital requirements applied for lhv asset management will be lowered and the exit fees of pillar funds will be abolished revenue generated in eur million financial results eur million change net fee income other financial income total net operating revenues operating expenses net profit assets under management number of clients in pension funds thousands as lhv group tartu mnt tallinn info lhv ee lhv ee uab mokilizingas hire purchase in hire purchase hp segment mokilizingas continues to work in a strong collaboration with main partners bite lietuva and kesko senukai launching seasonal campaigns and supporting the sales hp sales amounted to eur million eur million the number of contracts signed decreased by compared to average contract amount remained stable margins remained at similar levels with a slight decrease compared to due to increased year end campaigns consumer loan consumer loan cl sales amounted to eur million and decreased by compared to the number of contracts signed remained stable however average contract amount decreased by margins are stable with a slight increase compared to after the successful launch of credit card pilot the second phase started in total number of cards issued reached thousand by the end of with a total eur million credit limit average credit limit is eur card which is above the expectation average spending per month is also higher than expected however usage is quite low of activated cards have at least one transaction objectives of will be dedicated to the enhance ment of card usage rate and product knowledge allocation to the customers especially concentrating on the existing card owners to get in line and exploit high revolving balance perspectives the main event for non bank consumer credit segment in lithuania was a change of regulation in which tightened the evaluation of clients eg more conservative limits to evaluation of income and liabilities of married clients this also brought turbulence in the market and had a signif icant impact for sales especially cl in hp side mokilizingas and bite developed an invoice consolidation project the clients now have an option to receive single invoice from bite and mokilizingas for telco services and hired handsets respectively this allows more flexibility for both companies to promote additional sales as well as use telco input for management of late payments as planned mokilizingas has launched a renewed self ser vice website this is a major step towards unifying all customer related processes since all the following projects will be designed in the same manner the new process allows customers to take consumer loans without compul sory registration the consent ratification and general admis sion process is quicker and more user friendly senukai credit card sales were launched in december though the sales campaign will start in mokilizingas also implemented new integration with compulsory credit database provided by bank of lithuania will be dedicated to further optimizing internal processes and developing more effective financing solu tions to maximize sales in relation to tightened regulation of bank of lithuania further developments will include new services aimed to improve customer experience at all pos and e commerce platforms as lhv group tartu mnt tallinn info lhv ee lhv ee governance of the lhv group supervisory board rain lõhmus is a founder of lhv and the chairman of the supervisory board he is a member of the supervisory boards of as lhv pank as audentes and as arco vara and a member of the management boards and supervisory boards of other companies rain lõhmus graduated from tallinn university of technology and management program of harvard business school he is a member of estonian business association rain lõhmus owns and his related parties own shares of as lhv group raivo hein is the owner and a member of management board of oü kakssada kakskümmend volti he is a member of the supervisory board of as lhv pank and a member of management boards and supervisory boards of other companies raivo hein graduated from tallinn university of technology raivo hein does not own shares of as lhv group oü kakssada kakskümmend volti owns shares of as lhv group heldur meerits is the owner and a member of the manage ment board of as amalfi he is a member of the supervisory boards of as lhv pank as audentes and kodumaja as and a member of the management boards and supervisory boards of other companies heldur meerits graduated from the faculty of economics and business administration of tartu university he is a member of the supervisory boards of sa avatud eesti fond sa dharma and other foundations heldur meerits does not own shares of as lhv group as amalfi owns shares of as lhv group tiina mõis is the owner and manager of as genteel she is a member of the supervisory boards of as lhv pank as baltika and other companies tiina mõis graduated from tallinn university of technology tiina mõis does not own any shares of as lhv group as genteel owns shares of as lhv group sten tamkivi is the ceo of teleport prior to co founding teleport as an entrepreneur in residence at andreessen horowitz sten served as an early executive at skype he holds a number of management and advisory board seats sten tamkivi owns shares and seikatsu oü owns shares of as lhv group tauno tats is a member of the management board of ambient sound investments oü he is a member of the supervisory board of eften kinnisvarafond as and a member of management boards and supervisory boards of other companies tauno tats graduated from tallinn university of technology tauno tats does not own shares of as lhv group ambient sound investments oü owns shares of as lhv group as lhv group tartu mnt tallinn info lhv ee lhv ee andres viisemann is the founder of lhv and the manager of lhv pension funds he is a member of the supervisory boards of as lhv pank as lhv varahaldus and as fertil itas and a member of the supervisory boards and manage ment boards of other companies andres viisemann grad uated from the university of tartu and obtained a masters degree at insead in international business management he is a member of the supervisory board of estonian health insurance fund andres viisemann does not own shares of as lhv group his related parties own shares of as lhv group madis toomsalu is the chairman of the supervisory boards of as lhv pank and as lhv varahaldus madis toomsalu owns shares of as lhv group management board supervisory and management boards of the subsidiaries as lhv pank supervisory board madis toomsalu chairman raivo hein heldur meerits tiina mõis rain lõhmus sten tamkivi andres viisemann management board erki kilu chairman jüri heero andres kitter meelis paakspuu indrek nuume martti singi as lhv finance supervisory board erki kilu chairman rain lõhmus veiko poolgas jaan koppel management board nele roostalu as lhv varahaldus supervisory board madis toomsalu chairman erki kilu andres viisemann management board mihkel oja chairman joel kukemelk cuber tehnology oü management board jüri laur uab mokilizingas supervisory board as lhv group as lhv pank kūb razfin uab management board erki kilu mantas jonuška benas pavlauskas jurgis rubaževičius ceo benas pavlauskas as lhv group tartu mnt tallinn info lhv ee lhv ee lhv listed shares and subordinated bonds lhv group redeemed a promise given to shareholders years ago by issuing million new shares and by listing all shares on tallinn nasdaq stock exchange on may it was the first listing in more than five years the share was well received to the stock and by the end of the year the share price had increased and reached the value of eur last price adjusted volume last price adjusted volume during the year million shares in the amount of eur million changed the owner on stock exchange at the end of the year lhv group had over shareholders from whom were also customers of lhv pank they were followed by swedbank customers and seb customers of the investors were companies and were private individuals lhv has issued two subordinated debt issues on nasdaq tallinn stock exchange the first of them will end in and the second in in no new bonds were issued nor redeemed bonds were traded in in the total amount of eur million at the end of the year lhv group had almost bondholders of them were also customers of lhv pank they were followed by the customers of swedbank and seb of the investor base were companies and were private individuals share options in the general meeting of shareholders of as lhv group approved the criteria for the granting of share options to the management board members and key employees and department managers and key employees of group companies that are part of as lhv group the aim of issuing share options is to align the interests of management board members and employees more effectively with the interests of shareholders and customers another objective is to offer a system of compensation on the labour market that is equal to competitors and comprehensive the compensation system in lhv consists only from two pillars base salary and share options there are no other hidden bonuses in form of additional payments free of charge services or company cars similarly to the share options were issued also in and in most likely the similar share options will be issued in the granting and size of share options was dependent upon the successful achievement of operational targets of the overall company and the targets of individual management board members and employees in the beginning of the share options were granted to people in the amount of thousand euros people in the amount of thousand euros people in the amount of thousand euros the company entered into share option agreements with the members of the management board and employees for a three year term for the granting of share options share options issued in can be exer cised between the period of and shares with nominal value of euro can be acquired for euros per share share options issued in can be exer cised between the period of and shares with nominal value of euro can be acquired for euros per share share options issued in can be exer cised between the period of and shares with nominal value of euro can be acquired for euros per share members of the management board and employees were unable to opt for the determined amount in cash instead of share options share options cannot be exchanged sold pledged or encumbered share options can be inher ited the share option agreement will become void upon premature termination of the employment contract at the employees initiative exemptions to which can be made by the supervisory board of as lhv group and due to termi nation of employment contract on an extraordinary basis by the employer for a cause related to the employee pursuant to the credit institutions act the supervisory board of as lhv group may reduce the number of share options issued or cancel share options if the overall financial results of the company have materially deteriorated compared to the previous period a member of the management board or employee of the company no longer meets award criteria the company no longer meets the prudential ratios the risks of the company are not adequately covered with own funds or determination of the performance pay was based on information which was inaccurate or incorrect to a material extent as lhv group tartu mnt tallinn info lhv ee lhv ee this report is presented in accordance with the accounting act of the republic of estonia and provides an overview of the governance of as lhv group lhv and compliance of governance with the corporate governance recommen dations of nasdaq omx tallinn stock exchange and the financial supervision authority lhv adheres to the corpo rate governance recommendations unless otherwise specified in this report general meeting lhv is a public limited company with its governing bodies being the general meeting of shareholders the supervisory board and the management board the general meeting where shareholders can exercise their rights is the highest governing body of lhv the primary duties of the general meeting include amendment of the articles of association increase and decrease of share capital decisions on issuance of convertible bonds election of and extension of the powers of supervisory board members premature removal of supervisory board members approval of the annual report and profit allocation approval of the share option programme determination of the number of auditors appointment and recall of auditors and other decisions within the competence of the general meeting as provided by law and the articles of association a resolution on amendment of the articles of associa tion shall be adopted if at least two thirds of the votes who participate in the meeting are in favour a resolution on amendment of the articles of association shall enter into force as of the making of a corresponding entry in the commercial register the resolution of the general meeting on the amendment of the articles of association minutes of the general meeting and the new text of the articles of association shall be attached to the application filed to the commercial register a shareholder has the right to participate in the general meeting address the general meeting with regard to items on the agenda ask relevant questions and make proposals the general meeting is called by the management board the annual general meeting with the approval of the annual report on the agenda is held at least once a year the management board calls the annual general meeting within six months after the end of the financial year at the latest the management board gives at least three weeks notice of the annual or special general meeting the agenda of the general meeting proposals of the management board and supervisory board draft reso lutions and other relevant materials are made available to the shareholders prior to the general meeting materials are made available on lhvs website shareholders are given the opportunity to ask questions about items on the agenda before the general meeting with the questions and responses published on lhvs website the list of shareholders entitled to participate in the general meeting is established based on the share register seven days before the general meeting one annual general meeting of shareholders was held in no special general meetings were held the annual general meeting of shareholders held on april resolved to list all shares of lhv on the nasdaq tallinn stock exchange baltic main list with the share capital of lhv conditionally increased the meeting also approved the annual report distributed the profit for the financial year gave an overview of the economic results for and approved the dividend policy the general meeting was held in the estonian language the annual general meeting of was chaired by daniel haab head of legal department of as lhv pank who introduced the procedure of the general meeting and the procedure for asking questions regarding the companys activities from the management board in member of the management board erkki raasuke chairman of the supervisory board rain lõhmus members of the supervisory board andres viisemann tiina mõis heldur meerits raivo hein sten tamkivi and tauno tats and auditor verner uibo attended the annual general meeting none of the shareholders hold any shares which would grant them special control rights or voting rights lhv is not aware of any agreements in place between shareholders that would address the coordinated exercise of shareholder rights shareholders with significant influence include rain lõhmus and parties related to him and andres viisemann and parties related to him corporate governance report as lhv group tartu mnt tallinn info lhv ee lhv ee management board duties of the management board the management board is a governing body of lhv charged with the task of representing and managing lhv members of the management board are elected and removed by the supervisory board the consent of the candidate is required for the election of a member of the management board pursuant to the articles of association the management board has one to five members a member of the manage ment board is appointed for a term of five years unless otherwise decided by the supervisory board extension of the term of office of a member of the management board shall not be decided earlier than one year before the planned date of expiry of the term of office and not for a period longer than the maximum term of office prescribed by the law or the articles of association if the management board of lhv has more than two members the supervisory board must appoint the chairman of the management board the chairman of the management board organises the work of the management board the supervisory board may remove a member of the management board for any reason a member of the management board may resign for any reason by informing the supervisory board thereof the rights and obligations arising from the contract entered into with a member of the management board shall expire in accordance with the contract erkki raasuke served as the member of the management board of lhv until december madis toomsalu was elected the new member of the management board on the same day the supervisory board has entered into a management board member contract with madis toomsalu the supervisory board establishes the remuneration princi ples for management board members and sets the sever ance compensation policy for premature termination of the management board member contract the management board makes everyday management decisions at its own discretion governed by the best interests of lhv and its shareholders and not his or her personal interests or the interests of the controlling share holder the duties of the management board member include day to day management of lhv representation of the company management of lhvs operations in foreign markets and investor communication coordination of the development of lhvs strategy and its implementation by serving as an active chairman of the supervisory boards of the major subsidiaries of lhv the management board shall ensure the functioning of risk management and internal control within lhv depending on the business area pursued by lhv lhvs internal control system covers all activities carried out by the supervisory board management and staff members of lhv with the aim of ensuring efficiency of operations adequate risk manage ment reliability and accuracy of internal and external reporting unconditional compliance with all legal acts the internal audit system covers all business support and control units the objective of risk management within lhv is to recog nise correctly measure and manage the risks on a wider horizon the objective of risk management is to enhance corporate value by minimising damage and reducing vola tility of results risk management is based on a strong risk culture and built on the principle of three lines of defence with the first line of defence i e business areas being responsible for risk taking and day to day management of risks the second line of defence i e the risk management area is responsible for developing risk management methods and reporting on risks the third line of defence i e internal audit exercises independent supervision over the entire organisation management remuneration principles the purpose of establishing remuneration principles within lhv is the organisation of fair motivating and transparent remuneration in accordance with the law competence for the establishment of remuneration for members of the management board shall lie with the super visory board the remuneration committee shall review the basis of management remuneration on an annual basis in assessing the activities of the members of the manage ment board the remuneration committee shall above all consider the professional duties of the management board member his or her activities the activities of the entire management board as well as lhvs financial position the current status of and outlook for business operations in comparison with companies operating in the same sector of economy the remuneration payable to a member of the management board including the options programme must motivate the individual to operate within the best interests of lhv the basic salaries of members of the management board shall be set forth in the employment contract and the basic salary of the chairman of the management board in the management board member contract the principles of remuneration of management or staff members engaged in internal control and risk management shall ensure their independence and objectivity in the performance of their risk management internal control duties the remuneration of this staff shall not depend on the results posted by the units supervised the objectives shall be described on the level of individual staff members as lhv group tartu mnt tallinn info lhv ee lhv ee when measuring the key quantitative and qualitative results and assessing the risk lhv shall be governed by the following principles the performance criteria established by lhv must not stimulate excessive risk taking or sale of unsuitable products the performance criteria must not merely contain performance efficiency indicators e g profit revenue profitability expenditure and volume indicators or market based indicators e g stock price or bond yield but must also be adjusted with risk based indicators e g capital adequacy liquidity the criteria used for measuring risks and results shall be as closely related to the decisions of the staff member whose results are being appraised as possible and should ensure that the process of the establishment of the remuneration would have an appropriate impact on the staff members professional behaviour a good balance of both quantitative and qualitative as well as absolute and relative criteria shall be used in the specification of the performance criteria quantitative criteria shall cover a sufficient period of time so as to consider the risks taken by the staff members or business units the criteria shall be risk ad justed and contain economic efficiency indicators examples of qualitative criteria include achievement of strategic objectives customer satisfaction adherence to the risk management policy adherence to internal or external rules of procedure management skills crea tivity motivation and co operation with other business units and the internal control function lhv applies the long term performance pay programme i e the options programme an annual performance pay shall be added to the basic salaries of management board members top managers and key persons included in the options programme including staff members equivalent to management members the amount and disbursement of which corresponds to the achievement or non achievement of individual targets or lhvs targets the basis for estab lishment of performance pay in the options programme shall be objective and justifiable with the time period for which performance pay shall be paid previously determined the performance pay instruments consist fully of stock options severance compensation payable to a member of the management board shall correspond to his or her previous work results no severance compensation shall be paid if this is liable to damage the interests of lhv the size of the severance compensation shall correspond to the results achieved over time and to the remuneration received conflicts of interests management board members submit an annual declara tion of economic interests and conflicts of economic inter ests no transactions approved by the supervisory board were concluded between lhv and its management board members their close relatives or related parties in during his term of service erkki raasuke did not serve as a management board member in any other issuer erkki raasuke served as the chairman of the supervisory board of as eesti energia madis toomsalu is not a management board member or supervisory board member in any other issuer management board members have not been granted authority to issue or buy back shares supervisory board the supervisory board is a governing body of lhv charged with the task of planning the activities of lhv organising the management of lhv and conducting supervision over the activities of the management board the supervisory board determines and regularly reviews lhvs strategy the general plan of action risk management principles and the annual budget the supervisory board has five to seven members the members of the supervisory board are appointed for a term of three years the members of the supervisory board elect amongst themselves the chairman of the supervisory board charged with the task of organising the activities of the supervisory board rain lõhmus chairman andres viisemann tiina mõis heldur meerits raivo hein tauno tats and sten tamkivi serve as members of the supervisory board the term of office of supervisory board members is set to expire on may the general meeting of shareholders has resolved to establish for the supervisory board members a gross remuneration of euros per each meeting of the supervisory board where the member participates supervisory board members submit an annual declaration of economic interests and conflicts of economic interests no significant transactions were concluded between lhv and its supervisory board members their close relatives or related parties in a total of eleven ordinary meetings and one extraordinary meeting of the supervisory board was held in rain lõhmus andres viisemann and raivo hein participated in all meetings of the supervisory board in tiina mõis tauno tats and sten tamkivi were absent on one occasion and heldur meerits on two occasions all members of the supervisory board participated in more than half of the meetings of the supervisory board the supervisory board has formed three committees charged with the task of advising the supervisory board in issues related to audit remuneration and risk as described below within the framework of supervisory board meetings the supervisory board was regularly briefed on the operating and financial results of lhv and its subsidiaries in the supervisory board approved the conditional increase of share capital public offering of shares issue of new shares and the listing of all shares on the nasdaq tallinn stock exchange baltic main list the supervisory board author as lhv group tartu mnt tallinn info lhv ee lhv ee ised and approved the results of the subscription of shares and the allocation of shares the strategy of lhv was the principal topic of discussion at the meeting held in august the supervisory board approved the dividend payment prin ciples and dividend policy the acquisition of the shares of danske capital as by as lhv varahaldus and the change in the share capital of as lhv varahaldus the supervisory board approved the annual report the report of the supervisory board and the issue of share options for the results posted in the share capital of as lhv pank was increased erkki raasuke was removed from the posi tion of the management board member and madis toomsalu was appointed the new member of the management board audit committee the audit committee serves above all as an advisory body for the supervisory board with regard to accounting audit risk and capital committee internal control and internal audit supervision budgeting and compliance the audit committee is governed in its activities by the auditors activities act and the rules of procedure established by the supervisory board of lhv the audit committee has been charged with the task of monitoring and analysing the processing of financial infor mation in the extent necessary for preparing quarterly and annual financial statements efficiency of risk management and internal control the procedure for audit of the financial statements or consolidated financial statements as well as the independence of the audit firm or a sworn auditor repre senting the audit firm on the basis of the law and compli ance of their activities with the requirements of the auditors activities act the audit committee is responsible for making proposals to the supervisory board and providing recommendations for appointment or removal of the audit firm appointment or removal of the internal auditor prevention or elimination of problems or inefficiencies within the organisation and compliance with legal acts and good professional practice pursuant to the rules of procedure the audit committee gathers for meetings at least on a quarterly basis in the audit committee gathered on six occasions at five meetings the internal auditor gave an overview of the audits conducted and the audit findings the last meeting of the year discussed the findings of the external auditor on the basis of the interim audit for the last two meetings of the year discussed the internal audit work plan for the audit committee consists of a minimum of two members members of the audit committee have been appointed by the supervisory board for a term of one year with the aim of ensuring diversity of competences and inde pendence of committee members gerli kilusk chairman marilin hein and tauno tats serve as members of the audit committee gerli kilusk is a partner management board member and attorney at law with law firm primus gerli kilusk does not hold any shares in lhv marilin hein is the cofounder and cfo of eften capital as marilin hein does not hold any shares in lhv astrum oü a company owned by marilin hein holds shares of as lhv group tauno tats is a member of the supervisory board of lhv and the manage ment board of ambient sound investments oü tauno tats does not hold any shares in lhv ambient sound invest ments oü holds shares of as lhv group the remuneration payable to members of the audit committee shall be determined by the supervisory board of lhv members of the audit committee are paid a monthly net remuneration of euros the internal audit unit is governed in its activities by the auditors activities act credit institutions act other regu lations and the rules of procedure for internal audit as approved by the supervisory board of lhv the internal audit unit presents the supervisory board with an overview of the audits conducted and audit findings on a quarterly basis an overview of the audits carried out during the year and significant findings as well as performance of the annual plan is given once a year the annual internal audit plan is prepared on the basis of the requirements set forth by law or contract as well as the internal audit units assessment of the risks evident in the business areas and products aimed at covering all major areas and products with sufficient risk based regularity the internal auditor discusses all audit findings with the heads of the corresponding areas as well as the management board of the company and agrees upon the action plan for elimi nation of deficiencies the internal audit unit conducts quar terly follow up audits in order to monitor adherence to the agreed action plan to ensure efficiency the internal audit unit closely co operates with the risk control and compli ance units as well as the external auditor while maintaining its independence the internal auditor is invited to all super visory board meetings of lhv as lhv pank and as lhv varahaldus other important meetings of committees estab lished within group companies if possible and necessary and all management board meetings of as lhv pank and as lhv varahaldus in the reporting period the internal audit unit had two employees in estonia and one employee in lithuania remuneration committee the remuneration committee has been charged with the task of evaluating the principles of remuneration within lhvs subsidiary the credit institution as lhv pank and the impact of any remuneration related decision on adher ence to the requirements established for the management of risks own funds and liquidity the remuneration committee consists of a minimum of three members the remuneration committee conducts supervision of the remuneration of management board members and staff members evaluates implementation of the remuner as lhv group tartu mnt tallinn info lhv ee lhv ee ation policy at least on an annual basis and where neces sary proposes to update the remuneration principles and prepares the related draft resolutions for the supervisory board the remuneration committee held one meeting in in january the committee discussed and made a proposal to the supervisory board with regard to the recipients of share options and the related amounts for the results posted in as well as with regard to the pricing of share options the share options of resigning staff members and the remu neration of management board members rain lõhmus andres viisemann and until december erkki raasuke chairman served as members of the remuneration committee madis toomsalu has been serving as a member and the chairman of the remuneration committee from december onwards rain lõhmus is the founder of lhv and the chairman of the supervisory board of lhv rain lõhmus holds shares as lõhmus holdings shares and oü merona systems shares of as lhv group andres viisemann is the founder of lhv member of the supervisory board of lhv and manager of the pension funds of as lhv varahaldus andres viisemann does not hold any shares of as lhv group viisemann investments ag holds shares and viisemann holdings oü share of as lhv group madis toomsalu is a member of the management board of lhv madis toomsalu holds shares of lhv no remuneration is paid to members of the remuneration committee risk and capital committee the risk and capital committee has been charged with the task of advising the supervisory board and management board of lhv with regard to risk management principles and risk tolerance assessing the risks taken by lhv and moni toring implementation of the risk policy within lhv group companies the risk and capital committee also carries out supervision of the implementation of risk management principles by the management board in accordance with the instructions given by the supervisory board checks that the probability and timing of risk capital liquidity and revenue is taken into account in remuneration principles and that the business model and risk management principles are taken into account in the fees established for customers pursuant to the rules of procedure of the risk and capital committee the meetings of the risk and capital committee shall be held as a rule on a quarterly basis the chairman of the risk and capital committee shall give an overview of the activities of the risk and capital committee the reso lutions adopted and their implementation at least on an annual basis four ordinary meetings of the risk and capital committee were held in reviewing regular risk reports icaap work processes and capital plans key customers under monitoring changes in provisions current irb and the srep report the supervisory board was advised in issues related to risk management and capital management poli cies with the key principles of credit policy and loan portfolio management and the planned changes reviewed rain lõhmus chairman andres viisemann and tiina mõis serve as members of the risk and capital committee rain lõhmus is the founder of lhv and the chairman of the supervisory board of lhv rain lõhmus holds shares as lõhmus holdings shares and oü merona systems shares of as lhv group andres viisemann is the founder of lhv member of the supervisory board of lhv and manager of the pension funds of as lhv varahaldus andres viisemann does not hold any shares of as lhv group viisemann investments ag holds shares and viisemann holdings oü share of as lhv group tiina mõis is a member of the supervisory board of lhv and the owner and managing director of as genteel tiina mõis does not hold any shares of as lhv group as genteel holds shares of as lhv group no remuneration is paid to members of the risk and capital committee cooperation of the management and supervisory boards the management board and supervisory board work in close co operation to best protect the interests of lhv the co operation is above all based on an open exchange of views between as well as within the management board and the supervisory board the management board and the supervisory board joint their efforts to lay out the objectives and strategy of lhv in the management of lhv the management board is governed by the strategic instructions given by the super visory board the management board regularly discusses strategic management issues with the supervisory board within the framework of an open dialogue the management board regularly informs the supervi sory board of the key circumstances regarding the activity plans and business activities of lhv the risks involved and management of such risks no conflicts of interests occurred in the financial year disclosure of information lhv treats all shareholders equally and notifies all share holders of material developments accordingly information is disclosed in accordance with the rules set forth for publicly traded companies lhv has set up an investor relations website making avail able all documents and information in accordance with the corporate governance recommendations lhv publishes as lhv group tartu mnt tallinn info lhv ee lhv ee on its website a financial calendar containing the publication dates of the annual report and interim reports and the date of the annual general meeting of shareholders the published information is also made available in english and russian in lhv will start disclosing its annual plans and monthly results via the stock exchange announcements lhvs dividend policy is available on lhvs website stipu lating the dividend payment principles according to which the key precondition for payment of dividends includes external and internal regulatory capital standards which have to be met in a sustainable manner lhv may decide to waive dividend payment under the corresponding growth and or investment plans where the preconditions are met lhv shall pay a minimum of per cent of the shareholders profit before taxes in dividends income tax included in meetings and webinars with investors were held as needed and based on investor requests the relevant pres entations are available on lhvs website financial reporting and auditing lhv publishes its annual report on an annual basis the annual report is audited and approved by the supervisory board the number of auditors is determined and the auditors appointed by the general meeting of shareholders the general meeting also establishes the procedure for remu neration of auditors auditors are appointed for a single audit or for a specified term in the management board in cooperation with the audit committee organised a tender for the appointment of an auditor as part of the tender lhv met with and requested bids from four major internationally recognised companies as a result of the tender as pricewaterhouse coopers was chosen and appointed the companys auditor at the annual general meeting of shareholders held on april a three year contract was entered into with as pricewaterhousecoopers for the audit of the financial years the auditor has rendered contractual services to lhv consolidation group companies in including audits on the financial statements of group companies and reviews of the profits disclosed in quarterly financial statements consulting in connection with the performance of contractual obligations to the us tax authority audits of the as danske capital purchase analysis and other current matters transactions with related parties are disclosed in the notes to the financial statements statement of conformity lhv adheres to the corporate governance recommenda tions with the exception of the following instructions and recommendations for the reasons stated below issuers shall make participation in the general meeting possible by means of communication equip ment internet if the technical equipment is available and where doing so is not too cost prohibitive for the issuer observation of the general meeting and participation thereof has not been made possible by means of communication equipment there has been no need no corresponding requests and no suitable solution for such a facility none theless from onwards lhv stands ready to ensure participation of shareholders at the general meeting via secure electronic means without being physically present at the general meeting with the corresponding option added to the articles of association of lhv the management board shall have more than one member and a chairman elected by its members the management board or supervisory board shall establish an area of responsibility for each member of the management board defining as clearly as possible the duties and powers of each board member the principles for co operation between members of the board shall also be established the chairman of the supervisory board shall conclude a contract of service with each member of the board for discharge of their functions the management board of lhv has one member lhv is a holding company without any day to day operational busi ness activities thus a single member of the management board ensures sufficient and comprehensive management of lhv the major subsidiaries of lhv have management boards consisting of several members the member of the management board shall not be at the same time a member of more than two management boards of an issuer and shall not be the chairman of the supervisory board of another issuer a member of the management board can be the chairman of the supervisory board in company belonging to same group as the issuer erkki raasuke served as the chairman of the supervisory board of as eesti energia in as eesti energia has not issued any securities on the tallinn stock exchange but has issued bonds on the london stock exchange madis toomsalu is not a management board member or supervi sory board member in any other issuer basic wages performance pay sever ance packages other payable benefits and bonus schemes of a management board member as well as their essential features incl features based on comparison incentives and risk shall be published in clear and unambiguous form on website of the issuer and in the corporate governance recommendations report information published shall be deemed clear and unambiguous if it directly expresses the amount of expense to the issuer or the amount of foreseeable expense as of the day of disclosure as lhv group tartu mnt tallinn info lhv ee lhv ee the chairman of the supervisory board shall present the essential aspects of the management board remu neration and changes in it to the general meeting if the remuneration of some of the management board members has occurred on a different base then the general meeting shall be presented the differences together with the reasons therefore lhv discloses the total amount of remuneration paid to management board members in the notes to the financial statements lhv does not disclose the remuneration of individual members of the management board because this constitutes personal data the disclosure of which is not unavoidably necessary for evaluation of the operations and management quality of lhv at least half of the members of the supervisory board of the issuer shall be independent if the super visory board has an odd number of members then there may be one independent member less than the number dependent members independent members of the supervisory board of lhv are sten tamkivi and tauno tats other members of the super visory board of lhv are not independent in the meaning of the corporate governance recommendations as they have served in their current position for more than three consecutive years lhv has been in the stage of active development and growth where it has preferred people of long term management skills and banking experience as members of the supervisory board these people are also the largest shareholders of lhv lhv believes that as members of the supervisory board the largest shareholders are best motivated to contribute to the management of lhv and its development in the long term horizon the supervisory board shall describe in its evaluation report to judgment of the auditors work inter alia the services including advisory services that the auditor has provided to the issuer during the preceding year or shall provide during the next year also the remuneration the issuer has paid or shall pay to the auditor shall be published lhv has agreed with its foreign auditor not to disclose the contractual fees paid or payable to the auditor since this constitutes the parties confidential information the disclo sure of which is not unavoidably necessary for assessing the activities of lhv once a year lhvs audit committee shall review the services rendered by the foreign auditor the relevant volumes and fees pursuant to subsection of the accounting act a large undertaking whose securities granting voting rights have been admitted for trading on a regulated securities market of estonia or another contracting state shall describe in the corporate governance report the diversity policies carried out in the compa nys management board and senior management and the results of the implementation thereof during the accounting year if no diversity policies have been implemented during the accounting year the reasons for this should be explained in the corporate govern ance report lhv has not deemed it necessary to implement a diversity policy as lhv is governed in the recruitment of staff and management members by the best interests of lhv the education skills and previous experience of the person on a gender neutral on non discriminatory basis tiina mõis serves as a member of the supervisory board of lhv and a member of the risk and capital committee and gerli kilusk and marilin hein as members of the audit committee together with the management members lhvs subsidiaries engage a total of middle level managers of whom are women as lhv group tartu mnt tallinn info lhv ee lhv ee lhv strives to do business in a sustainable and socially responsible manner our greatest social contribution and responsibility is to provide a domestic alternative on the financial service market lhvs mission is to help raise estonian capital lhvs vision is focused on estonia where people and businesses dare to think big engage in enterprise and invest in the future lhvs values simple supportive effective lhv groups credit policy stipulates that in addition to potential obligations set forth by law corporate social responsibility also means voluntary consideration of the social factors in business processes social responsibility in lhv is a purposeful consistent voluntary contribution which extends beyond the obligations required by law and serves the interests of the society even though social responsibility constitutes a limitation in the pursuit of profit lhv has set its sights on enhancing social benefits without seeking profit the purpose of adherence to the principle of social respon sibility is to create long term value for lhvs counterparties social responsibility in lhvs operations is manifested in three ways as described below social responsibility in business operations responsible lending lhv cares about the business and living environment and makes an effort to minimise any negative environmental impact of its activities and processes the groups credit policy stipulates that lhv strives to operate in a socially responsible manner setting an example to both its staff members and the general public thus any credit decision for example must be governed by the prin ciple that lhv will not finance counterparties whose activ ities are liable to cause disproportionate damage to the environment or do not comply with the principles of social responsibility lhv takes special caution in financing tax transactions so as not to violate the spirit and meaning of the laws on tax private customers are financed based on the principle of responsible lending this is also evident in the groups high credit quality the positive register project has been launched on the initiative of lhv bank and lhv finance supporting estonian business environment the better the estonian economy is doing the higher the quality of life banking is the lifeblood of global economy as a local financial group we acknowledge our role in supporting and advancing the economy we engage capital from esto nian depositors directing it into estonian enterprises we contribute to the development of estonian capital markets we made an effort to set an example with the initial public offering of lhvs shares on the tallinn stock exchange our institutional brokerage helps baltic compa nies to engage funds from investors lhvs pension funds are among the largest local institutional investors pension funds investing in local economy lhvs pension funds have invested a considerable portion of the customers pension assets in estonia and the baltic states we have invested in estonian real estate enterprises forest in every seventh euro in pension fund assets contributed to the growth of estonian economy lhvs funds have invested an average of of their assets in esto nian securities for example pension fund l lhvs greatest pension fund in terms of customer numbers has made a quarter of its investments exposed to the stock market risk in estonia of investments exposed to the real estate and forest land risk has been made in funds investing predominantly in estonia we are always searching for and analysing local invest options we also stand ready to create such options ourselves lhv is the best market participant on baltic stock exchanges our everyday brokerage activities as well as our strive to set an example contribute to the functioning of the estonian capital markets in lhv bank received the member of the year award of nasdaq baltic stock exchanges lhv has been honoured with the same title on four previous occasions we make an effort to be a kind and understanding employer we would like lhv to serve as the best possible employer for our staff members lhv respects the occupational safety requirements established by law and considers the needs of the staff members facilitating study leaves paid health days etc the staff satisfaction survey conducted in revealed record high level of satisfaction among the groups employees open banking we create and develop our services based on the princi ples of open banking we aim at developing the estonian business environment and co operation by enhancing partnership innovation and transparency social responsibility in lhv as lhv group tartu mnt tallinn info lhv ee lhv ee sharing money wisdom and educating the investor base as a leader of the investor community lhv takes respon sibility for sharing money wisdom and advancing investor education we contribute via the following activities we organise regular free of charge investor school seminars for those interested in investing the investor school seminars had a total of participants in we have also started organising investor meets seminars allowing heads of publicly traded companies to meet investors we co operate with other estonian financial insti tutions and participate in the money wisdom month organised by the estonian banking association as well as other money wisdom projects initiated by estonian banks lhvs staff members gave lectures to school children on the money wisdom day we have set up a library of investment and economic literature our libraries in tallinn and tartu hold a total of more than different publications which are available for lending to our customers in addition to those interested in investing our library is also available to students of economy we continue to update the library we operate the lhv financial portal and the forum set up for discussing investing related subjects and sharing information lhv regularly contributes to creating content which could be of interest to esto nian investors ever since we have published the free of charge economic journal investeeri addressing various subjects related to entrepreneurship and investing four issues were published in the journal had a printrun of sponsorship within the limits of our available resources we support enterprises and initiatives that contribute to the develop ment of the estonian society when it comes to sponsorship projects we prefer a long term substantial partnership we stand ready to contribute to the community and the pursuit of innovative ideas which help to advance estonian life estonian culture estonian music days in we started supporting the music festival with the longest history in estonia by giving out the contemporary music award lhvs new music award went to liisa hirsch enn soosaar foundation with the support of lhv the annual enn soosaar award the ethical essayistics award tradition was started in in the award went to margus laidre books on estonian art we support publication of the books of estonian artists the latest project supported by lhv was painter kristi kongis authors book ma ei ole oma pead padjal liigutanud music events in lhv supported the intsikurmu festival in põlva and the acoussion live festival in central estonia education information technology foundation for education lhv partnered with the foundation to provide an lhv scholar ship for the best diploma papers in the field of information and communication technology in the scholarship was awarded to graduates of the tallinn university of tech nology liisi soots and viktor reinok sports healthy way of life estonian football association lhv bank has been the main sponsor of the estonian football association and the estonian national team since we have also estab lished the lhv football card support system which allowed to support estonian football clubs with a record amount in estonian cyclists union we have been the unions sponsor since and also supported the cyclists in estonian optimist class association we have helped to ensure sustainability of estonian sailing for the last years after the lhv supported revival of the association in the association has been actively engaged in promoting sailing among children and youngsters running competitions in lhv supported the organi sation of the peetri and rapla running competitions estonian economy and entrepreneurship estonian entrepreneur of the year we have been among the main sponsors of the estonian entrepreneur of the year competition organised by ey since gazelle movement lhv is a sponsor of the gazelle move ment the movement promotes and recognises rapidly growing estonian companies tallinn restaurant week we have supported the initiative of the estonian restaurant association since the initia tive aims at promoting the restaurant culture and introducing companies involved in the catering industry as lhv group tartu mnt tallinn info lhv ee lhv ee consolidated financial statements continuing operations interest income interest expense net interest income fee and commission income fee and commission expense net fee and commission income net gains from financial assets measured at fair value foreign exchange rate gains losses other financial income net gains from financial assets other income other expense staff costs administrative and other operating expenses profit before impairment losses on loans and advances share of result of associates impairment losses on loans and advances profit before taxes income tax expense net profit for the year from continuing operations profit from discontinued operations net profit for the year other comprehensive income loss items that may be reclassified subsequently to profit or loss available for sale investments revaluation of available for sale financial assets total profit and other comprehensive income for the year total profit attributable to owners of the parent non controlling interest total profit for the year total profit and other comprehensive income attributable to owners of the parent incl continuing operations incl discontinued operations non controlling interest total profit and other comprehensive income for the year basic earnings per share in euros diluted earnings per share in euros basic earnings per share in euros from continuing operations diluted earnings per share in euros from continuing operations consolidated statement of profit or loss and other comprehensive income in thousands of euros notes on pages to are an integral part of the consolidated financial statements new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee assets due from central bank due from credit institutions due from investment companies available for sale financial assets financial assets at fair value through profit or loss loans and advances to customers receivables from customers other financial assets other assets tangible assets intangible assets goodwill total assets liabilities deposits from customers and loans received financial liabilities at fair value through profit or loss accounts payable and other liabilities subordinated debt total liabilities equity share capital share premium statutory reserve capital other reserves retained earnings total equity attributable to owners of the parent non controlling interest total equity total liabilities and equity consolidated statement of financial position in thousands of euros notes on pages to are an integral part of the consolidated financial statements new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee cash flows from operating activities interest received interest paid fees and commissions received fees and commissions paid other income received staff costs paid administrative and other operating expenses paid cash flows from operating activities before change in operating assets and liabilities net increase decrease in operating assets net increase decrease in financial assets held for trading at fair value through profit or loss loans and advances to customers mandatory reserve at central bank security deposits other assets net increase decrease in operating liabilities demand deposits of customers term deposits of customers loans received repayments of loans received financial liabilities held for trading at fair value through profit or loss other liabilities net cash from used in operating activities from continuing operations cash generated from used in operating activities from discontinued operations net cash from used in operating activities cash flows from investing activities purchase of tangible and intangible assets disposal of tangible and intangible assets acquisition of subsidiaries net of cash acquired proceeds from disposal and redemption of investment securities available for sale net changes of investment securities at fair value through profit or loss cash from investment activities of discontinued operations net cash from used in investing activities cash flows from financing activities paid in share capital incl share premium proceeds from subordinated debt repayment of subordinated debt net cash from financing activities effect of exchange rate changes on cash and cash equivalents net increase decrease in cash and cash equivalents cash and cash equivalents at the beginning of the year cash and cash equivalents at the end of the year consolidated statement of cash flows in thousands of euros notes on pages to are an integral part of the consolidated financial statements new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee consolidated statement of changes in equity in thousands of euros share capital share premium accumulated deficit retained earnings total non cont rolling interest total equity statutory reserve capital other reserves balance as at transfer to statutory reserve capital note share options note profit for the year other comprehensive loss total profit and other comprehensive income for balance as at balance as at paid in share capital note transfer to statutory reserve capital note share options note profit for the year other comprehensive loss total profit and other comprehensive income for balance as at attributable to owners of lhv group notes on pages to are an integral part of the consolidated financial statements new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee these consolidated financial statements have been prepared in accordance with international financial reporting standards as adopted by european union for the year ended december for as lhv group and its subsidiaries as lhv pank as lhv finance subsidiary of as lhv pank as lhv varahaldus oü cuber technology and uab mokilizingas hereinafter referred together as the group as lhv group holds share interest in uab mokilizingas and interest in lhv finance through as lhv pank as lhv finance was incorporated january as lhv group is a limited liability company incorporated in and domiciled in estonia the address of its registered office is tartu mnt tallinn as lhv group is a holding company whose subsidiary as lhv pank hereinafter the bank provides banking financial advisory and securities brokerage services to customers in estonia and in lithu ania there are offices for client servicing in tallinn tartu and vilnius will be closed at march as lhv finance the subsidiary of as lhv pank offers hire purchase services to merchants and consumer finance as lhv vara haldus offers fund management services for pension funds and other funds to clients in estonia uab mokilizingas offers consumer finance and hire purchase services in lith uania as at the group employed full time employees incl non active employees incl non active the consolidated annual report incl financial statements was approved by the management board on of february rain lõhmus who owns of the voting rights and andres viisemann who owns of the voting rights see also note have significant influence over as lhv group the consolidated annual report approved by the manage ment board shall be authorised for approval by the super visory board and shareholders the shareholders have the right not to approve the consolidated annual report while the supervisory board does not have that right notes to the consolidated financial statements note general information new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the principal accounting policies applied in the prepara tion of these consolidated financial statements are set out below these policies have been consistently applied to all the years presented and to all the consolidated group enti ties unless otherwise stated basis of preparation the consolidated financial statements of the group for the financial year have been prepared in accordance with international financial reporting standards ifrs as adopted by the european union the financial statements have been prepared under the historical cost convention except as disclosed in some of the accounting policies below such as financial assets and liabilities at fair value through profit or loss including derivatives and assets avail able for sale the preparation of consolidated financial statements in accordance with ifrs requires the use of certain critical accounting estimates it also requires management to exer cise its judgement in the process of applying accounting policies the areas involving a higher degree of judgement or complexity or areas where assumptions and estimates are significant to the financial statements are disclosed in note the financial year started at january and ended at december the financial figures have been presented in thousands of euros unless referred differently in specific disclosures certain new ifrs standards amendments to existing stand ards and the interpretations of the standards have been published by the time of compiling these financial state ments which became mandatory for the groups reporting periods beginning on or after january the overview of these standards and the potential impact of applying the new standards and interpretations are stated below a adoption of new or revised standards and interpretations the following new or revised standards and interpretations became effective for the group from january annual improvements to ifrss effective in the eu for annual periods beginning on or after february the improvements consist of changes to seven standards ifrs was amended to clarify the definition of a vesting condition and to define separately performance condition and service condition the amendment is effective for share based payment transactions for which the grant date is on or after july ifrs was amended to require disclosure of the judge ments made by management in aggregating operating segments including a description of the segments which have been aggregated and the economic indicators which have been assessed in determining that the aggregated segments share similar economic characteristics and a reconciliation of segment assets to the entitys assets when segment assets are reported group uses segmentation in everyday management from year all the internal measurement including forecasting profitability measure ment and following key performance indicators is done based on segments the segmentation is done in principle that there is no double accounting of incomes costs sum of segments has to be equal to reporting entity results disclosure initiative amendments to ias effective in the eu for annual periods beginning on or after january the standard was amended to clarify the concept of materiality and explains that an entity need not provide a specific disclosure required by an ifrs if the information resulting from that disclosure is not material even if the ifrs contains a list of specific requirements or describes them as minimum requirements the standard also provides new guidance on subtotals in financial statements in particular such subtotals a should be comprised of line items made up of amounts recognised and measured in accordance with ifrs b be presented and labelled in a manner that makes the line items that constitute the subtotal clear and understandable c be consistent from period to period and d not be displayed with more prominence than the subto tals and totals required by ifrs standards there are no other new or revised standards or interpreta tions that are effective for the first time for the financial year beginning on or after january that have a material impact on the group b new accounting pronouncements certain new or revised standards and interpretations have been issued that are mandatory for the groups annual periods beginning on or after january and which the group has not early adopted ifrs financial instruments effective for annual periods beginning on or after january the standard is endorsed by the eu commission earlier application is permitted but the group does not intend to early adopt the note summary of significant accounting policies new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee standard the group intends to use relief of not restating comparative figures for in the annual report when first adopting ifrs key features of the new standard are as follows financial assets are required to be classified into three measurement categories assets to be measured subsequently at amortised cost assets to be measured subsequently at fair value through other comprehensive income fvoci assets to be measured subsequently at fair value through profit or loss fvpl the classification and measurement requirements in ifrs state that financial assets should be classified as and meas ured at amortized cost fair value through profit and loss or fair value through other comprehensive income the classifi cation of a financial instrument is dependent on the business model for the portfolio where the instrument is included and on whether the cash flows are solely payments of principal and interest in order to assess the business model the group has divided its financial assets into portfolios and or sub portfolios based on how groups of financial assets are managed together to achieve a particular business objective classification for debt instruments is driven by the entitys business model for managing the financial assets and whether the contractual cash flows represent solely payments of principal and interest sppi if a debt instru ment is held to collect it may be carried at amortised cost if it also meets the sppi requirement debt instruments that meet the sppi requirement that are held in a portfolio where an entity both holds to collect assets cash flows and sells assets may be classified as fvoci financial assets that do not contain cash flows that are sppi must be measured at fvpl for example derivatives embedded derivatives are no longer separated from financial assets but will be included in assessing the sppi condition investments in equity instruments are always measured at fair value however management can make an irrevocable election to present changes in fair value in other comprehen sive income provided the instrument is not held for trading if the equity instrument is held for trading changes in fair value are presented in profit or loss most of the requirements in ias for classification and measurement of financial liabilities were carried forward unchanged to ifrs the key change is that an entity will be required to present the effects of changes in own credit risk of financial liabilities designated at fair value through profit or loss in other comprehensive income the analysis of the business model described above is still ongoing current results have not indicated signifi cant changes compared to how the financial instruments are measured under ias no significant impact is thus expected on the groups financial position financial perfor mance or equity in the period of initial application these conclusions are tentative and may be affected by the actual financial positions at the transition the impairment requirements in ifrs are based on an expected credit loss ecl model as opposed to the current incurred loss model in ias the scope of ifrs impair ment requirements is also broader than ias ifrs requires all assets measured at amortized cost and fair value through other comprehensive income as well as off balance commitments including guarantees and loan commitments to be included in the impairment test the assets to test for impairment will be divided into three groups depending on the stage of credit deterioration stage includes assets where there has been no significant increase in credit risk stage includes assets where there has been a significant increase in credit risk and stage includes defaulted assets significant assets in stage are tested for impairment on an individual basis while for insignificant assets a collective assessment is performed in stage the provisions should equal the month ecl in stage and the provisions should equal the lifetime ecl one important driver for size of provisions under ifrs is the trigger for transforming an asset from stage to stage the group is in process of deciding what parameters to use for identifying the increase in credit risk and how much these parameters need to change in order to constitute a significant increase the analyses and decisions should be finalized latest in june for assets to be recognized going forward changes to the lifetime probability of default pd will be used as the trigger in addition customers with forbearance measures and customers with payments more than thirty days past due will also be transferred to stage the group s current model for calculating collective provi sions defines a loss event as a deterioration in rating scoring but it is not expected that the loss event in the current model will equal the triggering event for moving items from stage to stage under ifrs the provisions under ifrs will be calculated as the exposure at default times the probability of default times the loss given default for assets in stage this calculation will only be based on the coming months while it for assets in stage will be based on the expected lifetime of the asset for assets where there has been a significant increase in credit risk the group currently holds provisions based on the losses estimated to occur during the period between the date when the loss event occurred and the date when new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the loss event is identified on an individual basis while ifrs will require provisions equal to the lifetime expected loss when calculating lifetime losses under ifrs including the staging assessment the calculation should be based on probability weighted forward looking information it is expected the newer requirements will increase the volatility in loan loss provisions and have negative effect in equity in the period of initial application it is not expected to have any material impact on large exposures the impact on capital adequacy is not possible to determine as it is expected the basel committee will issue new rules for the transiton to ifrs but these are not yet final it is furthermore expected that the long term effects once the transitional rules become obsolete will be negative on capital adequacy as the reduc tion in equity is expected to reduce cet capital impairment calculations under ifrs will require more experienced credit judgement by the reporting entities than required by ias today and a higher subjectivity is thus introduced the inclusion of forward looking information adds complexity and makes provisions more dependent on management s view of the future economic outlook it is expected that the impairment calculations under ifrs will be more volatile and pro cyclical than under ias mainly due to the significant subjectivity applied in the forward looking scenarios the main change to the general hedge accounting require ments is that the standard aligns hedge accounting more closely with the risk management activities these changes will not have direct effect on the group as the group is currently not using hedge accounting same time we see that the regulation changes will generate some new possi bilities to the group to start using hedge accounting for reducing volatility in financial statements for positions which are hedged with ones treated differently in accounting group has set up a project developing the needed changes both in risk management and in it systems the final outcome of the impact of the new standard on its finan cial statements is not clear yet and will be clear in second quarter still group is clear that there will be added new volatility to provisioning the provisions models will be more complex and visibility will somewhat lower we have calculated the impact to the cost of development and its around thousand ifrs leases effective for annual periods beginning on or after january not yet adopted by the eu the new standard sets out the principles for the recognition meas urement presentation and disclosure of leases all leases result in the lessee obtaining the right to use an asset at the start of the lease and if lease payments are made over time also obtaining financing accordingly ifrs eliminates the classification of leases as either operating leases or finance leases as is required by ias and instead introduces a single lessee accounting model lessees will be required to recognise a assets and liabilities for all leases with a term of more than months unless the underlying asset is of low value and b depreciation of lease assets separately from interest on lease liabilities in the income statement ifrs substantially carries forward the lessor accounting requirements in ias accordingly a lessor continues to classify its leases as operating leases or finance leases and to account for those two types of leases differently the group is currently assessing the impact of the new standard on its financial statements disclosure initiative amendments to ias effective for annual periods beginning on or after january not yet adopted by the eu the amended ias will require disclo sure of a reconciliation of movements in liabilities arising from financing activities the group is currently assessing the impact of the amendment on its financial statements ifrs revenue from contracts with customers effec tive for annual periods beginning on or after january the new standard introduces the core principle that revenue must be recognised when the goods or services are transferred to the customer at the transaction price any bundled goods or services that are distinct must be separately recognised and any discounts or rebates on the contract price must generally be allocated to the separate elements when the consideration varies for any reason minimum amounts must be recognised if they are not at significant risk of reversal costs incurred to secure contracts with customers have to be capitalised and amor tised over the period when the benefits of the contract are consumed the largest impact will be positive to the as lhv varahaldus where we need to capitalize sales costs from previous years and amortize over the lifetime of the customer which in pension fund case is above years this effect will significantly increase lhv varahaldus own funds at the moment of adoption of this ifrs standard the group is currently assessing the exact impact of the new standard on its financial statements revenue from contracts with customers amendments to ifrs effective for annual periods beginning on or after january not yet adopted by the eu the amendments do not change the underlying principles of the standard but clarify how those principles should be applied the amendments clarify how to identify a performance obligation the promise to transfer a good or new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee a service to a customer in a contract how to determine whether a company is a principal the provider of a good or service or an agent responsible for arranging for the good or service to be provided and how to determine whether the revenue from granting a licence should be recognised at a point in time or over time in addition to the clarifications the amendments include two additional reliefs to reduce cost and complexity for a company when it first applies the new standard the group is currently assessing the impact of the new standard on its financial statements there are no other new or revised standards or interpreta tions that are not yet effective that would be expected to have a material impact on the group consolidation the consolidated financial statements include the financial statements of as lhv group parent company and its subsidiaries as lhv pank as lhv finance subsid iary of as lhv pank as lhv varahaldus uab mokilizingas and oü cuber technology as lhv group holds share interest in uab mokilizingas and interest in lhv finance through as lhv pank subsidiaries are all entities over which the group has control the group controls an entity when the group is exposed to or has rights to variable returns from its involvement with the entity and has the ability to affect those returns through its power over the entity the acquisition method of accounting is used to account for the acquisition of subsid iaries except for acquisitions among enterprises under common control and business operations the cost of an acquisition is measured as the fair value of the assets given equity instruments issued and liabilities incurred or assumed at the date of exchange transaction costs incurred for issuing equity instruments are deducted from equity trans action costs incurred for issuing debt are deducted from its carrying amount and all other transaction costs associated with the acquisition are expensed according to the acquisi tion method the identifiable assets liabilities and contingent liabilities of the acquired subsidiary are recognized at their fair values at the acquisition date irrespective of the extent of any non controlling interest the group measures non controlling interest that repre sents present ownership interest and entitles the holder to a proportionate share of net assets in the event of liquidation on a transaction by transaction basis either at a fair value or b the non controlling interest s proportionate share of net assets of the acquiree non controlling interests that are not present ownership interests are measured at fair value goodwill is measured by deducting the net assets of the acquiree from the aggregate of the consideration transferred for the acquiree the amount of non controlling interest in the acquiree and fair value of an interest in the acquiree held immediately before the acquisition date any nega tive amount negative goodwill is recognised in profit or loss after management reassesses whether it identified all the assets acquired and all liabilities and contingent liabilities assumed and reviews appropriateness of their measurement intercompany transactions balances and unrealised gains on transactions between group companies are eliminated unrealised losses are also eliminated unless transaction provides evidence of an impairment of the asset transferred revenues and expenses of the subsidiaries acquired within the financial year are consolidated into the groups state ment of profit or loss and other comprehensive income starting from the date of acquisition to the end of the finan cial year result of operations of subsidiaries disposed of during the year are consolidated into groups statement of profit or loss and other comprehensive income from the beginning of the financial year until the date of disposal pursuant to the accounting act of the republic of estonia information of the unconsolidated financial statements primary statements of the consolidating entity parent company shall be disclosed in the notes to the consolidated financial statements in preparing the primary financial state ments of the parent company the same accounting policies have been used as in preparing the consolidated financial statements the accounting policy for accounting subsid iaries has been amended in the separate primary financial statements disclosed as supplementary information in the annual report in conjunction with las separate finan cial statements in the parent separate primary financial statements disclosed in these consolidated financial statements see note the investments into the shares of subsidiaries are accounted for at cost less any impairment recognized non controlling interest is that part of the net results and of the net assets of a subsidiary which is attributable to interests which are not owned directly or indirectly by the group non controlling interest forms a separate compo nent of the groups equity non controlling interest in the consolidated statement of financial position is disclosed separately from the equity attributable to the shareholders of the parent company in consolidated statement of profit or loss and other comprehensive income non controlling interest share of profit is disclosed separately from owner s of the parent new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee associates an associate is an entity over which the group has signifi cant influence but which it does not control generally signif icant influence is assumed to exist when the group holds between and of the voting rights the group does not have any associate as at at group had one associate svipe oü in which thw group hold of voting rights in the consolidated financial statements investments in associates are accounted for using the equity method under this method the investment is initially recognized at cost which is thereafter adjusted for post acquisition changes in the investors share of the investees equity changes both in the profit loss of the associate as well as other equity items and with elimination or depreciation amortisation of the differences between fair values and carrying amounts of the investees assets liabilities and contingent liabilities as determined in the purchase analysis unrealized gains on transactions between the group and its associates are elim inated to the extent of the groups interest in the associates unrealized losses are also eliminated unless the transaction provides evidence of impairment of the assets when the groups share of losses in the associate accounted under the equity method exceeds the carrying amount of the associate the carrying amount of the investment is reduced to zero and such long term loans granted to the associate that in substance form a part of the investment are written down further losses are carried off balance sheet when the group has guaranteed or incurred obligations on behalf of the associate the respective liability as well as the loss under the equity method are recorded in the statement of financial position other receivables from the associate are recorded at amortised cost less a provision for impairment an investment in the assets and liabilities of the acquired associate and goodwill that arose on acquisition is presented as a net amount in the statement of financial position line investments in associates at each balance sheet date it is assessed whether there is any indication that the recoverable amount of the invest ment has fallen below its carrying amount if any such indi cations exist an impairment test is performed to determine the recoverable amount of the investment the principles described in section are used foreign currency translation a functional and presentation currency the functional and presentation currency of entities in the group is euro b foreign currency transactions and balances foreign currency transactions are recorded based on the foreign currency exchange rates of the european central bank prevailing at the dates of the transactions monetary assets and liabilities denominated in foreign currencies are translated into euros based on the foreign currency exchange rates of the european central bank prevailing at the balance sheet date non monetary financial assets and liabilities measured at fair value denominated in foreign currencies are translated into euros based on the foreign currency exchange rates of the european central bank prevailing at the fair value measurement date gains and losses on translation from monetary assets and liabilities are recorded in the statement of profit or loss under the line foreign exchange rate gains losses translation differ ences on non monetary items such as equities held at fair value through profit or loss are reported as part of the fair value gain or loss in the statement of profit or loss under the line foreign exchange rate gains losses cash and cash equivalents for the purposes of the cash flow statement cash and cash equivalents comprise cash on hand demand deposits with central bank and other banks and investment companies term deposits with original maturities of three months or less that are available for use without any restrictions and which are subject to an insignificant risk of changes in value financial assets the group classifies its financial assets into the following categories financial assets at fair value through profit or loss held to maturity financial investments available for sale financial assets loans and receivables the classification depends on the purpose for which the financial assets were acquired management of the group determines the classification of its financial assets at initial recognition new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee financial assets in thousands of euros ias category class applied by the group loans and receivables loans and advances to banks and investment companies loans and advances to customers loans to legal entities loans to individuals other receivables other financial assets financial assets at fair value securities held for trading equity securities through profit or loss listed debt securities derivatives designated as at fair value through profit or loss upon equity securities initial recognition and fund units available for sale financial assets investment securities listed debt securities a financial assets at fair value through profit or loss financial assets at fair value through profit or loss include financial assets held for trading incl derivatives financial assets designated at fair value through profit or loss at inception a financial asset is classified as held for trading if acquired or incurred principally for the purpose of selling or repur chasing it in the near term derivatives are classified as held for trading unless they are designated as hedges regular way purchases and sales of financial investments are recognised at the trade date in the statement of financial position during the year the above mentioned trans actions were recognised at the settlement date in the state ment of financial postion this change in the accounting policies did not have any effect on the statement of financial position as there were no transactions as at which trade date was before balance sheet date and settle ment date after the balance sheet date financial instru ments included in this category are initially recognised at fair value transaction costs are taken directly to the profit or loss after initial recognition financial assets in this cate gory are measured at fair value interest income is calcu lated based on financial assets amortised cost using the effective interest rate method and is recorded within interest income in the statement of profit or loss changes in fair values of these financial assets are recognised consistently either as a profit or loss in the statement of profit or loss of the reporting period under net gains losses from financial assets measured at fair value in accounting for financial assets at fair value through profit or loss the change in the value of assets between the trade date and the settlement date are taken to profit or loss similarly to the recognition of similar assets in the ownership of the group dividend income on financial assets measured at fair value through profit or loss are recognised in the line dividend income of the statement of profit or loss when the right to receive dividends by the group is established in case of listed securities i e the securities which have an active market the current bid price is considered as the fair value of investments to estimate the fair value of invest ments not actively traded in the market alternative methods such as the price of recent transactions under market conditions the discounted cash flow method or option valuation models are used derivative financial instruments futures forward swap and option contracts are initially recognised in the statement of financial position at fair value at the trade date and are subsequently valued at fair value through profit or loss if derivatives are quoted on an active market market value is used as the fair value otherwise the valuation techniques are used to estimate the fair value gains and losses from derivatives are recognised as income or expense of the period in the statement of profit or loss under net gains losses from financial assets measured at fair value deriva tives are carried in statement of financial position as assets if their fair value is positive and as liabilities if their fair value is negative the fair values of derivative assets and liabilities recorded in the statement of financial position are not netted the group does not use hedge accounting to account for its derivative financial instruments financial assets designated at fair value through profit or loss at inception securities are classified into this group if the group has upon initial recognition designated the secu rities to be recorded as at fair value through profit or loss and as a result the changes in the fair value of these secu rities are consistently recognised in the profit or loss of the reporting period in the statement of profit or loss new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee financial assets and financial liabilities are designated at fair value through profit or loss when certain investments such as debt or equity investments are managed and evaluated on a fair value basis in accordance with a documented risk management or investment strategy and reported to key management personnel on that basis refer to note for fair value of financial assets b held to maturity financial investments held to maturity financial investments are financial instru ments quoted in an active market with a fixed due date and which the management board of the group has an inten tion and opportunity to hold until their maturity they do not include investments designated as fair value through profit or loss upon their initial recognition investments classified as available for sale assets investments which meet the criteria of loans and receivables these investments are initially recognised at fair value plus all directly attributable transaction costs incremental to their acquisitions and they are subsequently measured at amor tised cost using the effective interest rate method interest income on held to maturity investments is included within interest income in the statement of profit or loss impairment losses are deducted from the carrying amount of the invest ments and the impairment charge is recorded in the state ment of profit or loss c available for sale financial assets available for sale financial assets are financial instruments which the group intends to hold for an indefinite period of time and which may be sold in response to needs for liquidity or changes in interest rates exchange rates or equity prices investment securities available for sale are carried at fair value interest income on available for sale debt securities is calculated using the effective interest method and recognised in profit or loss other changes in fair values of these financial assets are recognised in other comprehensive income d loans and receivables loans and receivables are non derivative financial assets with fixed or determinable payments that are not quoted in an active market loans and receivables to customers are recorded when the group advances money to purchase or originate an unquoted non derivative receivable from a customer due on fixed or determinable dates and has no intention of trading the receivable loans and receivables are initially recognised at fair value plus transaction costs and are derecognised only when they are repaid or writ ten off after initial recognition the group recognises loans and receivables at amortised cost less principal repayments and any potential impairment losses if necessary and calculates interest income on the receivable in subsequent periods using the effective interest method in case of credit cards the borrowers actual use of limit is recorded in the statement of financial position finance lease transactions are lease transactions under which all significant risks and rewards from using the assets are immediately transferred from the group to the lessee legal ownership of assets is transferred to the customer at the end of the lease term finance leases are capitalised at the inception of the lease at the lower of the fair value of the leased property and the present value of minimum lease payments plus any unguaranteed residual value of the leased asset each lease payment received from the lessee is allocated between the receivable and finance income lease income is recognised over the term of the lease using the net investment method which reflects a constant peri odic rate of return when there are service fees and initial direct costs attributable to the conclusion of lease agree ments they are included in the calculation of the effective interest rate of the lease and calculation of a finance lease receivable the basis for assessing receivables is the timely fulfilment of contractual obligations the estimated market price of the leased asset and excess collateral the financial position and reliability of the customer impairment of receiv ables reduces the respective asset category in the state ment of financial position loans restructured because of significant changes in contract terms are recognised as new loans when a new loan has been issued incl refinancing an existing loan or increasing an existing loan for payment of the loan principal interest overdue penalties or other fees related to it loans which include a change in other contractual terms as a result of deterioration of the customers credit quality do not qualify for derecognition trade receivables arise from provision of services to the customers and are initially recognised at fair value plus transaction costs and subsequently measured at amortised cost using the effective interest method less repayments and a provision for incurred impairment losses the impairment of loans and advances is assessed in accordance with the principles described in clause impairment of financial assets carried at amortised cost impairment losses are recognised in profit or loss for the year when incurred as a result of one or more events loss events that occurred after the initial recognition of the finan new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee cial asset and which have an impact on the amount or timing of the estimated future cash flows of the financial asset or group of financial assets that can be reliably estimated if the group determines that no objective evidence exists that impairment was incurred for an individually assessed finan cial asset whether significant or not it includes the asset in a group of financial assets with similar credit risk characteris tics and collectively assesses them for impairment the group assesses the risks taking into consideration all known information about the solvency of the debtor and whether there are any objective circumstances indicating impairment the buyers financial difficulties bankruptcy or inability to meet its obligations to the group corporate loans overdrafts and leases are assessed indi vidually primarily based on the overdue status of loan and realisable of related collateral if any other criteria used to determine whether there is objective evidence that an impairment loss has occurred are financial position of the company industry situation reliability of the borrower the competence of its management and other factors margin and repo loans secured by debt or equity securities both to legal as well as private individuals are assessed individ ually using primarily the market value of collateral as the basis consumer loans mortgage loans credit card loans and hire purchase loans are assessed as a group private individuals are assessed in terms of their timely fulfilment of obligations solvency value of the collateral age educa tion length of employment savings habits and other factors impacting credit risk for the purposes of a collective evaluation of impairment financial assets are grouped on the basis of similar credit risk characteristics those characteristics are relevant to the estimation of future cash flows for groups of such assets by being indicative of the debtors ability to pay all amounts due according to the contractual terms of the assets being evaluated future cash flows in a group of financial assets that are collectively evaluated for impairment are estimated on the basis of the contractual cash flows of the assets and the experience of management in respect of the extent to which amounts will become overdue as a result of past loss events and the success of recovery of overdue amounts past experience is adjusted on the basis of current observ able data to reflect the effects of current conditions that did not affect past periods and to remove the effects of past conditions that do not exist currently the group regularly reviews future cash flow assessment methods and assump tions in order to reduce potential differences between loss estimates and actual losses if the terms of an impaired financial asset held at amor tised cost are renegotiated or otherwise modified because of financial difficulties of the borrower or issuer impair ment is measured using the original effective interest rate before the modification of terms the renegotiated asset is then derecognized and a new asset is recognized at its fair value only if the risks and rewards of the asset substantially changed this is normally evidenced by a substantial differ ence between the present values of the original cash flows and the new expected cash flows impairment losses are always recognised through an allow ance account to write down the assets carrying amount to the present value of expected cash flows which exclude future credit losses that have not been incurred discounted at the original effective interest rate of the asset the calcu lation of the present value of the estimated future cash flows of a collateralised financial asset reflects the cash flows that may result from foreclosure less costs for obtaining and selling the collateral whether or not foreclosure is probable any impairment losses are charged to statement of profit or loss line impairment losses on loans and advances if in a subsequent period the amount of the impairment loss decreases and the decrease can be related objectively to an event occurring after the impairment was recognised such as an improvement in the debtors credit rating the previ ously recognised impairment loss is reversed by adjusting the allowance account through profit or loss for the year the reversal of the impairment loss is recognised as income in the line impairment losses on loans and advances in the statement of profit or loss uncollectible assets are written off against the related impairment loss provision after all the necessary procedures to recover the asset have been completed and the amount of the loss has been determined subsequent recoveries of amounts previously written off are credited to impairment loss account in profit or loss for the year interest income on loans is presented on the statement of profit or loss under interest income discontinued operations a discontinued operation is a component of the group that is classified as held for sale and represents a separate major line of business or geographical area of operations and is part of a single co ordinated plan to dispose of a separate major line of business or geographical area of operations non current asset or disposal group is classified as held for sale when their carrying amount is to be covered princi pally through a sale transaction rather than through contin uing use and the sale is considered highly probable within months the disposal group is recognized at carrying amount or fair value less costs to sell depending on which is lower earnings and cash flows of discontinued operations if any are disclosed separately from continuing operations with comparatives being re presented intragroup trans actions between discontinued and continuing operations are eliminated depending on whether the transactions will continue after the sale transaction new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee tangible fixed assets tangible fixed assets are non current assets used in the operating activities of the group with a useful life of over one year an item of tangible fixed assets is initially recognised at its cost which consists of the purchase price incl customs duties and other non refundable taxes and other expendi tures directly related to the acquisition that are necessary for bringing the asset to its operating condition and loca tion subsequent costs are included in the assets carrying amount or are recognised as a separate asset as appro priate only when it is probable that future economic benefits associated with the item will flow to the group and the cost of the item can be measured reliably all other repairs and maintenance are charged to operating expenses during the reporting period in which they are incurred an item of tangible fixed assets is carried in the statement of financial position at cost less any accumulated deprecia tion and any accumulated impairment losses depreciation is calculated on a straight line basis the annual depreci ation rate for computers furniture and fixtures is for improvements of rental space either p a or amorti zation throughout the rental period depending on which is shorter depreciation is calculated starting from the month of acquisition until the carrying amount reaches the residual value of the asset in case the residual value is greater than the carrying amount of the asset no further depreciation expense is calculated the appropriateness of the assets residual values depre ciation methods used and useful lives are reviewed and adjusted if appropriate at each balance sheet date the group performs an impairment test when there are circum stances indicating impairment where an assets carrying amount is greater than its estimated recoverable amount higher of an assets fair value less costs to sell or its value in use it is written down immediately to its recoverable amount recognising an impairment loss in the statement of profit or loss for the reporting period gains and losses on disposals are determined by comparing proceeds with carrying amount these are included in oper ating expenses or other income in the statement of profit or loss for the reporting period intangible assets intangible assets are identifiable non monetary assets without physical substance and currently comprise of acquired software licences an intangible asset is initially measured at cost comprising of its purchase price and any directly attributable expenditure on preparing the asset for its intended use after initial recognition an intangible asset is carried at its cost less any accumulated amortisation and any accumulated impairment losses amortisation is calcu lated on a straight line basis the annual amortisation rate for purchased licenses is the amortisation method for client agreements is dimishing balance method the annual amortisation rate for purchased client agreements is of the residual value of those assets at each balance sheet date the appropriateness of amortization rates methods and residual values is assessed the group reviews intan gible assets for impairment losses whenever events or changes in circumstances indicate that the carrying amount may not be recoverable an impairment loss is recognised for the amount by which the carrying amount of the asset exceeds its recoverable amount which is the higher of an assets fair value less costs to sell and value in use financial guarantees and performance guarantees the group issues financial guarantees and commitments to provide loans financial guarantees represent irrevocable assurances to make payments in the event that a customer cannot meet its obligations to third parties and carry the same credit risk as loans financial guarantees and commit ments to provide a loan are initially recognised at their fair value which is normally evidenced by the amount of fees received this amount is amortised on a straight line basis over the life of the commitment except for commitments to originate loans if it is probable that the group will enter into a specific lending arrangement and does not expect to sell the resulting loan shortly after origination such loan commit ment fees are deferred and included in the carrying value of the loan on initial recognition at the end of each reporting period the commitments are measured at the higher of i the remaining unamortised balance of the amount at initial recognition and ii the best estimate of expenditure required to settle the commitment at the end of each reporting period performance guarantees are contracts that provide compen sation if another party fails to perform a contractual obliga tion such contracts do not transfer credit risk performance guarantees are initially recognised at their fair value which is normally evidenced by the amount of fees received this amount is amortised on a straight line basis over the life of the contract at the end of each reporting period the perfor mance guarantee contracts are measured at the higher of i the unamortised balance of the amount at initial recognition and ii the best estimate of expenditure required to settle the contract at the end of each reporting period discounted to present value where the group has the contractual right to revert to its customer for recovering amounts paid to settle the performance guarantee contracts such amounts will be recognised as loans and receivables upon transfer of the loss compensation to the guarantees beneficiary financial liabilities the group classifies financial liabilities to categories a financial liabilities measured at fair value through profit or loss b financial liabilities measured at amortised cost new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee recognition of a financial liability derivative at fair value through profit or loss is disclosed under accounting policy a all other financial liabilities are subsequently meas ured at amortised cost using the effective interest method transaction costs are included in the calculation of the effective interest rate the difference between the proceeds less transaction costs and redemption value are recog nised in the statement of profit or loss during the term of the instrument using the effective interest rate interest costs are included in the statement of profit or loss line interest expenses deposits from customers are initially recorded on their settlement date at their fair value less transaction costs and are subsequently measured at amortised cost using effec tive interest method in the statement of financial position line deposits from customers and loans received accrued interest liabilities are included in the same line interest expense is recognised in the statement of profit or loss line interest expense on the accrual basis loans received debt securities issued and similar subordi nated debts are initially recognised at fair value less trans action costs cash received less transaction costs the subordinated debts are those liabilities which in case of a liquidation of a credit institution or declaration of bankruptcy are settled after the satisfaction of the justifiable claims of other creditors other financial liabilities trade payables accrued expenses and other borrowings are initially recog nised at fair value less transaction costs and are subse quently carried at amortised cost payables to employees payables to employees include unpaid salary accruals and a vacation pay accrual calculated in accordance with employ ment contracts and the laws in force in employee residential country as at the balance sheet date the liability related to the payment of a vacation pay accrual together with social security and unemployment insurance payments is included within current liabilities in the statement of financial position and as staff costs in the statement of profit or loss social security tax includes payments to the state pension fund group has neither existing legal nor constructive obli gations to make pension or similar payments supplementary to social security tax share based payments as lhv group operates a share based compensation plan under which the company receives services from group employees as consideration for equity instruments options of lhv group the fair value of the employee services received in exchange for the grant of the options is recog nised during the shared based compensation program as groups staff expense and as an increase in the equity other reserves the total amount to be expensed is determined by the fair value at the time of issuing the options the fair value of the options granted including any market performance conditions influencing the share price e g lhv group share price excluding the impact of any service and non market performance vesting conditions for example profitability sales growth targets and remaining an employee of the entity over a specified time period at the end of each reporting period the group revises its estimates of the number of options that are expected to vest based on the vesting conditions other than market condi tions it recognises the impact of the revision to original estimates if any in the statement of profit or loss with a corresponding adjustment to equity when the options are exercised lhv group issues new shares the grant by lhv group of options over its equity instru ments to the employees of subsidiary undertakings in the group is treated as a capital contribution the fair value of employee services received measured by reference to the grant date fair value is recognised over the vesting period as an increase to investment in subsidiary undertakings with a corresponding credit to equity in the parent entity accounts according to the contractual conditions of the share options there are no social security tax charges when exercising the options after the year period provisions and contingent liabilities liabilities arising from an obligating event before the balance sheet date that have either a legal basis or that have arisen from the groups current operating practice legal or constructive obligation that require probable outflow of resources the realisation of which is probable it is more likely than not that an outflow of resources will be required to settle the obligation and the amount of which can reliably be estimated but for which the timing or amount are not defi nitely known are recognised as provisions in the statement of financial position the provisions are recognised based on the managements estimates regarding the amount and timing of the expected outflows a provision is recog nised in the statement of financial position in the amount which according to the management is necessary as at the balance sheet date for the meeting of the obligation arising from the provision or transfer to the third party the provision expense and an expense from the change in the carrying amount of provisions are included within expenses in the reporting period provisions are not set up to cover future operating losses when it is probable that a provision will be realised later than months after the balance sheet date it is recorded at discounted value present value of expected outflows new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee unless the discounting effect is immaterial other contingent liabilities guarantees except financial guarantees and other commitments the realisation of which is less probable than non realisation or related costs cannot be determined reliably that in certain circumstances may become obligations are disclosed in the notes to the finan cial statements as contingent liabilities contingent liabilities may develop in a way not initially expected therefore they are assessed continually to determine whether an outflow of resources embodying economic benefits has become probable on a portfolio basis if it becomes probable that an outflow of future economic benefits will be required for an item or portfolio of items previously dealt with as a contingent liability a provision is recognised in the financial statements of the period in which the change in probability occurs except in the extremely rare circumstances where no reliable estimate can be made distinction between short and long term assets and liabilities assets from which resources are expected to flow to the group within months are recognised as current assets the portion of assets with expected inflows later than months after the balance sheet date is recognised as non current assets see note liabilities are classified as current when they are due within twelve months after the balance sheet date or if the group does not have an unconditional right to defer the payment for later than months after the balance sheet date loans received with due date within months after the balance sheet date which are refinanced as non current after the balance sheet date but before the financial statements are authorised for issue are recognised as current loans received that the lender has the right to recall at the balance sheet date due to breaches of contractual terms are also classified as current for all long term assets and liabilities the long term portion is separately disclosed in respective disclosure to these financial statements see note revenues and expenses revenues and expenses are recorded on an accrual basis revenue is recognised when it is probable that the economic benefits associated with the transaction will be received by the group the amount of the revenue can be measured reliably and services were rendered by the group revenue earned in the ordinary course of business is recognised at fair value of the fee received or receivable expenses are recognised when the group has obligation to pay for the expense and or the group has received goods or services and the latter occurs at an earlier date fee and commission income incl custody and portfolio management fees are recognised on an accrual basis when the service has been provided and the group has a right of claim to the receivablecommissions and fees arising from provision of advisory services to third parties purchase or sale of businesses acquisitions etc are generally recognised upon completion of the underlying transaction or when service is provided over a longer period of time based on the stage of completion method portfolio management other consultation fees and management fees related to investment funds are recognised according to the agreement usually over the period during which the service is provided the same principle is applied to wealth management financial planning and custody services that are continuously provided over an extended period of time performance linked fees or fee components are recognised when the performance criteria are fulfilled or based on the stage of completion other one off service revenues and other revenues are recognised on an accrual basis at the moment of executing the respective transaction fee and commission expenses are recognised after the service has been received and when the liability has incurred interest income and expense is recognised in the state ment of profit or loss for all financial instruments carried at amortised cost using the effective interest method or for debt securities measured at fair value through profit or loss transaction costs of debt securities measured at fair value through profit or loss is recognised as part of effec tive interest method loan commitment fees that are directly related with the undrawn portion of the loan commitment and which changes based on the portion of the unused commitment are recognised as interest income during the period when draw down is available the effective interest method is a method of calculating the amortised cost of a financial asset or a financial liability and allocating the interest income or interest expense over the relevant period the effective interest rate is the rate that exactly discounts estimated future cash payments or receipts through the expected life of the financial instru ment to the net carrying amount of the financial asset or financial liability when calculating the effective interest rate the group estimates cash flows considering all contractual terms of the financial instrument but does not consider future impairment losses the calculation includes all signif icant fees paid or received between parties to the contract that are an integral part of the effective interest rate transac tion costs and all other premiums or discounts when a receivable is impaired the group reduces the carrying amount to its recoverable amount being the esti mated future cash flows discounted at the original effective interest rate of the instrument and continues unwinding the discount as interest income once a financial asset or a group of similar financial assets has been written down as a new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee result of an impairment loss interest income is recognised using the rate of interest used to discount the future cash flows for the purpose of measuring the impairment loss dividend income is recognised when the legal right to receive dividends is established and inflow of economic benefits are probable asset management services the group is engaged in providing asset management services note such assets have been given to the group to be managed by third parties and these assets are recorded off balance sheet service fees are derived from management of such assets and no associated credit and market risks arise for the group leases group as the lessee leases of tangible fixed assets where the lessee acquires substantially all the risks and rewards of ownership are clas sified as finance leases other leases are classified as oper ating leases payments made under operating leases are charged to the statement of profit or loss on a straight line basis over the lease term the group primarily uses an operating lease for renting the premises a rental expense is recognised in the statement of profit or loss as administrative and other oper ating expenses taxation and deferred income tax corporate income tax in estonia according to the income tax act the annual profit earned by entities is not taxed in estonia and thus there are no temporary differences between the tax bases and carrying amounts of assets and liabilities and no deferred tax assets or liabilities arise instead of the income tax payable on profits the estonian entities pay corporate income tax on dividends fringe benefits gifts donations costs of entertaining guests non business related disbursements and adjustments of the transfer price the tax rate from is until on net dividends paid the corporate income tax arising from the payment of dividends is recog nised as a liability and an income tax expense in the period in which dividends are declared regardless of the period for which the dividends are paid or the actual payment date an income tax liability is due on the day of the month following the payment of dividends corporate income tax in other countries in accordance with the local income tax laws the net profit of companies located in latvia lithuania and finland that has been adjusted for the permanent and temporary differ ences as stipulated by law is subject to corporate income tax corporate income tax rates latvia lithuania finland deferred income tax is calculated on all significant tempo rary differences between the tax bases of assets and liabili ties and their carrying values in the consolidated statement of financial position the main temporary differences arise from depreciation and tax loss carry forwards deferred tax balances are measured at tax rates enacted or substan tively enacted at the balance sheet date which are expected to apply to the period when the temporary differences will reverse or the tax loss carry forwards will be utilised deferred tax assets and liabilities are netted only within the individual companies of the group deferred tax assets for deductible temporary differences and tax loss carry for wards are recorded only to the extent that it is probable that future taxable profit will be available against which the deductions can be utilised deferred income tax is provided on temporary differences arising on investments in subsidi aries and associates except where the timing of the reversal of the temporary difference is controlled by the group and it is probable that the temporary difference will not reverse in the foreseeable future a deferred income tax liability in respect of the groups avail able equity which would accompany the payment of avail able equity as dividends is not reported in the statement of financial position the maximum amount of income tax payable which would arise from paying out the retained earnings as dividends is disclosed in the note to the financial statements offsetting financial assets and liabilities are offset and the net amount reported in the consolidated statement of financial position only when there is a legally enforceable right to offset the recognised amounts and there is an intention to either settle on a net basis or to realise the asset and settle the liability simultaneously more detailed information in note statutory reserve capital statutory reserve capital is formed from annual net profit allocations to comply with the requirements of the commer cial code during each financial year at least one twentieth of the net profit shall be transferred to the statutory reserve until the reserve reaches one tenth of share capital stat utory reserve may be used to cover a loss or to increase share capital payments to shareholders from statutory reserve are not allowed new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee risk is defined as a potential negative deviation from expected financial results the group encounters several risks in its day to day operations the objective of risk management at the group is to recognise these risks to measure them appropriately and to manage them more broadly the aim of risk management is to increase the value of the group by minimising losses and reducing the volatility of results risk management at the group is based on a strong risk culture and is built on the principle of three lines of defence where the first line of defence the business units are responsible for taking risk and for day to day manage ment the second line of defence the risk management division is responsible for the development of risk manage ment methodologies and for the reporting of risks the third line of defence the internal audit conducts independent supervision over the entire group risk management princi ples requirements and areas of responsibility are described in policies and procedures in accordance with the capital management principles that the group has adopted it must have adequate capital to support risks capital management the goal of the groups capital management is to ensure continuity of the groups business and ability to generate return for its shareholders maintain a strong capital base supporting the development of business comply with capital requirements as established by supervision authorities the amount of capital that the group managed as of was eur thousand eur thousand the goals of the groups capital management are set based on both the regulative require ments and additional internal buffer the group follows the general principles in its capital management the group must be adequately capitalized at all times having necessary capital to ensure economic preservation and enabling financing of new profitable growth opportunities the main focus of the capital management is on tier own funds because only tier own funds can absorb losses all other capital layers in use are dependent of tier own funds volume capital of the group can be divided into regu lated minimum capital and capital buffer held by the group in order to reach its long term economic goals the group must on one hand strive towards propor tional lowering of the regulated minimum capital through minimizing risks and high transparency on the other hand the group must strive towards sufficient and conservative capital reserve which will ensure economic preservation even in the event of severe negative risk scenario the risk appetite set by the group is an impor tant input to capital management planning and capital goal setting higher risk appetite requires maintaining higher capital buffer note risk management paid in share capital share premium statutory reserves transferred from net profit other reserves accumulated profit deficit intangible assets subtracted net profit for accounting period non controlling interests total tier own funds subordinated debt total tier own funds total net own funds own funds in thousands of euros new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the net capital of a credit institution at any time shall be equal to or exceed the minimum amount of share capital prescribed in the credit institutions act eur million in the capital of banks and investment firms in the euro pean union became subject to a new legal framework crd iv crr largely based on the basel iii framework that was agreed in the basel committee on banking supervision the objective of the new legal framework is to strengthen the resilience of the financial sector to adverse economic shocks and thereby ensure an adequate and sustainable financing of the economy significant changes implemented by the new requirements include the requirement for credit institutions to maintain a higher level and quality of capital than before and a unified framework for designing liquidity buffers the new capital requirements directive also define measures for macro financial supervision that member states can use to control the behaviour of credit institutions in amplifying the cycles and to alleviate risks arising from market structure the regulation which is directly applicable in the member states obligates all credit institutions operating in the euro pean union including their consolidating holding compa nies and investment firms to maintain of common equity tier cet and of tier capital against risk assets the overall capital adequacy requirement cad including both tier and tier capital remains at the existing level in addition to the main requirements based on uniform rules the directive defines the principles for forming capital buffers in estonia in addition to the baseline capital requirements credit institutions have been subjected to capital mainte nance and systemic risk buffers which are imposed by the financial supervisory authority and imposed by estonian central bank respectively as the aforemen tioned buffers are incremental to the tier and the total baseline capital requirements the minimum tier require ment in estonia is and the total capital requirement is the latter is also subject to the specific supple mentary pillar requirement applicable to credit institutions an overview of capital requirements is provided in the table below capital requirements cad base capital requirement capital conservation buffer systemic risk buffer capital requirements total capital reqirements valid till august are provided in the table below capital requirements cad base capital requirement capital conservation buffer systemic risk buffer capital requirements total each year the groups supervisory board confirms the goals of capitalization and the target level of capital adequacy to cover potential risks arising from fast growth each year an internal capital adequacy assessment process icaap is performed the goal of which is to identify potential capital needs in addition to regulatory capital requirements internal capital adequacy targets for are next tier capital adequacy total capital adequacy capital adequacy and the use of regulatory capital are monitored by the finance department reports about the compliance with prudential and capital requirements for covering the risks are submitted regularly to supervision authorities lhv uses standard method for calculating capital requirements for credit and market risk and basic indicator approach calculating operational risk capital requirement the own funds of lhv varahaldus as the fund manager need at any given time exceed the minimum amount of share capital laid down in the investment funds act eur million the net own funds of a fund manager who managed pension funds with the market value of over eur million had to be till january at least eur million and of the market value of managed pension funds which exceeds eur million in addition the fund manager had to maintain additional own funds equal to of the portion of the market value of all managed funds which exceeds eur million starting from january the net own funds of a fund manager have to be of the market value of managed pension funds with the market value less than eur million and of the market value of managed pension funds with the market value more than eur million the group has complied with all capital requirements during the financial year and in previous years credit risk credit risk is the potential loss which would arise from the counterpartys inability to meet its obligations to the group credit risk arises from cash and cash equivalents deriva tives and deposits with banks and other financial institutions debt securities but mostly credit exposures to customers including outstanding loans given guarantees other receiv ables and commitments in order to mitigate credit risk group analyses the operations and financial position of its customers and business partners after authorising the loan the solvency of the customer and the value of the collateral are regularly monitored new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee distribution of credit risks the group classifies the financial assets exposed to credit risk in the following key categories a debt securities and derivatives b loans and advances to central bank and credit institutions referred to as banks in the tables and investment companies c leveraged loans loans secured by debt or equity securities incl repo loans d corporate loans and overdraft e retail loans f consumer loans without collateral g credit cards and overdraft to individuals h leasing i hire purchase j mortgage loans k financial guarantees l unused loan commitments a debt securities and derivatives the credit committee sets limits for taking credit risk asso ciated with debt securities considering the issuers rating the risk capital committee or authorised employees make decisions regarding investments within the limits set no principal and accrued interest receivables arising from debt securities are overdue the group s debt securities at fair value through profit or loss fvtpl and available for sale afs according to ratings given by standard poors or equivalent maximum exposure to credit risk in thousands of euros loans and advances to banks and investment companies note available for sale financial assets note financial assets at fair value debt securities note loans and advances to customers note receivables from customers note other financial assets note total assets exposures related to off balance sheet items note excluding performance guarantees total maximum exposure to credit risk ratings distribution in thousands of euros aaa a to a bbb to bbb non rated total note investment portfolio foreign exchange forwards afs total liquidity portfolio trading portfolio fvtpl new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee ratings distribution in thousands of euros aaa aa to aa a to a bbb to bbb b to bb total note rating in thousands of euros central bank the bank of estonia aa to aa a to a bbb to bbb non rated total note investment portfolio total total credit institutions credit institutions afs total liquidity portfolio investment companies investment companies trading portfolio fvtpl b loans and advances to banks and investment companies management estimates that the credit risk exposure from cash and cash equivalents held at the central bank other correspondent banks and investment institutions has inher ently low credit risk no loans and advances to central bank credit institutions and investments companies are overdue the funds of the group according to ratings given by standard poors or equivalent central bank without a rating in case there are ratings available from more than one rating agency the most conservative rating is used the positions are held as follows c leveraged loans the bank issues margin loans secured by debt or equity securities to its customers i e financial leverage the maximum amount of the loan depends on the market value of the assets held as collateral in the investment account and on the general limit set by the bank which is currently eur thousand or an equivalent in a foreign currency per customer granting loans above the threshold assumes an analysis of the portfolio offered as collateral by the customer and the decision is within the competence of the credit committee the list of acceptable marketable finan cial instruments and the levels of the required collateral are published on the banks website www lhv ee the risks arising from financial leverage are mitigated by constant monitoring of the market values of the financial instru ments required as collateral the group has set sufficiently conservative limits to the ratio of the loan and the collat eral assets value when the value of collateral assets falls below the established limit the bank is entitled to demand a transfer of additional collateral to the account or pay off the loan in the customers account without the customers trade order liquidating the collateral asset for this purpose see note for more detailed information on the credit quality of loans stress tests are carried out for evaluating the credit risk of leveraged loans in order to determine potential losses in case of changes in the value of collateral and for evaluating the credit risk of other loans the probability of the lender becoming insolvent the amount of loans not covered by collateral at the time of insolvency and the amount of the resulting related expected loss is assessed if necessary decisions are made in respect of allocating additional risk capital in order to cover credit risk the bank had no impaired leveraged loans as at and d corporate loans and overdraft corporate loans are evaluated on individual basis each customer is assigned a credit rating between where means the lowest probability of default and indicates insolvency credit rating is assigned to a customer consid ering combination of financial status and business risk new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee financial status is the basis for financial rating focusing on the companys assets and liabilities structure profitability cash flows and estimated results the emphasis of financial analysis is focused on the following period to the loan issu ance in parallel with financial analysis the companys busi ness risk including management and owners market situa tion and competition diversification and history of activities the quality of information and previous payment behaviour is analysed where there is consolidation group involved credit risk for the whole group is evaluated credit rating is assigned by credit analyst and confirmed by head of credit but final decision of risk taking is the unanimous decision by the credit committee the requirements for loan collateral are established in the credit policy of group and specified in credit risk manage ment rules of the bank the preferred collateral is where there is no strong correlation between the clients default risk and value of the collateral in general the pledged assets need to be secured the life of the collateral needs to be longer than the due date of the loan and the market value of the collateral needs to exceed the outstanding loan balance in addition to individual impairment assessment corpo rate loans provisions for potential credit loss is calculated based on historical performance of these loans and applied to current portfolio at the balance sheet date for the purpose of recognition of group based impairment losses the receivables are grouped into subclasses on the basis of homogeneous credit risk features considering customers payment practice and past due time for the receivables that have been grouped the amount of the impairment loss is the multiple of the carrying amount of the receivables in the group and the groups percentage rate of impairment loss the framework is based on the method of calculating the probability of default pd loss given default lgd and exposure at default ead ead consists of two compo nents the amount of the existing receivable and the amount of use of an estimated additional limit for calculating the latter the model includes the credit conversion factor ccf after issuing the loan follow up monitoring of each custom ers financial position is performed at least once a quarter all client ratings are reviewed at least once a year see note for more detailed information on the credit quality of loans as at the group based impairment reserve makes up of corporate loans and overdraft and the related interest receivables e retail loans in group started offering micro loans the loan is aimed to micro enterprices in growth stage and the maximum loan amount is up to eur during the year the existing corporate loan portfolio was divided into two credits below thousand euros were given over to retail banking and all credits exceeding thousand euros remained in corporate banking shifting smaller loans to retail banking was impor tant to keep corporate banking and credit analyses focus on larger and more individual approach requiring credit clients credits below thousand euros are analysed with more cost efficient scoring process the scoring is carried out at the request of the loan and it is one of the criterias for issuing the loan financial data and information on payment behav iour of the company is used when calculating the proba bility of default pd the structure of assets and liabilities are taken into account also the profitability and cash flow ratios regarding the guarantees the same principles apply as for business loans and overdrafts thus the essence of it does not constitute a new product but only retail loans separa tion from corporate loans in distinction then there is no comparative information as at december the retail loans are included inside loans to legal entities and overdrafts to legal entities lines in the report f consumer loans without collateral as consumer loans are homogeneous loans provisions for potential credit loss level is calculated based on historical performance of these homogeneous loans and applied to current portfolio at the balance sheet date for the purpose of recognition of group based impairment losses the receiv ables are grouped into subclasses on the basis of homoge neous credit risk features considering customers payment practice past due time and the time passed from initiation of proceedings by the bailiff in estonia the group has entered into agreements where overdue consumer loans are sold usually when loans reaching overdue of days similar contract was signed also in mokilizingas in lithuania in third quarter for the receivables that have been grouped the amount of the impairment loss is the multiple of the carrying amount of the receivables in the group and the groups percentage rate of impairment loss the framework is based on the method of calculating the probability of default pd loss given default lgd and exposure at default ead ead consists of two components the amount of the existing receivable and the amount of use of an estimated additional limit for calculating the latter the model includes the credit conversion factor ccf consumer loans are classified as performing less than days overdue and are not terminated non performing more than days overdue and or terminated and doubtful orig inal loan schedule has expired and there has been no cash flows received for continuous period of months fraud case etc loss rates are calculated from the past histor ical data for defaults and full history of recoveries in lithu new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee ania due to default rate volatility and uncertain data quality a conservative buffer of shall be imposed for portfolio as at buffer of was imposed in the new methodology valid since buffer for lithuanian branch will be and only for latvian branch and credit cards buffer will remain pd and lgd values are updated at least once a year with new historical data in order to maintain adequate provision levels on portfolio if necessary additional impairments will be made in the statement of financial position if non per forming or doubtful loan recovery is neither economical nor practical to continue the loan is written off as irrecoverable as at the group based impairment reserve makes up of consumer loans and the related interest receivables g credit cards the bank issues credit cards and partner credit cards in cooperation with tallinna kaubamaja similarly with consumer loans the credit card portfolio uses the credit scoring model to assess the customers credit behaviour the scoring is made at the time the application is filed and is one of the criteria for issuing a loan provisions for potential credit losses are made based on the same framework as the consumer loans the framework is based on pd lgd and ead indicators ead consists of two components the amount of the existing receivable and the amount of use of an estimated additional limit for calcu lating the latter the model includes the credit conversion factor ccf as of december the group based impairment reserve amounted to of credit card loans and related receivables h leasing the bank offers leasing products for individuals and legal entities the creditworthiness of customers is assessed by using scoring models which mainly consists of charac teristics described for consumer loans the leasing provi sioning is done based on leasing scoring models as of december the group based impairment reserve amounted to of leasing portfolio also in lithuania leasing products are offered to individuals as well as legal entities the creditworthiness of customers is assessed by using scoring models which mainly consists of characteristics described for consumer loans provision for finance lease is calculated using following methodology net book value of portfolio minus net present value of portfolio net present value is calculated from future cash flows discounted by effective interest rate and adjusted for coefficient of possible losses in addition to homogenous impairment some problematic loans are assessed individu ally based on the market distressed sale value of the under lying assets as of december the group based impairment reserve amounted to of leasing portfolio i hire purchase group offers hire purchase service for merchants through its subsidiary lhv finance in estonia the creditworthiness of customers is assessed by using scoring models which mainly consist of characteristics described for consumer loans similarly to other homogenous portfolios provisions are made based on the same framework as consumer loans as of december the group based impair ment reserve amounted to of hire purchase portfolio the group offers hire purchase to private individuals in lithuania the creditworthiness of customers is assessed and provisions are made based on the same framework as for lithuanian consumer loans as of december the group based impairment reserve amounted to of hire purchase portfolio j mortgage loans in group started to offer on a limited basis mortgage loans to customers in the tallinn and tartu region only in autumn group started offering the mortgage loan to a large scale of retail clients the creditworthiness of customers is assessed by using scoring models and maximum loan amount is in line with the regulations set by central bank of estonia the provisions are made based on the same framework as for consumer loans as of december the group based impair ment reserve amounted to of mortgage portfolio new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee credit quality of loans and advances to customers and off balance sheet liabilities loans to legal entities corporate loans retail loans overdraft hire purchase leveraged loans leasing credit card loans loans to individuals consumer loans hire purchase leveraged loans leasing mortgage loans credit card loans overdraft total note loans to legal entities corporate loans overdraft hire purchase leveraged loans leasing credit card loans loans to individuals consumer loans hire purchase leveraged loans leasing mortgage loans credit card loans overdraft total note loans and advances to customers in thousands of euros loans and advances to customers in thousands of euros neither past due nor impaired neither past due nor impaired past due but not impaired past due but not impaired collective impairment collective impairment individual impairment individual impairment net net individually impaired individually impaired total total there are several criteria for the loan to be considered indi vidually impaired groups internal default definition is in line with the generally accepted definition of default criteria such as payment delay more than days on any material credit obligation distressed restructuring of the credit obliga tion specific credit adjustment due to significant perceived decline in credit quality since inception significant financial difficulties of the obligor or likely bankruptcy process etc as part of risk analysis the group is regularly performing stress tests and sensitivity analysis regarding credit risk and its components such as pd lgd collective impairment new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee credit assessment of the group is based on historical loss rate and credit rating the group has performed stress test scenarios when pd and lgd estimations will both increase by percent for example from to the impact of the described stress test to impairments is aggregated in the table below the table includes loans which have collec tive impairment therefore excluding leveraged loans and which have material balances and potential impact loans to legal entities corporate loans incl overdraft leasing loans to individuals consumer loans hire purchase mortgage loan leasing credit card loans total impact to impairment as at in case pd and lgd levels will increase by in thousands of euros balance as at impairment with increased pds and lgds impact to impairment booked loans to legal entities corporate loans incl overdraft leasing loans to individuals consumer loans hire purchase mortgage loan leasing credit card loans total impact to impairment as at in case pd and lgd levels will increase by in thousands of euros balance as at impairment with increased pds and lgds impact to impairment booked the rating scale used for evaluating the pd of corporate loans consists of a total of credit ratings where stands for the lowest and for the highest credit risk ratings and are attributed only on the basis of average ratings of rating agencies fitch moodys and standard poors and are subject to the groups financial analysis as neces sary thus ratings and are attributed only to international enterprises organisations local governments and countries whose solvency has been confirmed by the rating agencies with their investment grade rating also rating classes are partly based on the evaluation of rating agencies although this category includes mainly estonian enterprises that do not have an international rating distribution of internal ratings of corporate loans minimum credit risk the rating is only assigned on the basis of the ratings of rating agencies the average of the ratings assigned by rating agencies must be the equivalent of at least aaa moodys aaa minimum credit risk the rating is only assigned on the basis of the ratings of rating agencies the average of the ratings assigned by rating agencies must be the equivalent of at least aa moodys low credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least aa moodys low credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least a moodys low credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least bbb moodys low credit risk the rating is assigned on the new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least bbb moodys average credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least bbb moodys average credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least bb moodys heightened credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equiv alent of at least bb moodys high credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least b moodys high credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least b moodys non satisfactory rating the rating is assigned on the basis of the assessments of rating agencies or lhv the credit analyst deems the companys finan cial position to be sufficiently weak indicating a very high probability of default the average rating of rating agencies must be the equivalent of at least c moodys the obligor is in default low credit risk low credit risk low credit risk medium credit risk medium credit risk heightened credit risk high credit risk high credit risk non satisfactory rating insolvent total low credit risk low credit risk low credit risk medium credit risk medium credit risk heightened credit risk high credit risk high credit risk non satisfactory rating insolvent total distribution of corporate loans and overdraft by internal ratings in thousands of euros distribution of corporate loans and overdraft by internal ratings in thousands of euros neither past due nor impaired neither past due nor impaired past due but not impaired past due but not impaired individually impaired individually impaired total total the group considers the loan as non performing and assesses impairment when the loan payments have not been collected by the due date and or the expected cash flows from the realization of collateral are not sufficient for covering the carrying amount of the loan principal and interest payments the group evaluates these loans indi vidually in case of individual evaluation the amount of the impairment loss is the difference between the carrying value of the receivable and expected cash flows discounted at the effective interest rate in the group provisioned new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee corporate loans in the total amount of eur thousand eur thousand loan customers with ratings of are subject to more frequent monitoring during which the solvency of the borrower is evaluated in case of a deterioration of payment behaviour the requirement for individual provisioning is eval uated considering among other factors the collateral of the loan therefore some loans are not individually impaired the ratings based rate of the collective provisioning is applied to those loans in addition to loans issued loan agreements have been concluded and signed for the unused loan commit ment amount of thousand euros at thousand euros see note in the following table there are presented unused portions of corporate loans and financial guarantee limits no provisions have been made for off balance sheet liabilities because the value of collateral exceeds the value of potential liabil ities in addition to that as at the group has also committed to grant mortgage loans hire purchase loans consumer loans overdraft loans and credit card loans to individuals in total amount of eur thousand eur thousand all uncommissioned loans available to individuals have at least good internal rating as at the end of each reporting period distribution of internal ratings for other loan products excellent leveraged loans secured by debt or equity securities and loans with very low business risk very good loans with lower business risks and no past due payments good loans with lower business risks and past due payments up to days satisfactory loans with average business risk and up to days past due weak or doubtful loans carrying higher busi ness risk past due more than days and portfolio in proceedings by the bailiff low credit risk low credit risk medium credit risk medium credit risk heightened credit risk high credit risk high credit risk total credit quality of off balance sheet liabilities unused loan commitments for corporate loans and financial guarantees in thousands of euros neither past due nor impaired excellent good and very good past due but not impaired good satisfactory weak or doubtful individually impaired good satisfactory weak or doubtful total as of in thousands of euros retail loans leveraged loans consumer loans hire purc hase over draft mortgage loans total credit cards leasing new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee neither past due nor impaired excellent good and very good past due but not impaired good satisfactory weak or doubtful individually impaired good satisfactory weak or doubtful total listed securities unlisted equity securities mortgages real estate guarantee of kredex and rural development foundation pledges of rights of claim deposits leased assets others unsecured loans total listed securities unlisted equity securities mortgages real estate guarantee of kredex and rural development foundation pledges of rights of claim deposits leased assets others unsecured loans total as of in thousands of euros loans against collateral as at in thousands of euros loans against collateral as at in thousands of euros leveraged loans consumer loans credit cards consumer loans overd raft to private individuals credit cards consumer loans overd raft to private individuals hire purc hase leasing leasing over draft hire purchase hire purchase mortgage loans mortgage loans mortgage loans total total total credit cards corporate and retail loans including overdraft corpo rate loans including overdraft leasing leveraged loans leveraged loans in the table below collateral information of loans and advances to customers are disclosed based on the collat eral type and the fair value of collateral held the under col lateralised amount is presented as unsecured loans new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee unsecured loans in the years of and include credit card loans and lithuanian consumer loans collaterals for leveraged loans are monitored on daily basis and if collateral value is falling immediate measures are taken to avoid credit losses as of december and as of december all leveraged loans and repur chase loans are over collateralized consumer loans and credit card loans are issued without collateral and risk miti gation is done by regular monitoring of clients payment behaviour leasing hire purchase mortgage loans and overdraft to private individuals are all over collateralized the group monitors customers in arrears of leasing hire pur chase mortgage loans and overdraft to private individuals on regular basis in relation to under collateralized corporate loans it should be taken into consideration that the group has assessed the market value of certain collaterals conservatively personal sureties commercial pledges under collateralized loans are mainly considered to bear higher risk for which the group carries out monthly monitoring in credit committee in order to mitigate potential credit losses under collateralized loans include contracts more than days overdue totalling eur thousand eur thousand euros with a collateral value of thousand eur thou sand euros corporate loans and corporate credit lines in thousands of euros as at as at fair value of collateralcarrying value fair value of collateral over collateralized loans under collateralized loans carrying value structure of past due but not impaired loans according to past due time loans which have overdue interest or principal payments as at the balance sheet date are divided in past due categories according to the past due time from the earliest outstanding payment structure of individually impaired loans according to past due time loans which have overdue interest or principal payments as at the balance sheet date are divided in past due categories according to the past due time from the earliest outstanding payment past due receivables days days days days days more than days total past due receivables days days days days days more than days total as at in thousands of euros as at in thousands of euros corporate loans incl overdraft corporate loans incl overdraft credit cards credit cards leasing leasing hire purchase hire purchase consumer loans consumer loans total total retail loans leveraged loans leveraged loans new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee no past due payments past due receivables days days days days days more than days total no past due payments past due receivables days days days days days more than days total credit quality of other receivables in thousands of euros receivables neither past due nor impaired receivables past due but not impaired incl receivables from individuals incl receivables from legal entities total note as at in thousands of euros as at in thousands of euros corporate loans incl overdraft corporate loans incl overdraft credit cards credit cards leasing leasing hire purchase hire purchase consumer loans consumer loans total total retail loans leveraged loans leveraged loans as of february other receivables of eur thou sand euros are uncollected and in previous years the group has not written off any receivables other financial assets see note in amount eur thousand eur thousand are guarantee deposits on the baltic stock exchanges held to guarantee securities trading activity on the stock exchanges of tallinn riga and vilnius and the deposits of visa and mastercard to guarantee credit card transactions the group considers the counterparties credit risk as very low market risk market risk arises from the groups trading and investment activities in the financial markets from interest rate products foreign exchange and stock markets as well as lending activities and collecting financial resources market risk is a potential loss which may arise from unfavourable changes in foreign exchange rates prices of securities or interest rates internal judgement is used to assess potential losses in order to mitigate market risk conservative limits have been established for the trading portfolio and open foreign currency exposures lhv varahaldus invests most of its available resources into his own managed pension funds the management of lhv varahaldus is responsible for monitoring of the market risk foreign currency risk foreign currency risk may arise from acquisition of secu rities mostly denominated in foreign currencies or foreign currency receivables and liabilities the treasury of the bank is responsible for daily monitoring of open foreign currency positions the groups foreign currency risk management is based on risk policies limits and internal procedures new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the internal limits for open foreign currency positions are proposed by risk control unit and treasury and the limits are set internally by risk and capital committee the limits set by the committee are maximum nominal limits in euro equivalent with a respect to the groups net own funds limits imposed on individual currencies are in line with the boundaries proposed by the bank of estonia if the open currency position exceeds the limits set by the committee measures are immediately implemented to close or reduce such positions hedging the risk with relevant instruments such as foreign currency forwards or futures a sensitivity analysis has been performed for the effect of possible reasonable changes attributable to open currency positions on statement of profit or loss with the assumption of other conditions remaining constant impact on statement of profit or loss in thousands of euros usd exchange rate sek exchange rate gbp exchange rate chf exchange rate open currency exposures the following tables present the risks arising from open currency exposures assets and liabilities denominated in foreign currencies have been presented in euro equiva lent in respective columns according to the exchange rate prevailing at the balance sheet date derivatives reported at fair value in the statement of financial position have been included at contractual amounts under off balance sheet assets and liabilities open currency exposure and the volume of financial assets and liabilities of the group at the balance sheet date do not significantly differ from the average exposure during the year the balances of total assets and total liabilities bearing currency risk do not include currency futures at their fair value but they are shown here at their full contractual cash flow amounts as off balance sheet assets and liabilities also the table does not include the assets tangible and intangible assets and liabilities provisions not bearing currency risk and equity in thousands of euros eur chf gbp sek usd other total assets bearing currency risk due from banks and investment companies note financial assets at fair value note loans and advances to customers note receivables from customers note other financial assets note total assets bearing currency risk liabilities bearing currency risk deposits from customers and loans received note financial liabilities at fair value note accounts payable and other financial liabilities note subordinated debt note total liabilities bearing currency risk open gross position derivative assets at contractual value open gross position derivative liabilities at contractual value open foreign currency position new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros eur chf gbp sek usd other total assets bearing currency risk due from banks and investment companies note financial assets at fair value note loans and advances to customers note receivables from customers note other financial assets note total assets bearing currency risk liabilities bearing currency risk deposits from customers and loans received note financial liabilities at fair value note accounts payable and other financial liabilities note subordinated debt note total liabilities bearing currency risk open gross position derivative assets at contractual value open gross position derivative liabilities at contractual value open foreign currency position price risk financial instruments bearing price risk at group are secu rities held in the trading portfolio and investment portfolio note at the group limits are set for the size of the trading portfolio and acceptable credit quality ratings are specified for debt securities in the investment portfolio the risk management unit monitors the compliance with limits pursuant to the investment funds act the mandatory shares of lhv varahaldus as the management company is of the number of units in each of the mandatory pension fund managed by it based on historical performance and volatility of the pension fund unit nav changes the price changes should remain in the next month period with the most likely change and the impact to state ment of profit or loss is presented in the next table lhv varahaldus invests the remaining available assets in euros to ensure liquidity and to not bear investment risk bank does not hold significant amounts of equity securities in its position see note due to which the sensitivity to change in the market value of these positions is marginal banks debt securities portfolio recognised at the market price is short term and of high quality therefore the effect of the changes in market risk premiums on the market value of the debt security portfolio should remain at around sensitivity analysis of the impact to net result from the risk exposures of the groups largest entity as lhv pank against reasonable possible change in thousands euros impact on statement of profit or loss in thousands of euros equity securities mandatory pension fund units debt securities fvtpl impact on other comprehensive income debt securities afs interest rate risk interest rate risk reflects the mismatch in the statement of financial position items and the off balance sheet items when interest rate repricing periods volumes or the under lying interest rate of assets liabilities and derivatives do not correspond exactly groups interest rate risk manage ment is based on risk policies limits and internal proce dures resulting in identification of all significant sources of interest rate risk and balanced risk taking interest rate risk is measured through various scenario analyses by calculating several net interest income scenarios and comparing the difference between these scenarios the basic measures of interest rate risk uses two scenarios an increase and new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee a decrease of interest rates to measure the effect on the groups net interest income for a month period internal limits for interest rate risk management are set by as lhv panks risk and capital committee and the operational risk management is the treasurys responsibility the goal of monitoring measuring and managing interest rate risk is to evaluate the profitability of the groups inter est bearing products to forecast interest income and to set limits for risk management in order to prevent signifi cant reduction of income through limitation of risks in three aspects cash flow interest rate risk whereby in case of a change in market interest rates a change in annual net interest income is limited as compared to the esti mated actual income based on the term structure of instru ments currently in the statement of financial position fair value interest rate risk whereby in case of a change in market interest rates a change in the groups economic value is limited fair value of assets and liabilities is determined by discounting the future cash flows at the market rate of interest the balance sheet and margins on assets and deposits are assumed to be constant over time the deposits interest rates did not change in remaining at the level of up to up to in the interest rates for leveraged loans granted are changed at most once a month according to fluctuations in market interest rates in the interest rate on loans received for specific purposes was the effec tive interest rate of subordinated debts entered into in was and the effective interest rate of subordinated debts entered into in and entered into in was the information about subordinated debt contractual interest rates is provided in note as at stress tests were performed to the biggest interest sensitive subsidiary of the group as lhv pank an increase of percentage point in interest rates would affect the banks annual net interest income and profit by eur thousand eur thou sand in the same time a decrease of percentage point in interest rates would affect the banks annual net interest income profit by eur thousand eur thousand a percentage point increase in market interest rates would raise the banks economic value i e equity by eur thousand eur thousand a percentage point decrease in market interest rates would decrease the banks economic value equity by eur thousand eur thousand effect on the banks economic value is positive due to the fact that the bank has majorly invested in current assets and because of the prolonged nature of demand deposits hence the average duration of assets exposures to interest rate risk is shorter than the average duration of liabilities exposures to interest rate risk the table below shows the structure of the financial assets and liabilities exposures to interest rate risk of the group according to the recalculation dates of interest rates at the principal amounts of receivables and liabilities leveraged loans are treated as a one month product maturing at the next interest fixing date demand deposits are divided as follows liquidity accounts with the duration of months and other demand deposits with the duration of years based on their behav ioural nature the interest rate of demand deposits is not sensitive to market rate fluctuations the prices of derivative contracts on the market are the bases for the assumption that the interest rates of the time deposits will not change drastically during the next years which would cause a significant amount of the demand deposits to transfer to term deposits financial assets due from banks and investment companies note financial assets at fair value debt securities note loans and advances to customers note total financial liabilities deposits from customers and loans received note subordinated debt note total net interest sensitivity gap exposures to interest rate risk in thousands of euros up to months over year subtotal accrued interest impair ments total months years new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee financial assets due from banks and investment companies note financial assets at fair value debt securities note loans and advances to customers note total financial liabilities deposits from customers and loans received note subordinated debt note total net interest sensitivity gap up to months over year subtotal accrued interest impair ments total months years liquidity risk liquidity risk relates to the solvency of the group to meet its contractual obligations on time and it arises from differences between maturities of assets and liabilities the groups liquidity management and strategy is based on risk policies resulting in various liquidity risk measures limits and internal procedures as per policy statements the groups liquidity management reflects a conservative approach towards liquidity risk the liquidity risk management includes stress testing and business continuity plan for liquidity manage ment stress testing framework includes a survival period metrics which represents a combined liquidity risk scenario including both idiosyncratic and market wide stress internal metrics are complemented by the basel iii metrics the liquidity coverage ratio lcr and the net stable funding ratio nsfr which the group is fully compliant with as of and the aim of the lcr standard is to ensure that the group has an adequate amount of unencumbered assets of high quality and liquidity that could be monetised without incurring material losses to meet a liquidity requirement in a day stress scenario the output of the standard is the ratio of liquid assets to stress scenario liquidity requirement which has a limit of the groups liquidity coverage ratio level as at was the objective of the nsfr is to ensure that the group has adequate stable funding sources to finance longer term assets according to the proposals published in the banking regulation and the banking committee of basel in october the groups nsfr level as at was the net stable funding ratio will be imposed as minimum requirement from the year the treasury of the bank is responsible for the manage ment of liquidity risk in order to hedge liquidity risk the probable net position of receivables and liabilities by matur ities is regularly monitored and adequate amounts of liquid assets are kept in each time period also the concentration of the groups liabilities by maturities is monitored to enable covering unexpected monetary outflows the group holds a liquidity buffer the liquidity buffer consists of cash and deposits with the central bank and liquid securities held by the treasury which can be readily sold or used as a collat eral in funding operations with the central bank the group has a sufficient supply of liquid resources to enable issuing standby loans as at and the group does not have any debts past due short term loans can be obtained from the central bank secured by the majority of instruments from the debt secu rities portfolio the following tables present the distribution of financial assets and liabilities excl derivatives by due dates and by future contractual undiscounted cash flows and in a sepa rate column the statement of financial position balance is disclosed in the maturity analysis the cash flows are split into the maturity buckets in which the cash flows occur including interest cash flows explanation of the fair value of these financial assets and liabilities is presented in note exposures to interest rate risk in thousands of euros the contractual term of subordinated debts received in and in is years and the interest rate is fixed at the contractual term of subordinated debts received in is years and the interest rate is fixed at new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee liabilities by contractual maturity dates deposits from customers and loans received note subordinated debt note accounts payable and other financial liabilities note unused loan commitments note financial guarantees by contractual amounts note foreign exchange derivatives gross settled financial liabilities at fair value note total liabilities assets held for managing liquidity risk by contractual maturity dates due from banks and investment companies note financial assets at fair value debt securities note loans and advances to customers note receivables from customers note other financial assets note foreign exchange derivatives gross settled total assets held for managing liquidity risk maturity gap from assets and liabilities liabilities by contractual maturity dates deposits from customers and loans received note subordinated debt note accounts payable and other financial liabilities note unused loan commitments note financial guarantees by contractual amounts note foreign exchange derivatives gross settled financial liabilities at fair value note total liabilities assets held for managing liquidity risk by contractual maturity dates due from banks and investment companies note financial assets at fair value debt securities note loans and advances to customers note receivables from customers note other financial assets note foreign exchange derivatives gross settled total assets held for managing liquidity risk maturity gap from assets and liabilities in thousands of euros in thousands of euros on demand on demand up to months up to months months months total total carrying amount carrying amount years years over years over years new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros current assets due from central bank note due from credit institutions note due from investment companies note available for sale financial assets note financial assets at fair value through profit or loss note loans and advances to customers note receivables from customers note other assets note total current assets non current assets available for sale financial assets note financial assets at fair value through profit or loss note loans and advances to customers note other financial assets note tangible assets note intangible assets note goodwill note total non current assets total assets note liabilities current liabilities deposits from customers and loans received note financial liabilities at fair value through profit or loss note accounts payable and other liabilities note total current liabilities non current liabilities deposits from customers and loans received note subordinated debt note total non current liabilities total liabilities note the following table presents the distribution of assets and liabilities by classification of current and non current distribution of assets and liabilities by geographic region is presented below as at the loans issued to customers total had a large risk exposure i e more than of the net own funds nof of group either individually or via group risk totalling of nof the group has invested in the debt securities of issuers with a large risk exposure totalling of nof unused loan commitments in amount of eur thousand are for estonian residents and in the amount of eur thousand for lithuanian residents eur thousand for estonian residents new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros in thousands of euros estonia estonia latvia latvia the nether lands the nether lands germany germany other eu other eu usa usa other other total total lithuania lithuania finland finland due from banks and investment companies note financial assets at fair value note loans and advances to customers note receivables from customers note other financial assets note total financial assets deposits from customers and loans received note subordinated debt note accounts payable and other financial liabilities note financial liabilities at fair value note total financial liabilities due from banks and investment companies note financial assets at fair value note loans and advances to customers note receivables from customers note other financial assets note total financial assets deposits from customers and loans received note subordinated debt note accounts payable and other financial liabilities note financial liabilities at fair value note total financial liabilities new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee distribution of loans granted by industry gross in thousands of euros individuals real estate manufacturing art and entertainment financial services wholesale and retail administrative activities transport and logistics agriculture other servicing activities construction information and communication professional scientific and technical activities education other areas at activities total note level level level financial assets at fair value through profit or loss shares and fund units note available for sale debt securities note debt securities at fair value through profit or loss note interest rate swaps and foreign exchange forwards note total financial assets financial liabilities at fair value through profit or loss interest rate swaps and foreign exchange forwards note total financial liabilities in thousands of euros level level level financial assets at fair value through profit or loss shares and fund units note available for sale debt securities note debt securities at fair value through profit or loss note total financial assets financial liabilities at fair value through profit or loss interest rate swaps and foreign exchange forwards note total financial liabilities fair value of financial assets and financial liabilities in thousands of euros shares and fund units include the groups subsidiary as lhv varahaldus investment into pension fund units in the amount of eur thousand eur thousand euros pursuant to the investment funds act the mandatory shares of lhv varahaldus as the management company is of the number of units in each of the mandatory pension fund managed by it the management board of the group has assessed the fair value of assets and liabilities carried at amortised cost in the statement of financial position for estimating fair value the future cash flows are discounted on the basis of the market interest yield curve levels used in hierarchy level quoted prices in active market level valuation technique based on observable market data as inputs rates and interest curves from similar transactions level other valuation techniques e g discounted cash flow method with judgemental inputs new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee interest rate swaps and foreign exchange forwards are instruments where active markets supply observable inputs to the valuation model which is used for establishing the fair value the fair value for such over the counter derivatives is calculated as a theoretical net present value npv based on independently sourced market parameters assuming no risk and uncertainties market observable zero coupon yield curve is used for discounting in npv calculations as lhv varahaldus investments into pension fund units have quoted prices from the market however the market is inactive and therefore disclosed as level as at the fair value of corporate loans is eur thousand lower than their carrying amount eur thousand higher the loans have been issued at market conditions in the segment that the group operates in and therefore their fair value is not much different from their carrying amount as at and the fair value level of corporate loans and overdraft is as significant judgmental assumptions are used for the valuation process discounted cash flow method with current market interest interest rate used is the average interest rate of corporate loans issued in last months of the reporting period and eur thousand carrying value fair value difference level lease interest rates offered to customers generally corre spond to interest rates prevailing in the market for such products considering that the interest rate environment has been relatively stable since the group started to provide leasing consequently the fair value of lease agreements does not materially differ from their carrying amount as significant management judgment is required to determine fair value leases are classified as level in the fair value hierarchy mortgage loans interest rates offered to customers corre spond to interest rates prevailing in the market for such prod ucts as mortgage product is long term product then bank has set up separate process monitoring the rates offeref by competitiors considering that bank has actively started offering the product only in october the interest rate environment has not changed consequently the fair value of mortgage agreements does not differ from their carrying amount as significant management judgment is required to determine fair value mortgage loans are classified as level in the fair value hierarchy leveraged loans hire purchase and credit cards granted to customers are of sufficiently short term nature and they have been issued at market terms therefore the fair market rate of interest and also the fair value of loans do not change significantly during the loan term the effective interest rate of consumer loans issued is at the same level as the interest rate of the loan product offered in the market and it can be stated that the carrying amount of loans does not signifi cantly differ from their fair value the fair value level of lever aged loans hire purchase credit cards and consumer loans is as significant judgmental assumptions are used for the valuation process trade receivables other than the receivables related to loans and advances to customers and accrued expenses and other financial liabilities have been incurred in the course of ordinary business and are payable in the short term there fore the management estimates that their fair value does not significantly differ from their carrying amount these receivables and liabilities are interest free the fair value level of those receivables accrued expenses and other financial liabilities is customer deposits and loans received with fixed interest rates are primarily short term and pricing of the deposits is subject to market conditions as a result the fair value of deposits determined using the discounted future cash flows do not significantly differ from their carrying amount the fair value level of customer deposits is as significant judgmental assumptions are used for the valuation process the majority of subordinated debts were received in and in and the remainder in which was redeemed in these loans carry approximately equal interest rates and and considering the short term between the loan received in october and the balance sheet date it can be concluded that no material changes have occurred in interest rates as of the balance sheet date therefore the fair value of loans does not differ significantly from their carrying amount as significant management judgment is required to determine fair value loans are classified as level in the fair value hierarchy for the term structure of financial assets and financial liabili ties refer to note operating risk operating risk is a potential loss caused by human process or information system flaws when completing transac tions transaction limits and competence systems are used to minimise potential losses and the segregation of duties principle is used in the groups working procedures according to which there should be an approval by at least new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee two employees or units in order to carry out a transaction or procedure the information received from monitoring operating risk helps to gather initial information to secure capital adequacy of the group and to evaluate capital requirements the anal ysis of cases collected into the database enables the group to identify the flaws in rules of procedure avoid making mistakes in the future and mitigate possible risks or define the terms of their acceptance the risk control manager of group is responsible for collecting information compliance control and internal audit department have an important role in evaluating monitoring and mitigating the operating risk pursuant to the credit institutions act and securities market act the main task of compliance officer is to define the risks of non compliance of the activities of the group with legislation recommended guidelines of the financial supervision authority and procedure rules of the group considering the nature range and complexity of busi ness and services rendered and arrangement of mitigating or avoiding those risks internal audit is an independent and objective function assuring and consulting activity that is targeted at improving the groups performance and adding value internal audit helps achieving the goals of the group using a systematic and disciplined approach to assess and increase the efficiency of the risk management control and organisational management process gross amounts before offsetting in the statement of financial position offsetting cash collateral received net amounts net amount of exposure presented in the statement of financial position related arrangements not set off in the statement of financial position financial instruments assets prepayments to merchants for registered customer contacts client receivables leveraged loans incl repo loans note liabilities payables to merchants assets prepayments to merchants for registered customer contacts client receivables leveraged loans incl repo loans note liabilities payables to merchants offsetting assets and liabilities the group has offset the following assets and liabilities new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note significant management estimates and assumptions note business segments in accordance with ifrs several financial figures presented in the financial statements are strictly based on critical accounting estimates and assumptions made by manage ment which affect the reported amounts of the assets and liabilities and disclosure of contingent assets and liabilities presented in the financial statements at the balance sheet date and the reported amounts of income and expenses of subsequent reporting periods although these estimates have been made to the best of managements knowledge and their judgement of current events the actual outcome may ultimately not coincide with them and may significantly differ from these estimates managements estimates have been applied to valuation of loans receivables and invest ments notes and estimates and judgments are continually evaluated based on historical experience and other factors including expec tations of future events that are believed to be reasonable in the circumstances changes in managements estimates are reported prospectively bank divides its business activities by main business segments retail banking private banking and corpo rate banking the business segments form a part of the group with a separate access to financial data and which are subject to regular monitoring of operating result by the groups decision maker the full planning and kpi setting is done on segments level for each of the segment full balance sheet and profit and loss statement is prepared in addition to these high level segments group has planning measure ment on department level grouping is done based on the client product base where similar departments are grouped under one segment private banking covers high net worth customers who main product is investments retail banking segment covers all private individuals and small legal enti ties with credit exposure under eur thousand this is the regular universal banking segment offering payments cards credits etc corporate banking segment covers all large corporates and other legal entities with credit exposure over eur thousand the main products are credits and payments asset management is the pension managemet segment covering pension second and third pillar hire pur chase units both in estonia and in lithuania are separate legal entities covering small loan business to private individ uals these portfolio are relatively small but due to clients different payment history these segments have to be looked separately the management board of as lhv group has been designated as the decision maker responsible for allocation of funds and assessment of the profitability of the business activities the result posted by a segment includes revenue and expenditure directly related to the segment the revenue of a reported segment includes gains from transactions between the segments i e loans granted by as lhv pank to other group companies the division of interest income and fee and commission income by customer loca tion has been presented in notes and the group does not have any customers whose income would account for more than of the corresponding type of revenue new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros in thousands of euros retail banking private banking hire purc hase and consumer finance in estonia hire purc hase and consumer finance in lithuania other activities intra seg ment eliminations total corporate banking asset manage ment retail banking private banking hire purc hase and consumer finance in estonia hire purc hase and consumer finance in lithuania other activities intra seg ment eliminations total corporate banking asset manage ment interest income interest expense net interest income fee and commission income fee and commission expense net fee and commission income net income net gains from financial assets administrative and other operating expenses staff costs operating profit income loss from associates impairment losses on loans and advances income tax net profit total assets total liabilities interest income interest expense net interest income fee and commission income fee and commission expense net fee and commission income net income net gains from financial assets administrative and other operating expenses staff costs operating profit income loss from associates impairment losses on loans and advances income tax discontinued operations net profit total assets total liabilities new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note subsidiaries and associated companies goodwill as at the groups subsidiaries which have been consolidated in these financial statements include as lhv pank estonia ownership interest as lhv varahaldus estonia ownership interest cuber technology oü estonia ownership interest uab mokilizingas lithuania ownership interest share as lhv finance estonia ownership interest through as lhv pank as lhv pank paid eur thousand euros of monetary contribution for of ownership in lhv finance and a non controlling interest paid eur thousand euros of monetary contribution for of ownership the contri bution of the non controlling interest has been reflected in the consolidated statement of financial position on the line non controlling interest as at the group does not have any associates as at the group had ownership interest in oü svipe which was sold in the beginning of the for eur thousand total book value of associates as of was eur thousand as in investment of oü svipe was impaired by eur thousand in january subsidiary as lhv pank disposed its finnish branch for the amount of thousand euros the transfer included all assets and rights connected with the business and all obligations liabilities and undertakings dormant subsidiary lhv finance oy was liquidated in as at goodwill in amount eur thousand in the consolidated financial statements of as lhv group consists of positive goodwill which had arisen on the acquisi tion of the ownership interests in as lhv varahaldus in the amount of eur thousand positive goodwill which had arisen after the conclusion of a purchase contract of as lhv varahaldus entered into in in the amount of eur thousand positive goodwill which had arisen on the acqui sition of the ownership interests in danske capital as by as lhv varahaldus in the amount of eur thousand impairment tests were performed as at and as at the cash generating unit of goodwill is as lhv varahaldus the calculation of the value in use is based on the following assumptions the volume of assets under management is expected to increase per annum increase of income of fund manager is expected to be average of per annum due to the economic environment growth of in indirect costs is expected per annum the discount rate used is when using the main assumptions the management used previous years experience and its best estimate in respect of probable expectations expectations are more conservative due to the changes in the market situation and legislation based on the results of the impairment test performed as at and the recoverable amount of cash generating unit is higher than its carrying amount as a result of which no impairment losses have been recognized on may as lhv varahaldus acquired shares of danske capital as and its managed funds this purchase enabled lhv varahaldus to increase significantly market share and also profitability immediately after conclusion of the transaction of purchase and sale of shares on may lhv varahaldus launched the merger of the two manage ment companies the merger took effect on july with the balance sheet date of may the impact of revenue in is eur thousand and the impact of profit and loss is eur thousand as lhv varahaldus recognised the acquisition of danske capital as in accordance with requirements of ifrs by carrying out purchase price allocation in the course of the purchase price allocation the value of assets of the danske capital as was assessed and the assets were recognised in fair value on the transaction date date of the financial information used for the purchase price allocation was which is the date closest to the transaction date with reliable financial information available no signifi cant transactions occurred between the financial information date and transaction date that had a significant impact on the value of net assets acquired the goodwill is related with the expected synergies of managing acquired pension funds acquisition related costs of eur thousand have been charged to administrative expenses in the consolidated statement of profit or loss for the year ended december the total fair value of assets cash clients agreements pension fund units and other receivables was thou sand euros the total fair value of payables payables to employees tax payables and other payables was thou sand euros new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros fair values recorded on acquisition cash and cash equivalents pension fund units other financial assets client agreements other financial liabilities total net assets identified amount paid by lhv varahaldus as goodwill acquired by the group outflow of cash and cash equivalents on acquisition set out below are the summarised financial information for each subsidiary that has non controlling interests that are material to the group the information disclosed is the amount before inter company eliminations uab mokilizingas as lhv finance summarised statement of financial position loans and advances to customers and other current assets non current assets current liabilities non current liabilities total net assets summarised statement of profit or loss and other comprehensive income total net interest and fee income profit before income tax income tax expense net profit total comprehensive income profit and other comprehensive income allocated to non controlling interests summarised statement of cash flows cash generated from operations interest paid income tax paid net cash generated from used in operating activities net cash generated from used in investing activities net cash generated from used in financing activities net increase decrease in cash and cash equivalents cash and cash equivalents at beginning of year cash and cash equivalents at end of year uab mokilizingas as lhv finance uab mokilizingas as lhv finance new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note net interest income interest income corporate loans incl retail loans incl loans to related parties note hire purchase consumer loans leasing incl loans to related parties note leveraged loans and lending of securities from debt securities incl debt securities available for sale incl debt securities at fair value through profit or loss credit card loans mortgage loans from balances with credit institutions and investment companies other loans total interest expense deposits from customers and loans received incl loans from related parties note from balances with central bank subordinated debt note incl loans from related parties note total net interest income interest income of loans by customer location interests from bank balances and debt securities not included estonia latvia lithuania total interests calculated on impaired loans in is eur eur thousand new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note net fee and commission income note operating expenses fee and commission income security brokerage and commission fees asset management and related fees incl funds managed by the group currency exchange fees fees from cards and settlements fee for snoras portfolio administration other fee and commission income total fee and commission expense security brokerage and commission fees paid expenses related to cards expenses related to acquiring other fee expense total net fee and commission income commission fees from pension and investment funds are calculated as a fixed percentage of total assets of the fund and varies between p a mokilizingas is providing portfolio administration services to snoras bank to whom the portfolio was sold in fee and commission income by customer location estonia latvia lithuania sweden other total in thousands of euros wages salaries and bonuses social security and other taxes total staff costs it expenses information services and banking services marketing expenses office expenses transportation and communication costs training and travelling expenses of employees other outsourced services other administrative expenses depreciation operating lease payments other operating expenses total other operating expenses total operating expenses lump sum payment of social health and other insurances the average number of employees working for lhv group in was new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note due from central bank credit institutions and investment companies note available for sale financial assets note financial assets and liabilities at fair value through profit or loss in thousands of euros demand and term deposits with maturity less than months statutory reserve capital at central bank demand deposit from central bank total cash and cash equivalents in the statement of cash flows securities held for trading shares and fund units debt securities designated as at fair value through profit or loss upon initial recognition fund units incl investments in managed pension funds foreign exchange forwards total financial assets interest rate swaps and foreign exchange forwards total financial liabilities financial assets at fair value through profit or loss net changes of investment securities at fair value through profit or loss interest income note revaluation financial assets at fair value through profit or loss net changes of investment securities at fair value through profit or loss interest income note revaluation financial assets at fair value through profit or loss available for sale financial assets proceeds from disposal and maturities of assets available for sale interest income note revaluation of available for sale assets available for sale financial assets proceeds from disposal and maturities of assets available for sale interest income note revaluation of available for sale assets revaluation reclassified through profit or loss available for sale financial assets distribution of receivables by countries is presented in note mandatory banking reserve as at was of all financial resources collected deposits from customers and loans received the reserve requirement is to be fulfilled as a monthly average in euros or in the foreign securities preapproved by the central bank the group has available for sale debt and equity securi ties portfolio the balance of other reserve in equity as of is eur thousand eur thousand see also note in and no gains or losses arose from the sales of debt securities in visa share was revalued to eur thousand which profit on disposal was recycled to profit and loss statement during new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee bid price is the fair value of investments in the case of publicly listed securities note in a gain of eur thousand was recognised on the revaluation of debt securities eur thousand no profit or loss arose on the revaluation of interest rate swaps in loss eur thousand the volume of pension and investment fund assets managed by the group as at was eur million eur million note discontinued operations note loans and advances to customers discontiniued operations presents the groups disposed business of the branch in finland which was sold on january in thousands of euros loans to legal entities incl corporate loans incl retail loans incl leasing incl overdraft incl leveraged loans incl hire purchase incl credit card loans loans to individuals incl hire purchase incl mortgage loans incl consumer loans incl leasing incl leveraged loans incl credit card loans incl overdraft total incl related parties note impairment provisions total net income from discontinued operations other financial income total expenses net profit for the year from discontinued operations new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee balance as at january impairment provisions set up during the year written off during the year balance as at december balance as at january impairment provisions set up during the year written off during the year balance as at december changes in impairments in changes in impairments in net and gross investments on finance leases according to remaining maturity corporate loans incl overdraft retail loans hire purchase leasing leve raged loans other loans incl mortgage total consumer loans credit cards corporate loans incl overdraft hire purchase leasing gross investment leve raged loans unearned future interest income other loans incl mortgage impair ment loss provision total present value of lease payments receivable consumer loans credit cards up to year years over years total as at up to year years over years total as at note receivables from customers asset management fees from customers incl related parties note other fees for providing services to customers payments in transit other receivables total for credit risk exposures and loan collateral see note distribution of loans granted by currencies is disclosed in note distribution of loans granted by maturity is disclosed in note the geographical distribution of loans granted is disclosed in note for interest income on loans granted see note all fees other than receivables related to collection of receivables are collected within months of the balance sheet date and are considered as current asset new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note other assets note tangible and intangible assets financial assets guarantee deposits of baltic stock exchanges guarantee deposits of visa and mastercard subtotal non financial assets prepayments to financial supervision authority tax prepayments repossessed assets prepayments to merchants for registered customer contracts other prepayments subtotal total prepayments include office rent insurance communica tion services periodicals and training prepayments are expected to be received or used within months of the balance sheet date and are therefore consid ered current assets guarantee deposits on the baltic stock exchanges are held to guarantee securities trading activity on the stock exchanges of tallinn riga and vilnius and the deposits of visa and mastercard to guarantee credit card transactions and should therefore both be considered non current assets tangible assets intangible assets total balance as at cost accumulated depreciation and amortisation assets of discontinued operations note carrying amount changes in purchase of non current assets depreciation amortisation charge write off of non current assets balance as at cost accumulated depreciation and amortisation carrying amount purchase of non current assets assets acquired on business combination disposal of non current assets depreciation amortisation charge write off of non current assets balance as at cost accumulated depreciation and amortisation carrying amount tangible assets include computer technology and office equipment furniture capitalized costs of office renovation intangible assets include licences and development costs in and there was no indication of impairment of tangible and intangible assets new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note deposits from customers and loans received note accounts payable and other liabilities individuals legal entities public sector total demand deposits term deposits loans received accrued interest liability total incl related parties note individuals legal entities public sector total demand deposits term deposits loans received accrued interest liability total incl related parties note financial liabilities trade payables payables to merchants other short term financial liabilities accrued interest on subordinated loans note payments in transit financial guarantee contracts issued subtotal non financial liabilities performance guarantee contracts issued tax liabilities payables to employees incl related parties note other short term liabilities subtotal total loans received from public sector includes loans from maaelu edendamise sihtasutus rural development foun dation in the amount of eur thousand eur thousand with an intended purpose to finance loans to small enterprises operating in rural areas in loans received included also loan from the european central bank secured with the debt securities portfolio in the amount of eur thousand and interest rate the nominal interest rates of most deposits from customers and loans received equal their effective interest rates as no other significant fees have been paid distribution of deposits from customers and loans received by currency is presented in note distribution of deposits from customers and loans received by maturity is presented in note distribution of deposits from customers and loans received by geography is presented in note new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee payables to employees consist of unpaid salaries bonus accruals and vacation pay accrual for the reporting period and the increase in liabilities is caused by the increase in the number of employees during the year payments in transit consist of foreign payments and payables to customers related to intermediation of securities transactions for which the customers current account have been debited all liabil ities except for financial guarantees are payable within months and are therefore recognised as current liabilities the group has received subordinated debts in order to increase long term capital in the case of default of the group the subordinated debts are repayable after all other debts have been paid but before debts to shareholders are paid the balances of subordinated debt as at the end of each reporting period is disclosed in the table below in october subordinated bonds were issued totalling eur thousand carrying a rate of interest at the due date of the bonds is october after five years the issuer has the right to prematurely redeem the bonds in june subordinated bonds were issued totalling eur thousand carrying a rate of interest at the due date of the bonds is june in december subordinated bonds were issued totalling eur thousand most of these bonds have been converted to share capital refer to note and the remaining bonds of eur thousand were redeemed in october interest expenses on subordinated bonds for each reporting period and accrued interest liabilities as at the end of each reporting period is disclosed in the table below interest liabilities are accounted in the statement of financial position using the effective interest rate subordinated debt in thousands of euros subordinated debt as at subordinated bonds redeemed subordinated bonds issued subordinated debt as at subordinated debt as at the group leases office premises under the operating lease terms all lease agreements are cancellable upon the consent of both parties the minimum unilaterally non can cellable lease payable in the next period are disclosed in the table below in the operating lease payments for office premises in the amount of eur thousand eur thousand note operating lease note subordinated debts up to year to year total non cancellable lease payables as of non cancellable lease payables as of accrued interest on subordinated debt in thousands of euros accrued interest on subordinated debts as at note interest calculated for note paid out during accrued interest on subordinated debts as at note interest calculated for note paid out during accrued interest on subordinated debts as at note new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note shareholders equity in the public limited company number of shares share premium share capital as at share capital as at paid in share capital may share capital as at share capital is paid in in full through cash contributions the nominal value of the shares is euro and as at the number of shares amounted to according to as lhv groups articles of association the minimum share capital is eur million and the maximum share capital is eur million eur and eur million rain lõhmus who owns of the voting rights and andres viisemann who owns of the voting rights in as lhv group have significant influence over the company and as at the retained earnings of the group totalled eur thousand eur thousand as of it is possible to pay out divi dends in amount thousand euros and the related income tax charge would be thousand euros statutory reserve capital in equity is as follows in thousands of euros statutory reserve as at transferred from net profit statutory reserve as at transferred from net profit statutory reserve as at other reserves in the consolidated statement of changes in equity consist of in thousands of euros revaluation reserve of available for sale securities reserve of share options granted to staff total the group is granting share options to the management board members and equivalent employees and department managers and equivalent employees of group companies that are part of as lhv group number of shares strike price per share eur exercise period number of people to whom the share options were granted outstanding amount of share options at granted amount during the period outstanding amount of share options at granted amount during the period outstanding amount of share options at granted amount during the period cancelled amount during the period outstanding amount of share options at new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the group will most likely issue the above mentioned share options also for the period the vesting period for all the share options program is years vesting date is the first day of the respective exercise period no options expired during the periods covered by the above tables members of the management board and employees were unable to opt for the determined amount in cash instead of share options share options cannot be exchanged sold pledged or encumbered share options can be inher ited the share option agreement will become void upon premature termination of the employment contract at the employees initiative exemptions to which can be made by the supervisory board of as lhv group and due to termination of employment contract on an extraordinary basis by the employer for a cause related to the employee pursuant to the credit institutions act the supervisory board of as lhv group may reduce the number of share options issued or cancel share options if the overall financial results of the group have materially deteriorated compared to the previous period a member of the management board or employee of the group no longer meets award criteria the group no longer meets the prudential ratios the risks of the group are not adequately covered with own funds or deter mination of the performance pay was based on information which was inaccurate or incorrect to a material extent total expenses arising from share based payment transactions recognised during the period as part of staff costs was eur thousand asset management fees for the management of these assets have been in the range of p a for respective income see note the monetary funds of the customers who use the platform of an active securities trader or the trading system lhv trader offered by the bank have been recognised as off balance sheet assets due to the nature of the system the bank has deposited these funds in personalised accounts with its partner and as the monetary funds of these customers are not used for business purposes by the bank they cannot be lent to other customers or used as collateral therefore the monetary funds are recognised as off balance sheet assets the bank earns commission and interest income on intermediation of transactions in these accounts similarly to customers accounts reflected in the groups financial statement the bank has provided contractual guarantees to its partner in respect of lhv trader accounts guaran teeing potential losses to be incurred from financing of the transactions executed by customers intermediated by itself leveraging with the primary collateral being the securities used as collateral for leveraged loans the customer has similar guarantee to the bank and therefore the bank has in substance transferred the guarantee to end customer to avoid potential losses the bank has a daily practice to monitor the potential decrease of collateral and deficiency of customer accounts in its current practice no such losses in respect of which the guarantee provided by the bank would apply have incurred and the bank has not paid any compen sation relating to the guarantee note assets under management as lhv pank operating as an account manager for its customers has custody of or intermediates the following customer assets cash balance of customers securities of customers incl shareholders of the parent company and related entities note total new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee tax authorities have the right to review the companys tax records for up to years after submitting the tax decla ration and upon finding errors impose additional taxes interest and fines the tax authorities have not performed any tax audits at the group during the groups management estimates that in there are no such circumstances which may lead the tax authorities to impose significant additional taxes on the group performance guarantees are contracts that provide compensation if another party fails to perform a contractual obligation such contracts do not transfer credit risk the risk under performance guarantee contracts is the possibility that the insured event i e the failure to perform the contrac tual obligation by another party occurs the key risks the group faces are significant fluctuations in the frequency and severity of payments incurred on such contracts relative to expectations the group uses historical data and statistical techniques to predict levels of such payments claims must be made before the contract matures and most claims are settled within short term this allows the group to achieve a high degree of certainty about the estimated payments and therefore future cash flows the group manages such risks by constantly monitoring the level of payments for such products and has the ability to adjust its fees in the future to reflect any change in claim payments experi ence the group has a claim payment requests handling process which includes the right to review the claim and reject fraudulent or non compliant requests the exposure and concentration of performance guarantees expressed at the amounts guaranteed is as follows performance guarantees are over collateralised as at each reporting date therefore the group has not borne any losses from performance guarantee contracts neither in nor in previous period according to sectors construction water supplies manufacturing professional scientific and technical activities other areas at activities total according to internal ratings low credit risk low credit risk medium credit risk medium credit risk heightened credit risk high credit risk high credit risk non satisfactory rating total note contingent assets and liabilities irrevocable transactions liability in contractual amount liability in contractual amount performance guarantees financial guarantees unused loan commitments total new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note transactions with related parties in preparing the financial statements of the group the following entities have been considered related parties owners that have significant impact on the group and the entities related to them members of the management board and legal entities controlled by them together referred to as management members of the supervisory board close relatives of the persons mentioned above and the entities related to them transactions note interest income incl management incl shareholders their related entities and close relatives that have significant influence fee and commission income incl management incl shareholders their related entities and close relatives that have significant influence interest expenses from deposits incl management incl shareholders their related entities and close relatives that have significant influence interest expenses from subordinated debt incl management incl shareholders their related entities and close relatives that have significant influence balances note loans and receivables as at the year end incl management incl shareholders their related entities and close relatives that have significant influence deposits as at the year end incl management incl shareholders their related entities and close relatives that have significant influence subordinated debt as at the year end incl management incl shareholders their related entities and close relatives that have significant influence the table provides an overview of the material balances and transactions involving related parties all other transac tions involving the close relatives and the entities related to members of the management board and supervisory board and the minority shareholders of the parent company as lhv group have occurred in the course and on the terms of ordinary business loans granted to related parties are issued at market conditions as at and the management did not have term deposits and the interest rate on demand deposits corresponds to the terms applicable to customers the subordinated debts received in october have the interest rate of the subordinated debts received in june have the interest rate of refer to note the subordinated debts received in december had the interest rate of during the first three years these loans were redeemed in october new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in salaries and other compensations paid to the management of the parent as lhv group and its subsid iaries totalled eur thousand eur thousand including all taxes as at remuner ation for december and accrued holiday pay in the amount of eur thousand eur thousand is reported as a payable to management note the group did not have any long term payables or commitments to the members of the management board and the supervisory board as at and pension liabilities termination benefits etc in the remuneration paid to the members of the groups supervisory board totalled eur thousand eur thousand the group has signed contracts with the members of the management board which do not provide for severance benefits upon termination of the contract in any matters not regulated by the contract the parties adhere to the proce dure specified in the legislation of the republic of estonia management is involved with share option program in the share options were granted to the members of the management board in the amount of eur thousand eur thousand information on assets of related parties held as an account manager is presented in note in order to calculate basic earnings per share net profit has been divided by the weighted average number of shares issued for calculation of diluted earnings per share for the annual average potential shares from subordi nated bonds is used although the last convertible bonds were redeemed in october see note and as at december no convertible bonds were outstanding note basic earnings and diluted earnings per share total profit incl discontinued operations attributable to owners of the parent eur thousand profit attributable to owners of the parent from continuing operations eur thousand weighted average number of shares in thousands of units basic earnings per share eur basic earnings per share from continuing operations eur basic earnings per share from discontinued operations eur weighted average number of shares used for calculating the diluted earnings per shares in thousands of units diluted earnings per share eur diluted earnings per share eur from continuing operations diluted earnings per share eur from discontinued operations weighted average number of shares used as the denominator in thousands of shares weighted average number of ordinary shares used as the denominator in calculating basic earnings per share adjustments for calculation of diluted earnings per share convertible subordinated bonds share options weighted average number of ordinary shares and potential ordinary shares used as the denominator in calculating diluted earnings per share new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note separate financial statements of parent company in accordance with the estonian accounting act information on the separate primary financial statements of the parent of the consolidation group shall be disclosed in the notes to the financial statements statement of profit or loss and other comprehensive income of the parent in thousands of euros interest income interest expense net interest expense net gains losses from investments to associates net gains losses from financial assets operating expenses net loss for the year total loss and other comprehensive loss for the year new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee statement of financial position of the parent in thousands of euros assets due from banks and investment companies loans granted other receivables and assets investments in subsidiaries total assets liabilities accrued expenses and other liabilities subordinated debt total liabilities equity share capital share premium statutory reserve capital other reserves accumulated deficit total shareholders equity total liabilities and equity new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee statement of cash flows of the parent in thousands of euros cash flows from operating activities interest received interest paid administrative and other operating expenses paid cash flows from operating activities before change in operating assets and liabilities adjustments investments in subsidiaries from share options net increase decrease in operating assets and liabilities net increase decrease of other receivables net increase decrease of other liabilities net cash from used in operating activities cash flows from investing activities loans granted disposals of subsidiaries and associates acquisitions of subsidiaries and associates net cash from used in investing activities cash flows from financing activities paid in to share capital incl share premium repayment of subordinated debt proceeds from subordinated debt net cash from financing activities increase decrease in cash and cash equivalents cash and cash equivalents at the beginning of the financial year cash and cash equivalents at the end of the financial year new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee statement of changes in shareholders equity in thousands of euros balance as at transfer to statutory reserve capital share options total loss and other comprehensive loss for balance as at carrying amount of holdings under control and significant influence value of holdings under control and significant influence under equity method adjusted unconsolidated equity as at balance as at paid in share capital transfer to statutory reserve capital share options total loss and other comprehensive loss for balance as at carrying amount of holdings under control and significant influence value of holdings under control and significant influence under equity method adjusted unconsolidated equity as at statutory reserve capital share premium share capital other reserves accumulated deficit retained earnings total adjusted unconsolidated equity is the maximum amount that can be distributed to shareholders according to estonian legislation new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee signatures of the management board to the consolidated annual report the management board has prepared the management report and the consolidated financial statements of as lhv group for the financial year ended december the management board confirms that the management report on pages provides a true and fair view of the business operations financial results and financial condition of the parent company and the entities included in consolidation as a whole the management board confirms that according to their best knowledge the consolidated financial report on pages presents a fair view of the assets liabilities financial position and profit or loss of the issuer and the entities involved in the consolidation as a whole according to the international financial reporting standards as they are adopted by the european union and contains a description of the main risks and doubts madis toomsalu stamp as pricewaterhousecoopers pärnu mnt tallinn estonia license no registry code t f www pwc ee independent auditors report translation of the estonian original to the shareholders of lhv group as our opinion in our opinion the consolidated financial statements present fairly in all material respects the consolidated financial position of lhv group as the company and its subsidiaries together the group as at december and its consolidated financial performance and its consolidated cash flows for the year then ended in accordance with international financial reporting standards as adopted by the european union what we have audited the groups consolidated financial statements comprise the consolidated statement of financial position as at december the consolidated statement of profit or loss and other comprehensive income for the year then ended the consolidated statement of cash flows for the year then ended the consolidated statement of changes in equity for the year then ended and the notes to the consolidated financial statements which include a summary of significant accounting policies and other explanatory information basis for opinion we conducted our audit in accordance with international standards on auditing isas our responsibilities under those standards are further described in the auditors responsibilities for the audit of the consolidated financial statements section of our report we believe that the audit evidence we have obtained is sufficient and appropriate to provide a basis for our opinion independence we are independent of the group in accordance with the international ethics standards board for accountants code of ethics for professional accountants iesba code and the ethical requirements of the auditors activities act of the republic of estonia we have fulfilled our other ethical responsibilities in accordance with the iesba code and the ethical requirements of the auditors activities act of the republic of estonia our audit approach overview materiality overall group materiality is eur thousand which represents of profit before tax adjusted for any significant one off gains or losses audit scope a full scope audit was performed by pwc network firms for group entities covering substantially all of the groups consolidated statements of financial position and profit or loss key audit matters impairment of loans and advances to customers accounting for the acquisition of danske capital as as part of designing our audit we determined materiality and assessed the risks of material misstatement in the consolidated financial statements in particular we considered where the management board made subjective judgments for example in respect of significant accounting estimates that involved making assumptions and considering future events that are inherently uncertain as in all of our audits we also addressed the risk of management override of internal controls including among other matters consideration of whether there was evidence of bias that represented a risk of material misstatement due to fraud materiality the scope of our audit was influenced by our application of materiality an audit is designed to obtain reasonable assurance whether the consolidated financial statements are free from material misstatement misstatements may arise due to fraud or error they are considered material if individually or in aggregate they could reasonably be expected to influence the economic decisions of users taken on the basis of the consolidated financial statements based on our professional judgment we determined certain quantitative thresholds for materiality including the overall group materiality for the consolidated financial statements as a whole as set out in the table below these together with qualitative considerations helped us to determine the scope of our audit and the nature timing and extent of our audit procedures and to evaluate the effect of misstatements both individually and in aggregate on the financial statements as a whole overall group materiality eur thousand how we determined it of profit before tax adjusted for any significant one off gains or losses rationale for the materiality benchmark applied the use of profit before tax is considered appropriate as in our view profit before tax is stakeholders primary measurement benchmark and key performance indicator for management and supervisory board the adjusting items are excluded on the basis that they do not reflect the day to day operational performance of the group materiality audit scope key audit matters we agreed with the audit committee that we would report to them misstatements identified during our audit above eur thousand as well as misstatements below that amount that in our view warranted reporting for qualitative reasons key audit matters key audit matters are those matters that in our professional judgment were of most significance in our audit of the consolidated financial statements of the current period these matters were addressed in the context of our audit of the consolidated financial statements as a whole and in forming our opinion thereon and we do not provide a separate opinion on these matters key audit matter how our audit addressed the key audit matter impairment of loans and advances to customers refer to note summary of significant accounting policies note credit risk and note loans and advances to customers for further details as at december loans and advances to customers amounted to eur million and related impairment loss for amounted to eur million we focused on this area because management makes complex and subjective judgments over both timing of recognition of impairment and the estimation of the size of any such impairment for corporate loans a portion of impairment is individually assessed by taking into consideration the credit rating of the customer and other known heightened risk factors credit rating is assigned considering a combination of financial and business risks associated with the customer individual impairment is calculated based on the exposure and realisable value of the collateral at the balance sheet date in addition to individual assessment and impairment calculation of corporate loans a significant portion of their impairment is calculated on a collective basis taking into account customer rating historical performance the probability of default loss given default and other factors loans to individuals and loans to legal entities other than corporate loans are grouped for impairment calculation purposes into sub classes on the basis of we assessed whether the groups accounting policies in relation to the impairment of loans and advances to customers are in compliance with ifrs we assessed the design and operating effectiveness of the controls over impairment data and calculations these controls included those over the identification of which loans and advances were individually impaired overdue loans monitoring and the calculation of the collective impairment provisions including annual validation process of collective impairment model inputs in addition we tested the design and operating effectiveness of the credit file periodic review and rating assessment and monitoring of collateral controls for corporate loans we determined that we could rely on these controls for the purposes of our audit we performed detailed testing over the completeness and accuracy of data in the impairment model for collective impairment calculations the internal assignment of credit ratings for corporate loan customers the assumptions used for critical inputs in the collective impairment model such as probability of default and loss given default we have checked the annual calculations of probability of default and loss given default performed by management for material loan products by evaluating the correctness of the calculation methodology and historical data used the correctness of information on collaterals homogeneous credit risk features the major part of impairment for those sub classes is calculated on a collective basis taking into account historical performance the probability of default loss given default and other factors areas that require the most significant judgments and estimates are the identification and monitoring process of corporate customers with ratings below investment grade and other known heightened risk factors and individual impairment calculations of such loans with special attention on the valuation of collaterals the key assumptions and judgments made by management that underlie the calculation of collective impairment key assumptions and judgments include the probability of default calculation the loss given default calculation and credit conversion factor calculation the assessment of annual internal validation process of group based valuation models taking into account the repayment behavior of different customer groups and the completeness of the customer accounts that are included in the impairment calculation and their values in the loan systems and the completeness of loans subject to individual impairment assessment and related calculations for customers operating in currently low performing industry sectors e g agriculture and oil industry we checked the total exposure of customer loans in those sectors and assessed the performance of those customers based on internal and external information in the case of some impairment provisions we formed a different view from that of the management but in our view the differences were within a reasonable range of outcomes in the context of the overall loans and advances and the uncertainties disclosed in the financial statements as a result of our work we noted no material exceptions accounting for the acquisition of danske capital as refer to note summary of significant accounting policies and note subsidiaries and associated companies goodwill for further details in may the group acquired the asset management company danske capital as together with pension and investment funds managed by it for a total consideration of eur million as further described in note accounting for business combination involves compiling the purchase price allocation during which the fair value of the purchase consideration paid is allocated to the fair values of the identifiable assets acquired liabilities and contingent liabilities assumed with any remaining difference recorded as goodwill these fair values were assessed by the management and it required judgment due to the size of the acquired business and related estimation uncertainty it is considered a key audit matter we assessed whether the groups accounting policy in relation to accounting for the business combinations is in compliance with ifrs we evaluated and performed detailed testing of the purchase price allocation compiled by the management including assessing the list of separately identified assets and liabilities for their reasonableness and compliance with ifrs requirements evaluating the managements assessment of the fair values of identifiable assets acquired and liabilities assumed involving pwc valuation experts to evaluate the methodology applied for determination of fair values deploying our pwc valuation experts we tested the reasonableness of key assumptions including cash flow forecasts customer churn rate and discount rates and checking the correctness of calculation of arising goodwill furthermore we assessed the adequacy of the disclosures related to the acquisition of danske capital as as a result of our work we noted no material exceptions how we tailored our audit scope we tailored the scope of our audit in order to perform sufficient work to enable us to provide an opinion on the consolidated financial statements as a whole taking into account the structure of the group the accounting processes and controls and the financial services industry in which the group operates the group comprises a number of subsidiaries that mostly operate in estonia and lithuania refer to note a full scope audit was performed by pwc estonia or other pwc network firms for the following group entities covering substantially all of the groups consolidated statements of financial position and profit or loss lhv group as lhv pank as including its subsidiary lhv finance as lhv varahaldus as and uab mokilizingas where the work was performed by other pwc network firms we determined the level of involvement we needed to have to be able to conclude whether sufficient appropriate audit evidence had been obtained as a basis for our opinion on the group financial statements as a whole the nature timing and extent of the work performed by other pwc network firms impacting the group audit opinion is set and monitored in estonia other information the management board is responsible for the other information contained in the consolidated annual report in addition to the consolidated financial statements and our auditors report thereon our opinion on the consolidated financial statements does not cover the other information and we do not express any form of assurance conclusion thereon in connection with our audit of the consolidated financial statements our responsibility is to read the other information identified above and in doing so consider whether the other information is materially inconsistent with the consolidated financial statements or our knowledge obtained in the audit or otherwise appears to be materially misstated if based on the work we have performed we conclude that there is a material misstatement of this other information we are required to report that fact we have nothing to report in this regard responsibilities of the management board and those charged with governance for the consolidated financial statements the management board is responsible for the preparation and fair presentation of the consolidated financial statements in accordance with international financial reporting standards as adopted by the european union and for such internal control as the management board determines is necessary to enable the preparation of consolidated financial statements that are free from material misstatement whether due to fraud or error in preparing the consolidated financial statements the management board is responsible for assessing the groups ability to continue as a going concern disclosing as applicable matters related to going concern and using the going concern basis of accounting unless the management board either intends to liquidate the group or to cease operations or has no realistic alternative but to do so those charged with governance are responsible for overseeing the groups financial reporting process auditors responsibilities for the audit of the consolidated financial statements our objectives are to obtain reasonable assurance about whether the consolidated financial statements as a whole are free from material misstatement whether due to fraud or error and to issue an auditors report that includes our opinion reasonable assurance is a high level of assurance but is not a guarantee that an audit conducted in accordance with isas will always detect a material misstatement when it exists misstatements can arise from fraud or error and are considered material if individually or in the aggregate they could reasonably be expected to influence the economic decisions of users taken on the basis of these consolidated financial statements as part of an audit in accordance with isas we exercise professional judgment and maintain professional scepticism throughout the audit we also identify and assess the risks of material misstatement of the consolidated financial statements whether due to fraud or error design and perform audit procedures responsive to those risks and obtain audit evidence that is sufficient and appropriate to provide a basis for our opinion the risk of not detecting a material misstatement resulting from fraud is higher than for one resulting from error as fraud may involve collusion forgery intentional omissions misrepresentations or the override of internal control obtain an understanding of internal control relevant to the audit in order to design audit procedures that are appropriate in the circumstances but not for the purpose of expressing an opinion on the effectiveness of the groups internal control evaluate the appropriateness of accounting policies used and the reasonableness of accounting estimates and related disclosures made by the management board conclude on the appropriateness of the management boards use of the going concern basis of accounting and based on the audit evidence obtained whether a material uncertainty exists related to events or conditions that may cast significant doubt on the groups ability to continue as a going concern if we conclude that a material uncertainty exists we are required to draw attention in our auditors report to the related disclosures in the consolidated financial statements or if such disclosures are inadequate to modify our opinion our conclusions are based on the audit evidence obtained up to the date of our auditors report however future events or conditions may cause the group to cease to continue as a going concern evaluate the overall presentation structure and content of the consolidated financial statements including the disclosures and whether the consolidated financial statements represent the underlying transactions and events in a manner that achieves fair presentation obtain sufficient appropriate audit evidence regarding the financial information of the entities or business activities within the group to express an opinion on the consolidated financial statements we are responsible for the direction supervision and performance of the group audit we remain solely responsible for our audit opinion we communicate with those charged with governance regarding among other matters the planned scope and timing of the audit and significant audit findings including any significant deficiencies in internal control that we identify during our audit we also provide those charged with governance with a statement that we have complied with relevant ethical requirements regarding independence and to communicate with them all relationships and other matters that may reasonably be thought to bear on our independence and where applicable related safeguards from the matters communicated with those charged with governance we determine those matters that were of most significance in the audit of the consolidated financial statements of the current period and are therefore the key audit matters we describe these matters in our auditors report unless law or regulation precludes public disclosure about the matter or when in extremely rare circumstances we determine that a matter should not be communicated in our report because the adverse consequences of doing so would reasonably be expected to outweigh the public interest benefits of such communication as pricewaterhousecoopers ago vilu verner uibo auditors certificate no auditors certificate no february this version of our report is a translation from the original which was prepared in estonian all possible care has been taken to ensure that the translation is an accurate representation of the original however in all matters of interpretation of information views or opinions the original language version of our report takes precedence over this translation as lhv group tartu mnt tallinn info lhv ee lhv ee proposal for profit distribution the management board of lhv group proposes to the general meeting of shareholders to distribute the profit of the financial year as follows transfer eur thousand to statutory reserve capital pay dividends eur per share in the total amount of thousand transfer the profit for reporting period attributable to shareholders of the parent in the amount of eur thousand to retained earnings as lhv group tartu mnt tallinn info lhv ee lhv ee signatures of the supervisory board to the annual report chairman of the supervisory board rain lõhmus members of the supervisory board raivo hein heldur meerits tiina mõis sten tamkivi tauno tats andres viisemann as lhv group tartu mnt tallinn info lhv ee lhv ee allocation of income according to emta classificators consolidated emtak activity security and commodity contracts brokerage credit institutions banks granting loans finance lease fund management total income separate emtak activity activities of holding companies total income '.lower().split()



nltk.download('stopwords', quiet=True)
stop_words = stopwords.words('english')
sentence_obama = [w for w in sentence_obama if w not in stop_words]
sentence_president = [w for w in sentence_president if w not in stop_words]


corpus = [sentence_obama, sentence_president]
dictionary = Dictionary(corpus)
tfidf = TfidfModel(dictionary=dictionary)

sentence_obama = tfidf[dictionary.doc2bow(sentence_obama)]
sentence_president = tfidf[dictionary.doc2bow(sentence_president)]


glove = api.load("glove-wiki-gigaword-50")
similarity_index = WordEmbeddingSimilarityIndex(glove)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

interpret_soft_cosine_measure(sentence_obama, sentence_president, dictionary, similarity_matrix)

0.00 = 


In [103]:
     

import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.matutils import cossim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

import nltk
from nltk.corpus import stopwords

sentence_obama = 'Audit and control, Board structure, Remuneration, Shareholder rights, Transparency and Performance'.lower().split()

sentence_president = 'group annual indd as lhv group consolidated annual report translation of the estonian original as lhv group tartu mnt tallinn info lhv ee lhv ee consolidated annual report legal name as lhv group commercial registry no legal address tartu mnt tallinn phone fax e mail lhv lhv ee main activities activities of holding companies banking security brokerage financial advisory finance lease and other lending management board erkki raasuke until madis toomsalu from supervisory board rain lõhmus andres viisemann tiina mõis heldur meerits raivo hein tauno tats sten tamkivi auditor as pricewaterhousecoopers as lhv group tartu mnt tallinn info lhv ee lhv ee contents statement of managing director management report business environment financial results governance of the group corporate governance report consolidated financial statements consolidated statements of profit or loss and other comprehensive income consolidated statements of financial position consolidated statements of cash flows consolidated statements of changes in equity notes to the consolidated financial statements note general information note summary of significant accounting policies note risk management note significant management estimates and assumptions note business segments note subsidiaries and associated companies goodwill note net interest income note net fee and commission income note operating expenses note due from central bank credit institutions and investment companies note available for sale financial assets note financial assets and liabilities at fair value through profit or loss note discontinued operations note loans and advances to customers note receivables from customers note other assets note tangible and intangible assets note deposits from customers and loans received note accounts payable and other liabilities note operating lease note subordinated debts note shareholders equity in the public limited company note assets under management note contingent assets and liabilities note transactions with related parties note basic earnings and diluted earnings per share note separate financial statements of parent company signatures of the management board to the consolidated annual report independent auditors report proposal for profit distribution signatures of the supervisory board to the annual report allocation of income according to emta classificators as lhv group tartu mnt tallinn info lhv ee lhv ee statement of managing director dear lhv investor lhv group and its subsidiaries hereinafter lhv have had another successful year most of our objectives were achieved with room to spare lhvs shares were listed on the tallinn stock exchange in may the overall staff satis faction is the highest on record and we received several important recognitions during the year the economic environment has remained stable despite the worrying underlying trends the inauguration of the new president of the united states is likely to add further to trade barriers to the detriment of most of the countrys trade partners including the european union against the backdrop of over regulation high debt burden negative demographic dynamics and radical tendencies in europe it is difficult to achieve long term broad based growth to fix these problems the governments of several major countries have decided to boost expenditure rather than maintain a budget balance with opec cutting production volumes this has fuelled expectations of inflation which can in itself be considered a positive trend in the current interest rate environment regional economic environment is characterised by a growth in wages and consumption in the baltic states a strong real estate cycle in sweden and a sluggish recovery in finnish economy investments are greatly missed in all of these markets despite a strong credit cycle growth in loan volumes is also evident in estonia although we can detect the first signs of a rise in credit pricing and growth in the number of companies in debt the greatest concern for enterprises is availability of labour and a quick growth in labour costs leading to a drop in profit nonetheless convergence with europe is unavoidable to enhance effi ciency of labour costs we therefore need to boost invest ments the record low fixed financing costs provide a good foundation for such efforts this is certainly something our enterprises should further exploit low interest rates have inflated the real estate sector trig gering a potential oversupply in rental cashflow projects lhv avoids financing projects without tenants residential developments are only financed if the corresponding condi tions serve to ensure a market balance on a positive note no other major balance shifts or risks are evident on the domestic scale lhv has done well in such an environment even though financial intermediation is among the most regulated busi ness sectors we have the opportunity to offer modern day customer tailored solutions our special talent lies in the ability to be involved and to understand the market this allows us to offer traditional products in a cutting edge dressing along with modern conditions model based products can be tailored to the customers needs we lay emphasis on speed with simplicity availability and speed having a significant bearing on the product in the competi tive digital age financial products do not have to be boring and old fashioned a product should create a positive emotional bond and serve as a well designed rational tool at the customers service lhvs results demonstrate that we have only scratched the surface with this approach certainly the year marked a milestone in corporate growth on may as lhv group launched public trading of its shares on the nasdaq tallinn stock exchange a total of more than five thousand investors decided to invest in the success of lhv contributing to our mission of boosting domestic capital the listing of the shares marked the fulfilment of a long term objective and a logical continua tion of the listing of subordinated bonds in in order to support lhvs growth we engaged a total of eur million from our investors issuing two million new shares the issue was oversubscribed three times with the earnings per share amounting to nearly in lhv posted a net profit of eur million in this constitutes a eur million increase from last year with a return on equity for the shareholders of lhv profit was supported by strong customer activity strong credit quality and the extraordinary revenue generated by the acquisition of danske capital and its managed funds busi ness volumes and profits were boosted in all core activities at the end of september the financial supervision authority issued the annual supervisory review and evaluation process srep to lhv group and lhv bank based on the rates as at the end of the financial supervision authority established the minimum total capital ratio at and the minimum tier capital ratio at for lhv group and lhv bank in view of the financial supervision authoritys evaluation and by adding the nega tive scenario buffer lhv group established the groups minimum total capital ratio at and the minimum tier capital ratio at the net profit posted by the largest business unit lhv bank amounted to eur million having grown by eur million year on year loans to customers increased by eur million and totalled eur million eur million worth of new deposits were engaged during the year with the total volume of deposits amounting to eur million by the end of the year the number of customers grew by thousand year on year amounting to thousand as at the end of the year the bank received various recognitions at the beginning of the year in january nasdaq declared lhv bank the baltic member of the year in february the survey company dive declared lhv bank the bank with the best service in estonia in the annual most valuable employer poll conducted by cv keskus lhv bank rose from last years position to the position succumbing only to estonias largest companies and ranking second among banks in lhv bank focused mainly on gaining new customers enhancing user convenience and expanding the credit product offers we launched the corporate micro loan product and laid greater emphasis on small enterprise as lhv group tartu mnt tallinn info lhv ee lhv ee financing together with the provision of the required banking services mortgage loan a product whose existence elim inates the main obstacle for a private customer wishing to change his or her bank was launched at the end of the year we have brought our cost structure to a level which allows to provide the mortgage loan product on favourable conditions we opened a new more spacious office in tartu both lhvs investment seminars and the börsihai stock exchange shark game witnessed a record number of participants we have responded to growing public interest by enhancing the flexibility of our growth account product allowing all interested customers to start regular investment with small amounts lhv bank has positioned itself the third largest local bank in estonia our home market is estonia and we wish to excel here this means ranking among the top three banks with regard to both customer numbers and business volumes as well as efficiency and profitability per employee lhv asset management posted a net profit of eur million in eur million more than last year net fee and commission income increased by eur million to eur million the expense side was affected by a depreciation of client agreements recognised through acquisition of danske capital and its managed funds in the amount of eur million the revenue of lhv asset management is bound to decrease by an average of in february when the new management fee rates for the pillar funds take effect the lowering of the fees is a benefit gained by pillar fund customers from the danske capital transaction for lhv asset management the highlight of the year was signing the acquisition of as danske capital on january immediately after conclusion of the contract and acquisition of the required permits lhv launched the merger of the two management companies the merger took effect on july with the balance sheet date of may as a result of the merger danske capital was dissolved with lhv asset management becoming its legal successor lhvs business plan also includes merger of pension funds with a similar strategy similar mandatory pension funds will be merged on may in order to finance the purchase of danske capital the share capital of lhv asset management was increased by eur million to eur million in february in view of the current profit and lowering of the capital requirements as a result of merger of the management companies lhv asset management decreased its share capital by eur million before the end of the year the acquisition of and merger with danske capital generated eur million in intangible assets with the client agreements to be depreciated over a period of several years alongside actively managed pension funds lhv asset management also launched passively managed pension funds in the second half of the year the principal feature of passive funds is that these funds are not actively managed this in turn allows to offer passive funds at lower manage ment fees passive funds are invested in markets at all times with the yield mimicking the fluctuations of global markets we witnessed several stock market declines during the year but the markets rebounded quickly actively managed pension funds avoided major investment risks with the funds easily navigating all market fluctuations our flagship lhv pension fund l ranked second in its category in terms of yield as at the end of the year among conservative funds lhv pension fund s generated the highest yield for the year the number of active customers of lhvs manda tory pension funds grew by thousand year on year amounting to thousand the pension funds acquired through the purchase of danske capital had a total of thousand customers as at the end of the year the volume of funds managed by lhv grew by eur million to eur million of which eur million can be attributed to the transaction related pension funds mokilizingas posted a net profit of eur million in a eur million increase year on year profit was positively affected by the disposal of uncollectable receivables in the amount of eur million in the loan portfolio shrank by eur million during the year totalling eur million mokilizingas return on equity was mokilizingas is the second largest hire purchase provider on the lithu anian market and is recognised for the good service and transparent pricing in cooperation with a partner in the telecommunication sector mokilizingas also started offering hire purchase services in latvia furthermore the credit card product is currently in pilot stage all in all the year was successful for lhv we follow the principle of simplicity and modernity and believe in the opportunities for further growth in all key segments retail banking will focus on modern day payment solutions and development of a futuristic mobile bank the corporate loan and consumer financing segment in estonia and lithuania will concentrate on strong credit quality to provide a good foundation for growth in the credit portfolio private banking will pursue the next level of quality asset management will focus on fond yield and expansion of the client base in addition to organic growth we will continue to search for opportunities beyond estonian and lithuanian borders we will continue to make forward looking investments with the basic principle of achieving long term capital efficiency lhvs objectives for the upcoming years we endeavour to serve as the best financial service provider to private customers and smes who wish to save and invest financial assets we endeavour to serve as an attractive and recognised employer offering progress self fulfilment and growth we endeavour to do business in such a way as to ensure a return on equity we endeavour to become estonias most valuable publicly traded company madis toomsalu as lhv group tartu mnt tallinn info lhv ee lhv ee as lhv group activities holding company as lhv pank activities banking as lhv varahaldus activities fund management oü cuber tehnology activities money transfering uab mokilizingas activities consumer finance hire purchase as lhv finance activities hire purchase consumer finance management report as lhv group hereinafter the group or lhv is the biggest domestic finance group and capital provider in estonia the main subsidiaries of as lhv group are as lhv pank hereinafter the bank and as lhv varahaldus hereinafter lhv varahaldus lhv was established in by people with long experience in investing and entrepreneurship lhv s offices for client servicing are located in tallinn tartu and vilnius over people work in lhv over customers use banking services offered by lhv and the lhvs pension funds have over clients key events in ipo and listing shares in may lhv issued million new shares and listed all its shares on nasdaq tallinn stock exchange by that extremely successful transaction lhv group fulfilled the promise given to shareholders years ago lhv bank as lhv pank launched in mortgage loan product and that filled last larger gap in product offering to retail clients in addition lhv pank was the first one to start issuing nfc cards in estonia mokilizingas mokillizingas stopped providing financial leasing services and is now focusing only on consumer loans and hire pur chase products lhv varahaldus lhv varahaldus bought danske capital and merged that into its organization in lhv varahaldus was the first to launch index funds to offer low fee based second and third pillar pension fund as lhv group tartu mnt tallinn info lhv ee lhv ee operating principles the mission of lhv is to help create local capital and to support the development of local entrepreneurship lhv concentrates on active and independent customers with an entrepreneurial mind set for private consumers lhv mainly offers services of depositing and increasing monetary funds for corporate clients lhv offers flexible and suitable financing and managing of everyday financial operations the products and services provided by lhv are simple transparent and appropriate client interaction is mainly conducted using modern electronic communication chan nels costs avoided will reach the clients of lhv through affordable cost of everyday service lhv was created as a partnership and partnership is the central way of growing lhvs business new initiatives will be implemented faster and more successfully when partners have same interests and long term support of each other lhv is public company which is listed on the tallinn stock exchange since may so all clients and partners of lhv may be owners of lhv shareholders as lhv group has common shares with par value of euro as at as lhv group had shareholders shares belonged to the members of the supervisory board and management board and to their related parties shares belonged to estonian entrepreneurs and investors and to their related parties ten biggest shareholders as at number of shares percentage name of the shareholder as lõhmus holdings rain lõhmus viisemann investment ag ambient sound investments oü oü krenno as genteel as amalfi oü kristobal sia krugmans oü merona systems as lhv group tartu mnt tallinn info lhv ee lhv ee business environment even though the final figures are yet to be revealed the world economy is likely to have decelerated in for the second year in a row according to imfs january forecast there is a good chance that the trend will be reversed with the global economic growth expected to gain momentum and reach this year the dynamics of the previous six months suggest that developed and developing markets have taken a different path the new administration in the united states foresees a growth in budgetary expenditure similarly the forecast for the chinese economy has been upwardly revised against endless government stimulus in latin america however economic recovery has remained sluggish failing all expec tations mexico is suffering from us related political uncer tainty and the indian economy from backlashes of the bank note replacement the keyword for is elections in europe the year will see elections in both germany and france while the newly inaugurated president of the united states is already revealing the first major changes in course growth in protec tionist tendencies review of international trade treaties and prolongation of the long awaited structural reforms have the potential to re shape the long term economic outlook economic growth in the euro area hovered around in with the annual change expected to remain in the same ballpark the imf forecasts a marginal slow down in growth to the return of inflation fuelled by the recovery in oil and energy prices will start exerting pressure on domestic consumption the main engine for growth so far with the pressure on wage increase remaining non existent consumption is positively affected by consistent growth in employment enhanced corporate profitability is bound to attract investments the economic sentiment indicator of estonias key trade partners remained at five year highs strongly advancing in the second half of the year a major advance in sentiment was evident in finland and lithuania in suggesting an improvement in the countries economic outlook despite overall optimism the economic growth in sweden is losing pace dropping to in this can mainly be attributed to stabilisation of domestic consumption which up to now has been stimulated by low interest rates the growth in investment volumes is also expected to slow down significantly the forecast for swedish economic growth this year is around estonias interests revolve around a strong growth in swedish import volumes and investor friendly tendencies in swedish banks finland is continuing its sluggish recovery with the economy growing in consumer confidence rose above long term average in and is nearing ten year highs the highest among eu member states the positive trend is reflected in the general economy where domestic consumption has remained the main engine for growth for quite some time the key for the upcoming year lies in the ability to boost export volumes so far this has remained quite a stumbling block similarly to sweden estonias inter ests lie in further growth in finnish import volumes the economic growth in lithuania a major destina tion market for lhv slowed down to in this can mainly be attributed to shrinking volumes of projects financed through eu assistance funds which also had an adverse effect on the construction sector strong domestic consumption remained the main engine for growth fuelled by a quick increase in wages and decrease in unemploy ment the lithuanian economic growth is expected to improve compared to the growth last year growth in the volume of eu financed projects could provide a further stimulus with the transition to the new programming period expected to contribute the estonian economy in can mainly be character ised by failure to meet the overly optimistic forecasts the estimated growth of is the slowest in the post crisis period difficulties in the oil shale industry and the energy sector mainly conditioned by external factors also had their bearing the poor investment volumes in recent years have impeded the potential growth of estonian economy the economic forecast published by the european central bank in december foresees a growth for the year income increase is expected to continue but the return of inflation will have a significant impact on growth in real wages conse quently export and investments are becoming increasingly important alongside domestic consumption government expenditure is expected to rise fuelled among other things by the eu presidency and the local government elections in the autumn of the policy measures implemented by the new coalition have caused a greater than usual uncer tainty with regard to the outlook for the upcoming year as lhv group tartu mnt tallinn info lhv ee lhv ee key economic indicators of estonia bank of estonia forecast nominal gdp eur billion gdp volume private consumption expenditures government consumption expenditures fixed capital formation exports imports cpi unemployment rate of the labour force current account of gdp budget balance of gdp numbers reported are annual rates of change in per cent if not noted otherwise actual gdp and its components are chain linked including npish the budget balance forecast considers only those measures on which sufficient information was availanble at the date of the forecast sources eesti pank eurostat statistics estonia lhv is not expecting any abrupt economic changes in estonia over the month horizon economic growth is expected to pick up pace but is unlikely to achieve its full potential the policy measures of the new coalition will be a major factor rapid growth in wages is exerting pres sure on the already dropping corporate profits corporate adaptability and successful enhancement of productivity will thus play an increasingly important role a qualitative leap in investments is required to achieve the objectives in terms of economic sectors higher than average risks are evident in the processing industry and energy related sectors which are facing wage pressures special attention should also be paid to transport and warehousing as well as the risks of the construction sector where the drop in profit figures has been significant compared to previous periods lhv has also taken a more conservative approach to the real estate market carefully monitoring the dynamics on the local and scandinavian markets rental property projects involve a significant risk of insufficient demand for absorbing the developed volumes in the near future with corrections expected in either rental prices or vacancy rates on a positive note the financing environment remains favourable saw a growth in the balance of loans taken from credit institutions the loan to deposit ratio and overdue loans have maintained their post crisis lows with the record low interest rates and interbank competition having a positive impact on local entrepreneurship the opportunities should be further exploited lhv stands ready to boldly and efficiently contribute to the search of new sources of growth offering a long term partnership to enterprises and making an effort to create additional value overdue loans share in loan portfolio overdue loans over days overdue loans days overdue loans days overdue loans up to days source bank of estonia as lhv group tartu mnt tallinn info lhv ee lhv ee financial results volume of deposits and loans received volume of net loans volume of funds net interest income net fee income net financial income net income expenses operating profit loan provisions income tax expense profit from discontinued operations profit including attributable to owners of the parent eur million the summary table of financial results does not separately show income and expenses of the finnish business operations in the annual report they are presented separately as a discontinued operations the groups pre tax profit for amounted to eur million and net profit totalled eur million pre tax profit is more than a year before eur million from continued operations net interest income grew and net fee income increased financial income grew by the end of december the total volume of the groups loan portfolio less impairments amounted to eur million december eur million the volume of portfolio increased in a year in the loan portfolio the majority is corporate loans that increased in a year to eur million eur million the portfolio of retail loans increased in a year amounting to eur million eur million the volume of groups deposits increased in a year and totalled eur million by the year end eur million the share of demand deposits of all deposits increased and reached operating expenses increased in a year by business units in consolidated figures in as lhv pank earned profit of eur million lhv varahaldus earned eur million and uab mokilizingas earned a profit of eur million lhv group itself ended the year with a loss of eur million due to the fact that it does not engage in operating business and the main expense in the group is the interest expense from issued subordinated debt liquidity and capitalization the liquidity coverage ratio lcr of the group calculated according to the definitions of the basel committee was at the end of december the group includes cash and bond portfolios into a liquidity buffer accounting for of the balance sheet total among its liquidity buffer the groups loan to deposit ratio at the end of was the groups level of own funds as at was eur million million the group is well capitalised as at the end of the reporting period with a capital adequacy level of and tier capital ratio of the group uses the standard method for the calculation of credit risk and market risk capital requirements and the basic indicator approach for the calculation of operational risk capital requirements the group has complied with all capital requirements during the financial year and in previous year as lhv group tartu mnt tallinn info lhv ee lhv ee each year an internal capital adequacy assessment process icaap is performed the goal of which is to identify potential capital needs in addition to regulatory capital requirements leverage ratio calculated according to basel committee on banking supervision requirements as at was leverage ratio is calculated as groups total tier own funds divided by group s total risk exposure measure incl risk position on assets and off the balance sheet liabilities as at the end of the reporting period the leverage ratio is arithmetic mean of the monthly leverage ratios over the last quarter key figures eur million change change change net profit net profit attributable to owners of the parent average equity return on equity roe average assets return on assets roa net interest income average interest earning assets net interest margin nim spread cost to income ratio return on equity is calculated based on lhv group net profit and equity attributable to owners of the parent and does not include non controlling interest explanations average equity attributable to owners of the parent equity of current year end equity of previous year end return on equity roe net profit attributable to owners of the parent average equity attributable to owners of the parent average assets assets of current year end assets of previous year end return on assets roa net profit average assets net interest margin nim net interest income average interest earning assets spread yield on interest earning assets cost of interest bearing liabilities yield on interest earning assets interest income average interest earning assets cost of interest bearing liabilities interest expenses average interest bearing liabilities cost to income ratio total operating expenses total income as lhv group tartu mnt tallinn info lhv ee lhv ee risk management the purpose of risk management is to identify assess manage and control all risks related to the activities of the group in order to ensure the group s reliability stability and profitability the principles of identification measuring reporting and control of risks at the group are set out in the policies and procedures approved by the supervisory board and daily risk management is organised by risk manage ment unit in the group risk management is based on three lines of defence business units as the first line of defence are responsible for taking and managing risks the second line of defence which includes the risk management unit is responsible for developing and maintenance of the risk management framework the third line of defence internal audit function carries out independent control over processes operations and control functions in the entire group the rules and procedures of risk management are regularly reviewed at least once a year and updated in case of a need detailed overview of the risks taken by the group is presented in note of the annual report paid in share capital share premium statutory reserves transferred from net profit other reserves accumulated profit deficit intangible assets subtracted net profit for accounting period dividend offer including income tax non controlling interests total tier capital subordinated debt total tier capital exceeding limitations of subordinated debts and preference shares net own funds for capital adequacy calculation capital requirements central governments and central bank under standard method credit institutions and investment companies under standard method companies under standard method retail claims under standard method public sector under standard method housing real estate under standard method overdue claims under standard method investment funds shares under standard method other assets under standard method total capital requirements for covering the credit risk and counterparty credit risk capital requirement against foreign currency risk under standard method capital requirement against interest position risk under standard method capital requirement against equity portfolio risks under standard method capital requirement against credit valuation adjustment risks under standard method capital requirement for operational risk under base method total capital requirements for adequacy calculation capital adequacy tier capital ratio audited numbers which include the dividend offer capital base in eur thousands as lhv group tartu mnt tallinn info lhv ee lhv ee overview of the groups subsidiaries in as lhv pank consolidation group as lhv pank is based on estonian capital the banks clients include private individuals small and medium sized compa nies and institutional investors the branch offices of the bank are located in tallinn tartu and vilnius will be closed riga office was closed as lhv pank employs over people more than clients use the banks services as lhv pank is one of the largest brokers on nasdaq omx baltic stock exchanges and the largest broker for baltic retail investors in international markets the bank owns of the subsidiary as lhv finance that provides hire purchase and consumer finance services business activities has been year of growth in all business segments growth of loans was according to expectation and deposits transaction volumes exceeded the budget the volume of deposits grew by year on year and reached million euros by year end million euros as a result of the overall low interest rate environ ment a change has occurred in the deposit structure as the deployment of money in term deposits is no longer as attractive the bank has used the liquidity account concept to attract more active customers during the we have significantly managed to reduce deposits concentration at the end of year there were only five depositors exceeding of total deposits in total their share was of total deposits we have managed significantly to increase the share of retail deposits the proportion of demand deposits has increased accounting for of all deposits as at the end of december one year ago the loan portfolio volume grew by year on year and amounted to eur million at the end of december eur million the majority of the loan portfolio is made up of corporate loans to legal entities which grew by year on year to eur million eur million the loan portfolio given to private individuals grew by year on year to eur million eur million as lhv pank issued a loan to the other subsidiary of as lhv group uab mokilizingas the amount of which at the year end was eur million eur million in autumn bank started offering mortgage loans clients have shown very high interest from the very first days approximately of applicants have been lhv clients and another half new clients as new developments lhv connect and new functionality in mobile were launched lhv connect offers corporate customers a possibility to link their bank account informa tion directly to their accounting system lhv connect allows making payments receive notifications and import account statements banks organizational structure is divided into three larger business lines retail banking private banking and corporate banking and three larger support functions it finance and support and risk management marketing and communica tion and human resource department are across the group during the year corporate loan portfolio was divided into two credits below thousand euros were given over to retail banking and all credits exceeding thousand euros remained in corporate banking shifting smaller loans to retail banking was important to keep corporate banking and credit analyses focus on larger and more individual approach requiring credit clients credits below thousand euros are analysed more cost efficient scoring process at the end of first quarter latvian branch was closed the banks operations in latvia were small scale in its business pursuits the bank has been governed above all by the business interests of its customers and endeavours to keep pace with the customers estonian enterprises currently need more services and support on the western european front at the end of year new office was opened in tartu in second half of year the internal personnel satisfaction survey was carried out the respondent percentage and also the marks have been all time high bank received several awards we rec eived member of the year award by nasdaq research agency dive nominated us bank with best service in baltics according to yearly poll made by cv keskus lhv pank raised to eight spot as lhv group tartu mnt tallinn info lhv ee lhv ee financial results the net profit of the bank in amounted to million euros up compared to the previous year million euros net interest income grew by and net fee and commission income increased by financial income was higher by compared to the previous year the large increase in net interest income is attributable to a growth in lending and deposit volumes among fee and commission income the greatest contri bution continues to come from security brokerage income income on cards and settlements are also showing a growth trend operating expenses grew by year on year which was expected given the current phase of high growth and new business development loan losses were higher compared to the previous year organisation the organisational structure of the bank continues to be divided in six major areas retail banking private banking business banking information technology finance and operations and risk management incl compliance statement of profit or loss and other comprehensive income eur million change net interest income net fee and commission income net gains from financial assets total net operating revenues other income operating expenses loan losses discontinued operations net income volumes eur million change loan portfolio bond portfolio deposits equity number of customers holding assets in bank as lhv group tartu mnt tallinn info lhv ee lhv ee as lhv varahaldus as lhv varahaldus hereinafter lhv asset management is involved in the management of investment funds the prin cipal units of lhv asset management are the investment unit risk management unit operations unit and sales and customer service unit for lhv asset management the highlight of the year was the acquisition of and merger with as danske capital the contract of purchase and sale was entered into with danske bank a s and danske bank a s estonia branch on january upon entry into the contract lhv started applying for the permits required for the purchase of danske capital the permits were issued in april immediately after conclusion of the transaction of purchase and sale of shares on may lhv launched the merger of the two manage ment companies the merger took effect on july with the balance sheet date of may lhv asset management served as the acquiring company in the merger as a result of the merger danske capital was dissolved with lhv asset management becoming its legal successor three funds with no customers were dissolved upon the merger in august lhv world equities fund as the acquiring fund merged with danske invest new europe fund lhv asset management s business plan also includes merger of pension funds with a similar strategy similar pillar funds will be merged on may in lhv asset manage ment set up new passively managed pillar and pillar pension funds with their operations launched on november as at the end of the year lhv asset management managed mandatory pension funds the figure will drop to after the planned fund mergers voluntary pension funds after the planned merger and ucits and offered invest ment consulting to one ucits sef lhv persian gulf fund which merged with the lhv persian gulf fund in in order to finance the purchase of danske capital the share capital of lhv asset management was increased by eur million to eur million in february in view of the current profit and lowering of the capital requirements as a result of merger of the management companies lhv asset management decreased its share capital by eur million before the end of the year lhv asset management posted a net profit of eur million in eur million more than the year before net fee and commission income increased by eur million to eur million eur million of the increase in revenue can be attributed to business volumes added as a result of merger with danske capital operating expenses increased by eur million to eur million the acqui sition of and merger with danske capital generated eur million in intangible assets with the client agreements to be depreciated over a period of several years depreciation of client agreements added eur million to expenses in as the volume goes up the management fees for pillar funds will be lowered the management fees of lhv s actively managed pension funds will thus be lowered from february when the new rates will take effect by an average of the lowering of the management fees is a benefit gained by pillar fund customers from the danske capital transaction the number of lhv asset management s mandatory pension fund customers grew by thousand year on year amounting to thousand the pension funds acquired through the purchase of danske capital had a total of thousand customers as at the end of the year the volume of funds managed by lhv grew by eur million to eur million of which eur million can be attributed to the transaction related pension funds as at the end of the year lhv asset management s pillar market share grew from to in terms of volume and from to in terms of active customer numbers in we witnessed several stock market declines during the year but the markets rebounded quickly actively managed pension funds avoided major investment risks with the funds smoothly navigating all market fluctuations our flagship lhv pension fund l ended the year by ranking second in its category among conservative funds lhv pension fund s generated the highest yield for the year in the parliament adopted the long awaited invest ment funds act regulating the functioning of management companies and funds with the entry into force of the act on january the capital requirements applied for lhv asset management will be lowered and the exit fees of pillar funds will be abolished revenue generated in eur million financial results eur million change net fee income other financial income total net operating revenues operating expenses net profit assets under management number of clients in pension funds thousands as lhv group tartu mnt tallinn info lhv ee lhv ee uab mokilizingas hire purchase in hire purchase hp segment mokilizingas continues to work in a strong collaboration with main partners bite lietuva and kesko senukai launching seasonal campaigns and supporting the sales hp sales amounted to eur million eur million the number of contracts signed decreased by compared to average contract amount remained stable margins remained at similar levels with a slight decrease compared to due to increased year end campaigns consumer loan consumer loan cl sales amounted to eur million and decreased by compared to the number of contracts signed remained stable however average contract amount decreased by margins are stable with a slight increase compared to after the successful launch of credit card pilot the second phase started in total number of cards issued reached thousand by the end of with a total eur million credit limit average credit limit is eur card which is above the expectation average spending per month is also higher than expected however usage is quite low of activated cards have at least one transaction objectives of will be dedicated to the enhance ment of card usage rate and product knowledge allocation to the customers especially concentrating on the existing card owners to get in line and exploit high revolving balance perspectives the main event for non bank consumer credit segment in lithuania was a change of regulation in which tightened the evaluation of clients eg more conservative limits to evaluation of income and liabilities of married clients this also brought turbulence in the market and had a signif icant impact for sales especially cl in hp side mokilizingas and bite developed an invoice consolidation project the clients now have an option to receive single invoice from bite and mokilizingas for telco services and hired handsets respectively this allows more flexibility for both companies to promote additional sales as well as use telco input for management of late payments as planned mokilizingas has launched a renewed self ser vice website this is a major step towards unifying all customer related processes since all the following projects will be designed in the same manner the new process allows customers to take consumer loans without compul sory registration the consent ratification and general admis sion process is quicker and more user friendly senukai credit card sales were launched in december though the sales campaign will start in mokilizingas also implemented new integration with compulsory credit database provided by bank of lithuania will be dedicated to further optimizing internal processes and developing more effective financing solu tions to maximize sales in relation to tightened regulation of bank of lithuania further developments will include new services aimed to improve customer experience at all pos and e commerce platforms as lhv group tartu mnt tallinn info lhv ee lhv ee governance of the lhv group supervisory board rain lõhmus is a founder of lhv and the chairman of the supervisory board he is a member of the supervisory boards of as lhv pank as audentes and as arco vara and a member of the management boards and supervisory boards of other companies rain lõhmus graduated from tallinn university of technology and management program of harvard business school he is a member of estonian business association rain lõhmus owns and his related parties own shares of as lhv group raivo hein is the owner and a member of management board of oü kakssada kakskümmend volti he is a member of the supervisory board of as lhv pank and a member of management boards and supervisory boards of other companies raivo hein graduated from tallinn university of technology raivo hein does not own shares of as lhv group oü kakssada kakskümmend volti owns shares of as lhv group heldur meerits is the owner and a member of the manage ment board of as amalfi he is a member of the supervisory boards of as lhv pank as audentes and kodumaja as and a member of the management boards and supervisory boards of other companies heldur meerits graduated from the faculty of economics and business administration of tartu university he is a member of the supervisory boards of sa avatud eesti fond sa dharma and other foundations heldur meerits does not own shares of as lhv group as amalfi owns shares of as lhv group tiina mõis is the owner and manager of as genteel she is a member of the supervisory boards of as lhv pank as baltika and other companies tiina mõis graduated from tallinn university of technology tiina mõis does not own any shares of as lhv group as genteel owns shares of as lhv group sten tamkivi is the ceo of teleport prior to co founding teleport as an entrepreneur in residence at andreessen horowitz sten served as an early executive at skype he holds a number of management and advisory board seats sten tamkivi owns shares and seikatsu oü owns shares of as lhv group tauno tats is a member of the management board of ambient sound investments oü he is a member of the supervisory board of eften kinnisvarafond as and a member of management boards and supervisory boards of other companies tauno tats graduated from tallinn university of technology tauno tats does not own shares of as lhv group ambient sound investments oü owns shares of as lhv group as lhv group tartu mnt tallinn info lhv ee lhv ee andres viisemann is the founder of lhv and the manager of lhv pension funds he is a member of the supervisory boards of as lhv pank as lhv varahaldus and as fertil itas and a member of the supervisory boards and manage ment boards of other companies andres viisemann grad uated from the university of tartu and obtained a masters degree at insead in international business management he is a member of the supervisory board of estonian health insurance fund andres viisemann does not own shares of as lhv group his related parties own shares of as lhv group madis toomsalu is the chairman of the supervisory boards of as lhv pank and as lhv varahaldus madis toomsalu owns shares of as lhv group management board supervisory and management boards of the subsidiaries as lhv pank supervisory board madis toomsalu chairman raivo hein heldur meerits tiina mõis rain lõhmus sten tamkivi andres viisemann management board erki kilu chairman jüri heero andres kitter meelis paakspuu indrek nuume martti singi as lhv finance supervisory board erki kilu chairman rain lõhmus veiko poolgas jaan koppel management board nele roostalu as lhv varahaldus supervisory board madis toomsalu chairman erki kilu andres viisemann management board mihkel oja chairman joel kukemelk cuber tehnology oü management board jüri laur uab mokilizingas supervisory board as lhv group as lhv pank kūb razfin uab management board erki kilu mantas jonuška benas pavlauskas jurgis rubaževičius ceo benas pavlauskas as lhv group tartu mnt tallinn info lhv ee lhv ee lhv listed shares and subordinated bonds lhv group redeemed a promise given to shareholders years ago by issuing million new shares and by listing all shares on tallinn nasdaq stock exchange on may it was the first listing in more than five years the share was well received to the stock and by the end of the year the share price had increased and reached the value of eur last price adjusted volume last price adjusted volume during the year million shares in the amount of eur million changed the owner on stock exchange at the end of the year lhv group had over shareholders from whom were also customers of lhv pank they were followed by swedbank customers and seb customers of the investors were companies and were private individuals lhv has issued two subordinated debt issues on nasdaq tallinn stock exchange the first of them will end in and the second in in no new bonds were issued nor redeemed bonds were traded in in the total amount of eur million at the end of the year lhv group had almost bondholders of them were also customers of lhv pank they were followed by the customers of swedbank and seb of the investor base were companies and were private individuals share options in the general meeting of shareholders of as lhv group approved the criteria for the granting of share options to the management board members and key employees and department managers and key employees of group companies that are part of as lhv group the aim of issuing share options is to align the interests of management board members and employees more effectively with the interests of shareholders and customers another objective is to offer a system of compensation on the labour market that is equal to competitors and comprehensive the compensation system in lhv consists only from two pillars base salary and share options there are no other hidden bonuses in form of additional payments free of charge services or company cars similarly to the share options were issued also in and in most likely the similar share options will be issued in the granting and size of share options was dependent upon the successful achievement of operational targets of the overall company and the targets of individual management board members and employees in the beginning of the share options were granted to people in the amount of thousand euros people in the amount of thousand euros people in the amount of thousand euros the company entered into share option agreements with the members of the management board and employees for a three year term for the granting of share options share options issued in can be exer cised between the period of and shares with nominal value of euro can be acquired for euros per share share options issued in can be exer cised between the period of and shares with nominal value of euro can be acquired for euros per share share options issued in can be exer cised between the period of and shares with nominal value of euro can be acquired for euros per share members of the management board and employees were unable to opt for the determined amount in cash instead of share options share options cannot be exchanged sold pledged or encumbered share options can be inher ited the share option agreement will become void upon premature termination of the employment contract at the employees initiative exemptions to which can be made by the supervisory board of as lhv group and due to termi nation of employment contract on an extraordinary basis by the employer for a cause related to the employee pursuant to the credit institutions act the supervisory board of as lhv group may reduce the number of share options issued or cancel share options if the overall financial results of the company have materially deteriorated compared to the previous period a member of the management board or employee of the company no longer meets award criteria the company no longer meets the prudential ratios the risks of the company are not adequately covered with own funds or determination of the performance pay was based on information which was inaccurate or incorrect to a material extent as lhv group tartu mnt tallinn info lhv ee lhv ee this report is presented in accordance with the accounting act of the republic of estonia and provides an overview of the governance of as lhv group lhv and compliance of governance with the corporate governance recommen dations of nasdaq omx tallinn stock exchange and the financial supervision authority lhv adheres to the corpo rate governance recommendations unless otherwise specified in this report general meeting lhv is a public limited company with its governing bodies being the general meeting of shareholders the supervisory board and the management board the general meeting where shareholders can exercise their rights is the highest governing body of lhv the primary duties of the general meeting include amendment of the articles of association increase and decrease of share capital decisions on issuance of convertible bonds election of and extension of the powers of supervisory board members premature removal of supervisory board members approval of the annual report and profit allocation approval of the share option programme determination of the number of auditors appointment and recall of auditors and other decisions within the competence of the general meeting as provided by law and the articles of association a resolution on amendment of the articles of associa tion shall be adopted if at least two thirds of the votes who participate in the meeting are in favour a resolution on amendment of the articles of association shall enter into force as of the making of a corresponding entry in the commercial register the resolution of the general meeting on the amendment of the articles of association minutes of the general meeting and the new text of the articles of association shall be attached to the application filed to the commercial register a shareholder has the right to participate in the general meeting address the general meeting with regard to items on the agenda ask relevant questions and make proposals the general meeting is called by the management board the annual general meeting with the approval of the annual report on the agenda is held at least once a year the management board calls the annual general meeting within six months after the end of the financial year at the latest the management board gives at least three weeks notice of the annual or special general meeting the agenda of the general meeting proposals of the management board and supervisory board draft reso lutions and other relevant materials are made available to the shareholders prior to the general meeting materials are made available on lhvs website shareholders are given the opportunity to ask questions about items on the agenda before the general meeting with the questions and responses published on lhvs website the list of shareholders entitled to participate in the general meeting is established based on the share register seven days before the general meeting one annual general meeting of shareholders was held in no special general meetings were held the annual general meeting of shareholders held on april resolved to list all shares of lhv on the nasdaq tallinn stock exchange baltic main list with the share capital of lhv conditionally increased the meeting also approved the annual report distributed the profit for the financial year gave an overview of the economic results for and approved the dividend policy the general meeting was held in the estonian language the annual general meeting of was chaired by daniel haab head of legal department of as lhv pank who introduced the procedure of the general meeting and the procedure for asking questions regarding the companys activities from the management board in member of the management board erkki raasuke chairman of the supervisory board rain lõhmus members of the supervisory board andres viisemann tiina mõis heldur meerits raivo hein sten tamkivi and tauno tats and auditor verner uibo attended the annual general meeting none of the shareholders hold any shares which would grant them special control rights or voting rights lhv is not aware of any agreements in place between shareholders that would address the coordinated exercise of shareholder rights shareholders with significant influence include rain lõhmus and parties related to him and andres viisemann and parties related to him corporate governance report as lhv group tartu mnt tallinn info lhv ee lhv ee management board duties of the management board the management board is a governing body of lhv charged with the task of representing and managing lhv members of the management board are elected and removed by the supervisory board the consent of the candidate is required for the election of a member of the management board pursuant to the articles of association the management board has one to five members a member of the manage ment board is appointed for a term of five years unless otherwise decided by the supervisory board extension of the term of office of a member of the management board shall not be decided earlier than one year before the planned date of expiry of the term of office and not for a period longer than the maximum term of office prescribed by the law or the articles of association if the management board of lhv has more than two members the supervisory board must appoint the chairman of the management board the chairman of the management board organises the work of the management board the supervisory board may remove a member of the management board for any reason a member of the management board may resign for any reason by informing the supervisory board thereof the rights and obligations arising from the contract entered into with a member of the management board shall expire in accordance with the contract erkki raasuke served as the member of the management board of lhv until december madis toomsalu was elected the new member of the management board on the same day the supervisory board has entered into a management board member contract with madis toomsalu the supervisory board establishes the remuneration princi ples for management board members and sets the sever ance compensation policy for premature termination of the management board member contract the management board makes everyday management decisions at its own discretion governed by the best interests of lhv and its shareholders and not his or her personal interests or the interests of the controlling share holder the duties of the management board member include day to day management of lhv representation of the company management of lhvs operations in foreign markets and investor communication coordination of the development of lhvs strategy and its implementation by serving as an active chairman of the supervisory boards of the major subsidiaries of lhv the management board shall ensure the functioning of risk management and internal control within lhv depending on the business area pursued by lhv lhvs internal control system covers all activities carried out by the supervisory board management and staff members of lhv with the aim of ensuring efficiency of operations adequate risk manage ment reliability and accuracy of internal and external reporting unconditional compliance with all legal acts the internal audit system covers all business support and control units the objective of risk management within lhv is to recog nise correctly measure and manage the risks on a wider horizon the objective of risk management is to enhance corporate value by minimising damage and reducing vola tility of results risk management is based on a strong risk culture and built on the principle of three lines of defence with the first line of defence i e business areas being responsible for risk taking and day to day management of risks the second line of defence i e the risk management area is responsible for developing risk management methods and reporting on risks the third line of defence i e internal audit exercises independent supervision over the entire organisation management remuneration principles the purpose of establishing remuneration principles within lhv is the organisation of fair motivating and transparent remuneration in accordance with the law competence for the establishment of remuneration for members of the management board shall lie with the super visory board the remuneration committee shall review the basis of management remuneration on an annual basis in assessing the activities of the members of the manage ment board the remuneration committee shall above all consider the professional duties of the management board member his or her activities the activities of the entire management board as well as lhvs financial position the current status of and outlook for business operations in comparison with companies operating in the same sector of economy the remuneration payable to a member of the management board including the options programme must motivate the individual to operate within the best interests of lhv the basic salaries of members of the management board shall be set forth in the employment contract and the basic salary of the chairman of the management board in the management board member contract the principles of remuneration of management or staff members engaged in internal control and risk management shall ensure their independence and objectivity in the performance of their risk management internal control duties the remuneration of this staff shall not depend on the results posted by the units supervised the objectives shall be described on the level of individual staff members as lhv group tartu mnt tallinn info lhv ee lhv ee when measuring the key quantitative and qualitative results and assessing the risk lhv shall be governed by the following principles the performance criteria established by lhv must not stimulate excessive risk taking or sale of unsuitable products the performance criteria must not merely contain performance efficiency indicators e g profit revenue profitability expenditure and volume indicators or market based indicators e g stock price or bond yield but must also be adjusted with risk based indicators e g capital adequacy liquidity the criteria used for measuring risks and results shall be as closely related to the decisions of the staff member whose results are being appraised as possible and should ensure that the process of the establishment of the remuneration would have an appropriate impact on the staff members professional behaviour a good balance of both quantitative and qualitative as well as absolute and relative criteria shall be used in the specification of the performance criteria quantitative criteria shall cover a sufficient period of time so as to consider the risks taken by the staff members or business units the criteria shall be risk ad justed and contain economic efficiency indicators examples of qualitative criteria include achievement of strategic objectives customer satisfaction adherence to the risk management policy adherence to internal or external rules of procedure management skills crea tivity motivation and co operation with other business units and the internal control function lhv applies the long term performance pay programme i e the options programme an annual performance pay shall be added to the basic salaries of management board members top managers and key persons included in the options programme including staff members equivalent to management members the amount and disbursement of which corresponds to the achievement or non achievement of individual targets or lhvs targets the basis for estab lishment of performance pay in the options programme shall be objective and justifiable with the time period for which performance pay shall be paid previously determined the performance pay instruments consist fully of stock options severance compensation payable to a member of the management board shall correspond to his or her previous work results no severance compensation shall be paid if this is liable to damage the interests of lhv the size of the severance compensation shall correspond to the results achieved over time and to the remuneration received conflicts of interests management board members submit an annual declara tion of economic interests and conflicts of economic inter ests no transactions approved by the supervisory board were concluded between lhv and its management board members their close relatives or related parties in during his term of service erkki raasuke did not serve as a management board member in any other issuer erkki raasuke served as the chairman of the supervisory board of as eesti energia madis toomsalu is not a management board member or supervisory board member in any other issuer management board members have not been granted authority to issue or buy back shares supervisory board the supervisory board is a governing body of lhv charged with the task of planning the activities of lhv organising the management of lhv and conducting supervision over the activities of the management board the supervisory board determines and regularly reviews lhvs strategy the general plan of action risk management principles and the annual budget the supervisory board has five to seven members the members of the supervisory board are appointed for a term of three years the members of the supervisory board elect amongst themselves the chairman of the supervisory board charged with the task of organising the activities of the supervisory board rain lõhmus chairman andres viisemann tiina mõis heldur meerits raivo hein tauno tats and sten tamkivi serve as members of the supervisory board the term of office of supervisory board members is set to expire on may the general meeting of shareholders has resolved to establish for the supervisory board members a gross remuneration of euros per each meeting of the supervisory board where the member participates supervisory board members submit an annual declaration of economic interests and conflicts of economic interests no significant transactions were concluded between lhv and its supervisory board members their close relatives or related parties in a total of eleven ordinary meetings and one extraordinary meeting of the supervisory board was held in rain lõhmus andres viisemann and raivo hein participated in all meetings of the supervisory board in tiina mõis tauno tats and sten tamkivi were absent on one occasion and heldur meerits on two occasions all members of the supervisory board participated in more than half of the meetings of the supervisory board the supervisory board has formed three committees charged with the task of advising the supervisory board in issues related to audit remuneration and risk as described below within the framework of supervisory board meetings the supervisory board was regularly briefed on the operating and financial results of lhv and its subsidiaries in the supervisory board approved the conditional increase of share capital public offering of shares issue of new shares and the listing of all shares on the nasdaq tallinn stock exchange baltic main list the supervisory board author as lhv group tartu mnt tallinn info lhv ee lhv ee ised and approved the results of the subscription of shares and the allocation of shares the strategy of lhv was the principal topic of discussion at the meeting held in august the supervisory board approved the dividend payment prin ciples and dividend policy the acquisition of the shares of danske capital as by as lhv varahaldus and the change in the share capital of as lhv varahaldus the supervisory board approved the annual report the report of the supervisory board and the issue of share options for the results posted in the share capital of as lhv pank was increased erkki raasuke was removed from the posi tion of the management board member and madis toomsalu was appointed the new member of the management board audit committee the audit committee serves above all as an advisory body for the supervisory board with regard to accounting audit risk and capital committee internal control and internal audit supervision budgeting and compliance the audit committee is governed in its activities by the auditors activities act and the rules of procedure established by the supervisory board of lhv the audit committee has been charged with the task of monitoring and analysing the processing of financial infor mation in the extent necessary for preparing quarterly and annual financial statements efficiency of risk management and internal control the procedure for audit of the financial statements or consolidated financial statements as well as the independence of the audit firm or a sworn auditor repre senting the audit firm on the basis of the law and compli ance of their activities with the requirements of the auditors activities act the audit committee is responsible for making proposals to the supervisory board and providing recommendations for appointment or removal of the audit firm appointment or removal of the internal auditor prevention or elimination of problems or inefficiencies within the organisation and compliance with legal acts and good professional practice pursuant to the rules of procedure the audit committee gathers for meetings at least on a quarterly basis in the audit committee gathered on six occasions at five meetings the internal auditor gave an overview of the audits conducted and the audit findings the last meeting of the year discussed the findings of the external auditor on the basis of the interim audit for the last two meetings of the year discussed the internal audit work plan for the audit committee consists of a minimum of two members members of the audit committee have been appointed by the supervisory board for a term of one year with the aim of ensuring diversity of competences and inde pendence of committee members gerli kilusk chairman marilin hein and tauno tats serve as members of the audit committee gerli kilusk is a partner management board member and attorney at law with law firm primus gerli kilusk does not hold any shares in lhv marilin hein is the cofounder and cfo of eften capital as marilin hein does not hold any shares in lhv astrum oü a company owned by marilin hein holds shares of as lhv group tauno tats is a member of the supervisory board of lhv and the manage ment board of ambient sound investments oü tauno tats does not hold any shares in lhv ambient sound invest ments oü holds shares of as lhv group the remuneration payable to members of the audit committee shall be determined by the supervisory board of lhv members of the audit committee are paid a monthly net remuneration of euros the internal audit unit is governed in its activities by the auditors activities act credit institutions act other regu lations and the rules of procedure for internal audit as approved by the supervisory board of lhv the internal audit unit presents the supervisory board with an overview of the audits conducted and audit findings on a quarterly basis an overview of the audits carried out during the year and significant findings as well as performance of the annual plan is given once a year the annual internal audit plan is prepared on the basis of the requirements set forth by law or contract as well as the internal audit units assessment of the risks evident in the business areas and products aimed at covering all major areas and products with sufficient risk based regularity the internal auditor discusses all audit findings with the heads of the corresponding areas as well as the management board of the company and agrees upon the action plan for elimi nation of deficiencies the internal audit unit conducts quar terly follow up audits in order to monitor adherence to the agreed action plan to ensure efficiency the internal audit unit closely co operates with the risk control and compli ance units as well as the external auditor while maintaining its independence the internal auditor is invited to all super visory board meetings of lhv as lhv pank and as lhv varahaldus other important meetings of committees estab lished within group companies if possible and necessary and all management board meetings of as lhv pank and as lhv varahaldus in the reporting period the internal audit unit had two employees in estonia and one employee in lithuania remuneration committee the remuneration committee has been charged with the task of evaluating the principles of remuneration within lhvs subsidiary the credit institution as lhv pank and the impact of any remuneration related decision on adher ence to the requirements established for the management of risks own funds and liquidity the remuneration committee consists of a minimum of three members the remuneration committee conducts supervision of the remuneration of management board members and staff members evaluates implementation of the remuner as lhv group tartu mnt tallinn info lhv ee lhv ee ation policy at least on an annual basis and where neces sary proposes to update the remuneration principles and prepares the related draft resolutions for the supervisory board the remuneration committee held one meeting in in january the committee discussed and made a proposal to the supervisory board with regard to the recipients of share options and the related amounts for the results posted in as well as with regard to the pricing of share options the share options of resigning staff members and the remu neration of management board members rain lõhmus andres viisemann and until december erkki raasuke chairman served as members of the remuneration committee madis toomsalu has been serving as a member and the chairman of the remuneration committee from december onwards rain lõhmus is the founder of lhv and the chairman of the supervisory board of lhv rain lõhmus holds shares as lõhmus holdings shares and oü merona systems shares of as lhv group andres viisemann is the founder of lhv member of the supervisory board of lhv and manager of the pension funds of as lhv varahaldus andres viisemann does not hold any shares of as lhv group viisemann investments ag holds shares and viisemann holdings oü share of as lhv group madis toomsalu is a member of the management board of lhv madis toomsalu holds shares of lhv no remuneration is paid to members of the remuneration committee risk and capital committee the risk and capital committee has been charged with the task of advising the supervisory board and management board of lhv with regard to risk management principles and risk tolerance assessing the risks taken by lhv and moni toring implementation of the risk policy within lhv group companies the risk and capital committee also carries out supervision of the implementation of risk management principles by the management board in accordance with the instructions given by the supervisory board checks that the probability and timing of risk capital liquidity and revenue is taken into account in remuneration principles and that the business model and risk management principles are taken into account in the fees established for customers pursuant to the rules of procedure of the risk and capital committee the meetings of the risk and capital committee shall be held as a rule on a quarterly basis the chairman of the risk and capital committee shall give an overview of the activities of the risk and capital committee the reso lutions adopted and their implementation at least on an annual basis four ordinary meetings of the risk and capital committee were held in reviewing regular risk reports icaap work processes and capital plans key customers under monitoring changes in provisions current irb and the srep report the supervisory board was advised in issues related to risk management and capital management poli cies with the key principles of credit policy and loan portfolio management and the planned changes reviewed rain lõhmus chairman andres viisemann and tiina mõis serve as members of the risk and capital committee rain lõhmus is the founder of lhv and the chairman of the supervisory board of lhv rain lõhmus holds shares as lõhmus holdings shares and oü merona systems shares of as lhv group andres viisemann is the founder of lhv member of the supervisory board of lhv and manager of the pension funds of as lhv varahaldus andres viisemann does not hold any shares of as lhv group viisemann investments ag holds shares and viisemann holdings oü share of as lhv group tiina mõis is a member of the supervisory board of lhv and the owner and managing director of as genteel tiina mõis does not hold any shares of as lhv group as genteel holds shares of as lhv group no remuneration is paid to members of the risk and capital committee cooperation of the management and supervisory boards the management board and supervisory board work in close co operation to best protect the interests of lhv the co operation is above all based on an open exchange of views between as well as within the management board and the supervisory board the management board and the supervisory board joint their efforts to lay out the objectives and strategy of lhv in the management of lhv the management board is governed by the strategic instructions given by the super visory board the management board regularly discusses strategic management issues with the supervisory board within the framework of an open dialogue the management board regularly informs the supervi sory board of the key circumstances regarding the activity plans and business activities of lhv the risks involved and management of such risks no conflicts of interests occurred in the financial year disclosure of information lhv treats all shareholders equally and notifies all share holders of material developments accordingly information is disclosed in accordance with the rules set forth for publicly traded companies lhv has set up an investor relations website making avail able all documents and information in accordance with the corporate governance recommendations lhv publishes as lhv group tartu mnt tallinn info lhv ee lhv ee on its website a financial calendar containing the publication dates of the annual report and interim reports and the date of the annual general meeting of shareholders the published information is also made available in english and russian in lhv will start disclosing its annual plans and monthly results via the stock exchange announcements lhvs dividend policy is available on lhvs website stipu lating the dividend payment principles according to which the key precondition for payment of dividends includes external and internal regulatory capital standards which have to be met in a sustainable manner lhv may decide to waive dividend payment under the corresponding growth and or investment plans where the preconditions are met lhv shall pay a minimum of per cent of the shareholders profit before taxes in dividends income tax included in meetings and webinars with investors were held as needed and based on investor requests the relevant pres entations are available on lhvs website financial reporting and auditing lhv publishes its annual report on an annual basis the annual report is audited and approved by the supervisory board the number of auditors is determined and the auditors appointed by the general meeting of shareholders the general meeting also establishes the procedure for remu neration of auditors auditors are appointed for a single audit or for a specified term in the management board in cooperation with the audit committee organised a tender for the appointment of an auditor as part of the tender lhv met with and requested bids from four major internationally recognised companies as a result of the tender as pricewaterhouse coopers was chosen and appointed the companys auditor at the annual general meeting of shareholders held on april a three year contract was entered into with as pricewaterhousecoopers for the audit of the financial years the auditor has rendered contractual services to lhv consolidation group companies in including audits on the financial statements of group companies and reviews of the profits disclosed in quarterly financial statements consulting in connection with the performance of contractual obligations to the us tax authority audits of the as danske capital purchase analysis and other current matters transactions with related parties are disclosed in the notes to the financial statements statement of conformity lhv adheres to the corporate governance recommenda tions with the exception of the following instructions and recommendations for the reasons stated below issuers shall make participation in the general meeting possible by means of communication equip ment internet if the technical equipment is available and where doing so is not too cost prohibitive for the issuer observation of the general meeting and participation thereof has not been made possible by means of communication equipment there has been no need no corresponding requests and no suitable solution for such a facility none theless from onwards lhv stands ready to ensure participation of shareholders at the general meeting via secure electronic means without being physically present at the general meeting with the corresponding option added to the articles of association of lhv the management board shall have more than one member and a chairman elected by its members the management board or supervisory board shall establish an area of responsibility for each member of the management board defining as clearly as possible the duties and powers of each board member the principles for co operation between members of the board shall also be established the chairman of the supervisory board shall conclude a contract of service with each member of the board for discharge of their functions the management board of lhv has one member lhv is a holding company without any day to day operational busi ness activities thus a single member of the management board ensures sufficient and comprehensive management of lhv the major subsidiaries of lhv have management boards consisting of several members the member of the management board shall not be at the same time a member of more than two management boards of an issuer and shall not be the chairman of the supervisory board of another issuer a member of the management board can be the chairman of the supervisory board in company belonging to same group as the issuer erkki raasuke served as the chairman of the supervisory board of as eesti energia in as eesti energia has not issued any securities on the tallinn stock exchange but has issued bonds on the london stock exchange madis toomsalu is not a management board member or supervi sory board member in any other issuer basic wages performance pay sever ance packages other payable benefits and bonus schemes of a management board member as well as their essential features incl features based on comparison incentives and risk shall be published in clear and unambiguous form on website of the issuer and in the corporate governance recommendations report information published shall be deemed clear and unambiguous if it directly expresses the amount of expense to the issuer or the amount of foreseeable expense as of the day of disclosure as lhv group tartu mnt tallinn info lhv ee lhv ee the chairman of the supervisory board shall present the essential aspects of the management board remu neration and changes in it to the general meeting if the remuneration of some of the management board members has occurred on a different base then the general meeting shall be presented the differences together with the reasons therefore lhv discloses the total amount of remuneration paid to management board members in the notes to the financial statements lhv does not disclose the remuneration of individual members of the management board because this constitutes personal data the disclosure of which is not unavoidably necessary for evaluation of the operations and management quality of lhv at least half of the members of the supervisory board of the issuer shall be independent if the super visory board has an odd number of members then there may be one independent member less than the number dependent members independent members of the supervisory board of lhv are sten tamkivi and tauno tats other members of the super visory board of lhv are not independent in the meaning of the corporate governance recommendations as they have served in their current position for more than three consecutive years lhv has been in the stage of active development and growth where it has preferred people of long term management skills and banking experience as members of the supervisory board these people are also the largest shareholders of lhv lhv believes that as members of the supervisory board the largest shareholders are best motivated to contribute to the management of lhv and its development in the long term horizon the supervisory board shall describe in its evaluation report to judgment of the auditors work inter alia the services including advisory services that the auditor has provided to the issuer during the preceding year or shall provide during the next year also the remuneration the issuer has paid or shall pay to the auditor shall be published lhv has agreed with its foreign auditor not to disclose the contractual fees paid or payable to the auditor since this constitutes the parties confidential information the disclo sure of which is not unavoidably necessary for assessing the activities of lhv once a year lhvs audit committee shall review the services rendered by the foreign auditor the relevant volumes and fees pursuant to subsection of the accounting act a large undertaking whose securities granting voting rights have been admitted for trading on a regulated securities market of estonia or another contracting state shall describe in the corporate governance report the diversity policies carried out in the compa nys management board and senior management and the results of the implementation thereof during the accounting year if no diversity policies have been implemented during the accounting year the reasons for this should be explained in the corporate govern ance report lhv has not deemed it necessary to implement a diversity policy as lhv is governed in the recruitment of staff and management members by the best interests of lhv the education skills and previous experience of the person on a gender neutral on non discriminatory basis tiina mõis serves as a member of the supervisory board of lhv and a member of the risk and capital committee and gerli kilusk and marilin hein as members of the audit committee together with the management members lhvs subsidiaries engage a total of middle level managers of whom are women as lhv group tartu mnt tallinn info lhv ee lhv ee lhv strives to do business in a sustainable and socially responsible manner our greatest social contribution and responsibility is to provide a domestic alternative on the financial service market lhvs mission is to help raise estonian capital lhvs vision is focused on estonia where people and businesses dare to think big engage in enterprise and invest in the future lhvs values simple supportive effective lhv groups credit policy stipulates that in addition to potential obligations set forth by law corporate social responsibility also means voluntary consideration of the social factors in business processes social responsibility in lhv is a purposeful consistent voluntary contribution which extends beyond the obligations required by law and serves the interests of the society even though social responsibility constitutes a limitation in the pursuit of profit lhv has set its sights on enhancing social benefits without seeking profit the purpose of adherence to the principle of social respon sibility is to create long term value for lhvs counterparties social responsibility in lhvs operations is manifested in three ways as described below social responsibility in business operations responsible lending lhv cares about the business and living environment and makes an effort to minimise any negative environmental impact of its activities and processes the groups credit policy stipulates that lhv strives to operate in a socially responsible manner setting an example to both its staff members and the general public thus any credit decision for example must be governed by the prin ciple that lhv will not finance counterparties whose activ ities are liable to cause disproportionate damage to the environment or do not comply with the principles of social responsibility lhv takes special caution in financing tax transactions so as not to violate the spirit and meaning of the laws on tax private customers are financed based on the principle of responsible lending this is also evident in the groups high credit quality the positive register project has been launched on the initiative of lhv bank and lhv finance supporting estonian business environment the better the estonian economy is doing the higher the quality of life banking is the lifeblood of global economy as a local financial group we acknowledge our role in supporting and advancing the economy we engage capital from esto nian depositors directing it into estonian enterprises we contribute to the development of estonian capital markets we made an effort to set an example with the initial public offering of lhvs shares on the tallinn stock exchange our institutional brokerage helps baltic compa nies to engage funds from investors lhvs pension funds are among the largest local institutional investors pension funds investing in local economy lhvs pension funds have invested a considerable portion of the customers pension assets in estonia and the baltic states we have invested in estonian real estate enterprises forest in every seventh euro in pension fund assets contributed to the growth of estonian economy lhvs funds have invested an average of of their assets in esto nian securities for example pension fund l lhvs greatest pension fund in terms of customer numbers has made a quarter of its investments exposed to the stock market risk in estonia of investments exposed to the real estate and forest land risk has been made in funds investing predominantly in estonia we are always searching for and analysing local invest options we also stand ready to create such options ourselves lhv is the best market participant on baltic stock exchanges our everyday brokerage activities as well as our strive to set an example contribute to the functioning of the estonian capital markets in lhv bank received the member of the year award of nasdaq baltic stock exchanges lhv has been honoured with the same title on four previous occasions we make an effort to be a kind and understanding employer we would like lhv to serve as the best possible employer for our staff members lhv respects the occupational safety requirements established by law and considers the needs of the staff members facilitating study leaves paid health days etc the staff satisfaction survey conducted in revealed record high level of satisfaction among the groups employees open banking we create and develop our services based on the princi ples of open banking we aim at developing the estonian business environment and co operation by enhancing partnership innovation and transparency social responsibility in lhv as lhv group tartu mnt tallinn info lhv ee lhv ee sharing money wisdom and educating the investor base as a leader of the investor community lhv takes respon sibility for sharing money wisdom and advancing investor education we contribute via the following activities we organise regular free of charge investor school seminars for those interested in investing the investor school seminars had a total of participants in we have also started organising investor meets seminars allowing heads of publicly traded companies to meet investors we co operate with other estonian financial insti tutions and participate in the money wisdom month organised by the estonian banking association as well as other money wisdom projects initiated by estonian banks lhvs staff members gave lectures to school children on the money wisdom day we have set up a library of investment and economic literature our libraries in tallinn and tartu hold a total of more than different publications which are available for lending to our customers in addition to those interested in investing our library is also available to students of economy we continue to update the library we operate the lhv financial portal and the forum set up for discussing investing related subjects and sharing information lhv regularly contributes to creating content which could be of interest to esto nian investors ever since we have published the free of charge economic journal investeeri addressing various subjects related to entrepreneurship and investing four issues were published in the journal had a printrun of sponsorship within the limits of our available resources we support enterprises and initiatives that contribute to the develop ment of the estonian society when it comes to sponsorship projects we prefer a long term substantial partnership we stand ready to contribute to the community and the pursuit of innovative ideas which help to advance estonian life estonian culture estonian music days in we started supporting the music festival with the longest history in estonia by giving out the contemporary music award lhvs new music award went to liisa hirsch enn soosaar foundation with the support of lhv the annual enn soosaar award the ethical essayistics award tradition was started in in the award went to margus laidre books on estonian art we support publication of the books of estonian artists the latest project supported by lhv was painter kristi kongis authors book ma ei ole oma pead padjal liigutanud music events in lhv supported the intsikurmu festival in põlva and the acoussion live festival in central estonia education information technology foundation for education lhv partnered with the foundation to provide an lhv scholar ship for the best diploma papers in the field of information and communication technology in the scholarship was awarded to graduates of the tallinn university of tech nology liisi soots and viktor reinok sports healthy way of life estonian football association lhv bank has been the main sponsor of the estonian football association and the estonian national team since we have also estab lished the lhv football card support system which allowed to support estonian football clubs with a record amount in estonian cyclists union we have been the unions sponsor since and also supported the cyclists in estonian optimist class association we have helped to ensure sustainability of estonian sailing for the last years after the lhv supported revival of the association in the association has been actively engaged in promoting sailing among children and youngsters running competitions in lhv supported the organi sation of the peetri and rapla running competitions estonian economy and entrepreneurship estonian entrepreneur of the year we have been among the main sponsors of the estonian entrepreneur of the year competition organised by ey since gazelle movement lhv is a sponsor of the gazelle move ment the movement promotes and recognises rapidly growing estonian companies tallinn restaurant week we have supported the initiative of the estonian restaurant association since the initia tive aims at promoting the restaurant culture and introducing companies involved in the catering industry as lhv group tartu mnt tallinn info lhv ee lhv ee consolidated financial statements continuing operations interest income interest expense net interest income fee and commission income fee and commission expense net fee and commission income net gains from financial assets measured at fair value foreign exchange rate gains losses other financial income net gains from financial assets other income other expense staff costs administrative and other operating expenses profit before impairment losses on loans and advances share of result of associates impairment losses on loans and advances profit before taxes income tax expense net profit for the year from continuing operations profit from discontinued operations net profit for the year other comprehensive income loss items that may be reclassified subsequently to profit or loss available for sale investments revaluation of available for sale financial assets total profit and other comprehensive income for the year total profit attributable to owners of the parent non controlling interest total profit for the year total profit and other comprehensive income attributable to owners of the parent incl continuing operations incl discontinued operations non controlling interest total profit and other comprehensive income for the year basic earnings per share in euros diluted earnings per share in euros basic earnings per share in euros from continuing operations diluted earnings per share in euros from continuing operations consolidated statement of profit or loss and other comprehensive income in thousands of euros notes on pages to are an integral part of the consolidated financial statements new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee assets due from central bank due from credit institutions due from investment companies available for sale financial assets financial assets at fair value through profit or loss loans and advances to customers receivables from customers other financial assets other assets tangible assets intangible assets goodwill total assets liabilities deposits from customers and loans received financial liabilities at fair value through profit or loss accounts payable and other liabilities subordinated debt total liabilities equity share capital share premium statutory reserve capital other reserves retained earnings total equity attributable to owners of the parent non controlling interest total equity total liabilities and equity consolidated statement of financial position in thousands of euros notes on pages to are an integral part of the consolidated financial statements new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee cash flows from operating activities interest received interest paid fees and commissions received fees and commissions paid other income received staff costs paid administrative and other operating expenses paid cash flows from operating activities before change in operating assets and liabilities net increase decrease in operating assets net increase decrease in financial assets held for trading at fair value through profit or loss loans and advances to customers mandatory reserve at central bank security deposits other assets net increase decrease in operating liabilities demand deposits of customers term deposits of customers loans received repayments of loans received financial liabilities held for trading at fair value through profit or loss other liabilities net cash from used in operating activities from continuing operations cash generated from used in operating activities from discontinued operations net cash from used in operating activities cash flows from investing activities purchase of tangible and intangible assets disposal of tangible and intangible assets acquisition of subsidiaries net of cash acquired proceeds from disposal and redemption of investment securities available for sale net changes of investment securities at fair value through profit or loss cash from investment activities of discontinued operations net cash from used in investing activities cash flows from financing activities paid in share capital incl share premium proceeds from subordinated debt repayment of subordinated debt net cash from financing activities effect of exchange rate changes on cash and cash equivalents net increase decrease in cash and cash equivalents cash and cash equivalents at the beginning of the year cash and cash equivalents at the end of the year consolidated statement of cash flows in thousands of euros notes on pages to are an integral part of the consolidated financial statements new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee consolidated statement of changes in equity in thousands of euros share capital share premium accumulated deficit retained earnings total non cont rolling interest total equity statutory reserve capital other reserves balance as at transfer to statutory reserve capital note share options note profit for the year other comprehensive loss total profit and other comprehensive income for balance as at balance as at paid in share capital note transfer to statutory reserve capital note share options note profit for the year other comprehensive loss total profit and other comprehensive income for balance as at attributable to owners of lhv group notes on pages to are an integral part of the consolidated financial statements new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee these consolidated financial statements have been prepared in accordance with international financial reporting standards as adopted by european union for the year ended december for as lhv group and its subsidiaries as lhv pank as lhv finance subsidiary of as lhv pank as lhv varahaldus oü cuber technology and uab mokilizingas hereinafter referred together as the group as lhv group holds share interest in uab mokilizingas and interest in lhv finance through as lhv pank as lhv finance was incorporated january as lhv group is a limited liability company incorporated in and domiciled in estonia the address of its registered office is tartu mnt tallinn as lhv group is a holding company whose subsidiary as lhv pank hereinafter the bank provides banking financial advisory and securities brokerage services to customers in estonia and in lithu ania there are offices for client servicing in tallinn tartu and vilnius will be closed at march as lhv finance the subsidiary of as lhv pank offers hire purchase services to merchants and consumer finance as lhv vara haldus offers fund management services for pension funds and other funds to clients in estonia uab mokilizingas offers consumer finance and hire purchase services in lith uania as at the group employed full time employees incl non active employees incl non active the consolidated annual report incl financial statements was approved by the management board on of february rain lõhmus who owns of the voting rights and andres viisemann who owns of the voting rights see also note have significant influence over as lhv group the consolidated annual report approved by the manage ment board shall be authorised for approval by the super visory board and shareholders the shareholders have the right not to approve the consolidated annual report while the supervisory board does not have that right notes to the consolidated financial statements note general information new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the principal accounting policies applied in the prepara tion of these consolidated financial statements are set out below these policies have been consistently applied to all the years presented and to all the consolidated group enti ties unless otherwise stated basis of preparation the consolidated financial statements of the group for the financial year have been prepared in accordance with international financial reporting standards ifrs as adopted by the european union the financial statements have been prepared under the historical cost convention except as disclosed in some of the accounting policies below such as financial assets and liabilities at fair value through profit or loss including derivatives and assets avail able for sale the preparation of consolidated financial statements in accordance with ifrs requires the use of certain critical accounting estimates it also requires management to exer cise its judgement in the process of applying accounting policies the areas involving a higher degree of judgement or complexity or areas where assumptions and estimates are significant to the financial statements are disclosed in note the financial year started at january and ended at december the financial figures have been presented in thousands of euros unless referred differently in specific disclosures certain new ifrs standards amendments to existing stand ards and the interpretations of the standards have been published by the time of compiling these financial state ments which became mandatory for the groups reporting periods beginning on or after january the overview of these standards and the potential impact of applying the new standards and interpretations are stated below a adoption of new or revised standards and interpretations the following new or revised standards and interpretations became effective for the group from january annual improvements to ifrss effective in the eu for annual periods beginning on or after february the improvements consist of changes to seven standards ifrs was amended to clarify the definition of a vesting condition and to define separately performance condition and service condition the amendment is effective for share based payment transactions for which the grant date is on or after july ifrs was amended to require disclosure of the judge ments made by management in aggregating operating segments including a description of the segments which have been aggregated and the economic indicators which have been assessed in determining that the aggregated segments share similar economic characteristics and a reconciliation of segment assets to the entitys assets when segment assets are reported group uses segmentation in everyday management from year all the internal measurement including forecasting profitability measure ment and following key performance indicators is done based on segments the segmentation is done in principle that there is no double accounting of incomes costs sum of segments has to be equal to reporting entity results disclosure initiative amendments to ias effective in the eu for annual periods beginning on or after january the standard was amended to clarify the concept of materiality and explains that an entity need not provide a specific disclosure required by an ifrs if the information resulting from that disclosure is not material even if the ifrs contains a list of specific requirements or describes them as minimum requirements the standard also provides new guidance on subtotals in financial statements in particular such subtotals a should be comprised of line items made up of amounts recognised and measured in accordance with ifrs b be presented and labelled in a manner that makes the line items that constitute the subtotal clear and understandable c be consistent from period to period and d not be displayed with more prominence than the subto tals and totals required by ifrs standards there are no other new or revised standards or interpreta tions that are effective for the first time for the financial year beginning on or after january that have a material impact on the group b new accounting pronouncements certain new or revised standards and interpretations have been issued that are mandatory for the groups annual periods beginning on or after january and which the group has not early adopted ifrs financial instruments effective for annual periods beginning on or after january the standard is endorsed by the eu commission earlier application is permitted but the group does not intend to early adopt the note summary of significant accounting policies new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee standard the group intends to use relief of not restating comparative figures for in the annual report when first adopting ifrs key features of the new standard are as follows financial assets are required to be classified into three measurement categories assets to be measured subsequently at amortised cost assets to be measured subsequently at fair value through other comprehensive income fvoci assets to be measured subsequently at fair value through profit or loss fvpl the classification and measurement requirements in ifrs state that financial assets should be classified as and meas ured at amortized cost fair value through profit and loss or fair value through other comprehensive income the classifi cation of a financial instrument is dependent on the business model for the portfolio where the instrument is included and on whether the cash flows are solely payments of principal and interest in order to assess the business model the group has divided its financial assets into portfolios and or sub portfolios based on how groups of financial assets are managed together to achieve a particular business objective classification for debt instruments is driven by the entitys business model for managing the financial assets and whether the contractual cash flows represent solely payments of principal and interest sppi if a debt instru ment is held to collect it may be carried at amortised cost if it also meets the sppi requirement debt instruments that meet the sppi requirement that are held in a portfolio where an entity both holds to collect assets cash flows and sells assets may be classified as fvoci financial assets that do not contain cash flows that are sppi must be measured at fvpl for example derivatives embedded derivatives are no longer separated from financial assets but will be included in assessing the sppi condition investments in equity instruments are always measured at fair value however management can make an irrevocable election to present changes in fair value in other comprehen sive income provided the instrument is not held for trading if the equity instrument is held for trading changes in fair value are presented in profit or loss most of the requirements in ias for classification and measurement of financial liabilities were carried forward unchanged to ifrs the key change is that an entity will be required to present the effects of changes in own credit risk of financial liabilities designated at fair value through profit or loss in other comprehensive income the analysis of the business model described above is still ongoing current results have not indicated signifi cant changes compared to how the financial instruments are measured under ias no significant impact is thus expected on the groups financial position financial perfor mance or equity in the period of initial application these conclusions are tentative and may be affected by the actual financial positions at the transition the impairment requirements in ifrs are based on an expected credit loss ecl model as opposed to the current incurred loss model in ias the scope of ifrs impair ment requirements is also broader than ias ifrs requires all assets measured at amortized cost and fair value through other comprehensive income as well as off balance commitments including guarantees and loan commitments to be included in the impairment test the assets to test for impairment will be divided into three groups depending on the stage of credit deterioration stage includes assets where there has been no significant increase in credit risk stage includes assets where there has been a significant increase in credit risk and stage includes defaulted assets significant assets in stage are tested for impairment on an individual basis while for insignificant assets a collective assessment is performed in stage the provisions should equal the month ecl in stage and the provisions should equal the lifetime ecl one important driver for size of provisions under ifrs is the trigger for transforming an asset from stage to stage the group is in process of deciding what parameters to use for identifying the increase in credit risk and how much these parameters need to change in order to constitute a significant increase the analyses and decisions should be finalized latest in june for assets to be recognized going forward changes to the lifetime probability of default pd will be used as the trigger in addition customers with forbearance measures and customers with payments more than thirty days past due will also be transferred to stage the group s current model for calculating collective provi sions defines a loss event as a deterioration in rating scoring but it is not expected that the loss event in the current model will equal the triggering event for moving items from stage to stage under ifrs the provisions under ifrs will be calculated as the exposure at default times the probability of default times the loss given default for assets in stage this calculation will only be based on the coming months while it for assets in stage will be based on the expected lifetime of the asset for assets where there has been a significant increase in credit risk the group currently holds provisions based on the losses estimated to occur during the period between the date when the loss event occurred and the date when new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the loss event is identified on an individual basis while ifrs will require provisions equal to the lifetime expected loss when calculating lifetime losses under ifrs including the staging assessment the calculation should be based on probability weighted forward looking information it is expected the newer requirements will increase the volatility in loan loss provisions and have negative effect in equity in the period of initial application it is not expected to have any material impact on large exposures the impact on capital adequacy is not possible to determine as it is expected the basel committee will issue new rules for the transiton to ifrs but these are not yet final it is furthermore expected that the long term effects once the transitional rules become obsolete will be negative on capital adequacy as the reduc tion in equity is expected to reduce cet capital impairment calculations under ifrs will require more experienced credit judgement by the reporting entities than required by ias today and a higher subjectivity is thus introduced the inclusion of forward looking information adds complexity and makes provisions more dependent on management s view of the future economic outlook it is expected that the impairment calculations under ifrs will be more volatile and pro cyclical than under ias mainly due to the significant subjectivity applied in the forward looking scenarios the main change to the general hedge accounting require ments is that the standard aligns hedge accounting more closely with the risk management activities these changes will not have direct effect on the group as the group is currently not using hedge accounting same time we see that the regulation changes will generate some new possi bilities to the group to start using hedge accounting for reducing volatility in financial statements for positions which are hedged with ones treated differently in accounting group has set up a project developing the needed changes both in risk management and in it systems the final outcome of the impact of the new standard on its finan cial statements is not clear yet and will be clear in second quarter still group is clear that there will be added new volatility to provisioning the provisions models will be more complex and visibility will somewhat lower we have calculated the impact to the cost of development and its around thousand ifrs leases effective for annual periods beginning on or after january not yet adopted by the eu the new standard sets out the principles for the recognition meas urement presentation and disclosure of leases all leases result in the lessee obtaining the right to use an asset at the start of the lease and if lease payments are made over time also obtaining financing accordingly ifrs eliminates the classification of leases as either operating leases or finance leases as is required by ias and instead introduces a single lessee accounting model lessees will be required to recognise a assets and liabilities for all leases with a term of more than months unless the underlying asset is of low value and b depreciation of lease assets separately from interest on lease liabilities in the income statement ifrs substantially carries forward the lessor accounting requirements in ias accordingly a lessor continues to classify its leases as operating leases or finance leases and to account for those two types of leases differently the group is currently assessing the impact of the new standard on its financial statements disclosure initiative amendments to ias effective for annual periods beginning on or after january not yet adopted by the eu the amended ias will require disclo sure of a reconciliation of movements in liabilities arising from financing activities the group is currently assessing the impact of the amendment on its financial statements ifrs revenue from contracts with customers effec tive for annual periods beginning on or after january the new standard introduces the core principle that revenue must be recognised when the goods or services are transferred to the customer at the transaction price any bundled goods or services that are distinct must be separately recognised and any discounts or rebates on the contract price must generally be allocated to the separate elements when the consideration varies for any reason minimum amounts must be recognised if they are not at significant risk of reversal costs incurred to secure contracts with customers have to be capitalised and amor tised over the period when the benefits of the contract are consumed the largest impact will be positive to the as lhv varahaldus where we need to capitalize sales costs from previous years and amortize over the lifetime of the customer which in pension fund case is above years this effect will significantly increase lhv varahaldus own funds at the moment of adoption of this ifrs standard the group is currently assessing the exact impact of the new standard on its financial statements revenue from contracts with customers amendments to ifrs effective for annual periods beginning on or after january not yet adopted by the eu the amendments do not change the underlying principles of the standard but clarify how those principles should be applied the amendments clarify how to identify a performance obligation the promise to transfer a good or new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee a service to a customer in a contract how to determine whether a company is a principal the provider of a good or service or an agent responsible for arranging for the good or service to be provided and how to determine whether the revenue from granting a licence should be recognised at a point in time or over time in addition to the clarifications the amendments include two additional reliefs to reduce cost and complexity for a company when it first applies the new standard the group is currently assessing the impact of the new standard on its financial statements there are no other new or revised standards or interpreta tions that are not yet effective that would be expected to have a material impact on the group consolidation the consolidated financial statements include the financial statements of as lhv group parent company and its subsidiaries as lhv pank as lhv finance subsid iary of as lhv pank as lhv varahaldus uab mokilizingas and oü cuber technology as lhv group holds share interest in uab mokilizingas and interest in lhv finance through as lhv pank subsidiaries are all entities over which the group has control the group controls an entity when the group is exposed to or has rights to variable returns from its involvement with the entity and has the ability to affect those returns through its power over the entity the acquisition method of accounting is used to account for the acquisition of subsid iaries except for acquisitions among enterprises under common control and business operations the cost of an acquisition is measured as the fair value of the assets given equity instruments issued and liabilities incurred or assumed at the date of exchange transaction costs incurred for issuing equity instruments are deducted from equity trans action costs incurred for issuing debt are deducted from its carrying amount and all other transaction costs associated with the acquisition are expensed according to the acquisi tion method the identifiable assets liabilities and contingent liabilities of the acquired subsidiary are recognized at their fair values at the acquisition date irrespective of the extent of any non controlling interest the group measures non controlling interest that repre sents present ownership interest and entitles the holder to a proportionate share of net assets in the event of liquidation on a transaction by transaction basis either at a fair value or b the non controlling interest s proportionate share of net assets of the acquiree non controlling interests that are not present ownership interests are measured at fair value goodwill is measured by deducting the net assets of the acquiree from the aggregate of the consideration transferred for the acquiree the amount of non controlling interest in the acquiree and fair value of an interest in the acquiree held immediately before the acquisition date any nega tive amount negative goodwill is recognised in profit or loss after management reassesses whether it identified all the assets acquired and all liabilities and contingent liabilities assumed and reviews appropriateness of their measurement intercompany transactions balances and unrealised gains on transactions between group companies are eliminated unrealised losses are also eliminated unless transaction provides evidence of an impairment of the asset transferred revenues and expenses of the subsidiaries acquired within the financial year are consolidated into the groups state ment of profit or loss and other comprehensive income starting from the date of acquisition to the end of the finan cial year result of operations of subsidiaries disposed of during the year are consolidated into groups statement of profit or loss and other comprehensive income from the beginning of the financial year until the date of disposal pursuant to the accounting act of the republic of estonia information of the unconsolidated financial statements primary statements of the consolidating entity parent company shall be disclosed in the notes to the consolidated financial statements in preparing the primary financial state ments of the parent company the same accounting policies have been used as in preparing the consolidated financial statements the accounting policy for accounting subsid iaries has been amended in the separate primary financial statements disclosed as supplementary information in the annual report in conjunction with las separate finan cial statements in the parent separate primary financial statements disclosed in these consolidated financial statements see note the investments into the shares of subsidiaries are accounted for at cost less any impairment recognized non controlling interest is that part of the net results and of the net assets of a subsidiary which is attributable to interests which are not owned directly or indirectly by the group non controlling interest forms a separate compo nent of the groups equity non controlling interest in the consolidated statement of financial position is disclosed separately from the equity attributable to the shareholders of the parent company in consolidated statement of profit or loss and other comprehensive income non controlling interest share of profit is disclosed separately from owner s of the parent new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee associates an associate is an entity over which the group has signifi cant influence but which it does not control generally signif icant influence is assumed to exist when the group holds between and of the voting rights the group does not have any associate as at at group had one associate svipe oü in which thw group hold of voting rights in the consolidated financial statements investments in associates are accounted for using the equity method under this method the investment is initially recognized at cost which is thereafter adjusted for post acquisition changes in the investors share of the investees equity changes both in the profit loss of the associate as well as other equity items and with elimination or depreciation amortisation of the differences between fair values and carrying amounts of the investees assets liabilities and contingent liabilities as determined in the purchase analysis unrealized gains on transactions between the group and its associates are elim inated to the extent of the groups interest in the associates unrealized losses are also eliminated unless the transaction provides evidence of impairment of the assets when the groups share of losses in the associate accounted under the equity method exceeds the carrying amount of the associate the carrying amount of the investment is reduced to zero and such long term loans granted to the associate that in substance form a part of the investment are written down further losses are carried off balance sheet when the group has guaranteed or incurred obligations on behalf of the associate the respective liability as well as the loss under the equity method are recorded in the statement of financial position other receivables from the associate are recorded at amortised cost less a provision for impairment an investment in the assets and liabilities of the acquired associate and goodwill that arose on acquisition is presented as a net amount in the statement of financial position line investments in associates at each balance sheet date it is assessed whether there is any indication that the recoverable amount of the invest ment has fallen below its carrying amount if any such indi cations exist an impairment test is performed to determine the recoverable amount of the investment the principles described in section are used foreign currency translation a functional and presentation currency the functional and presentation currency of entities in the group is euro b foreign currency transactions and balances foreign currency transactions are recorded based on the foreign currency exchange rates of the european central bank prevailing at the dates of the transactions monetary assets and liabilities denominated in foreign currencies are translated into euros based on the foreign currency exchange rates of the european central bank prevailing at the balance sheet date non monetary financial assets and liabilities measured at fair value denominated in foreign currencies are translated into euros based on the foreign currency exchange rates of the european central bank prevailing at the fair value measurement date gains and losses on translation from monetary assets and liabilities are recorded in the statement of profit or loss under the line foreign exchange rate gains losses translation differ ences on non monetary items such as equities held at fair value through profit or loss are reported as part of the fair value gain or loss in the statement of profit or loss under the line foreign exchange rate gains losses cash and cash equivalents for the purposes of the cash flow statement cash and cash equivalents comprise cash on hand demand deposits with central bank and other banks and investment companies term deposits with original maturities of three months or less that are available for use without any restrictions and which are subject to an insignificant risk of changes in value financial assets the group classifies its financial assets into the following categories financial assets at fair value through profit or loss held to maturity financial investments available for sale financial assets loans and receivables the classification depends on the purpose for which the financial assets were acquired management of the group determines the classification of its financial assets at initial recognition new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee financial assets in thousands of euros ias category class applied by the group loans and receivables loans and advances to banks and investment companies loans and advances to customers loans to legal entities loans to individuals other receivables other financial assets financial assets at fair value securities held for trading equity securities through profit or loss listed debt securities derivatives designated as at fair value through profit or loss upon equity securities initial recognition and fund units available for sale financial assets investment securities listed debt securities a financial assets at fair value through profit or loss financial assets at fair value through profit or loss include financial assets held for trading incl derivatives financial assets designated at fair value through profit or loss at inception a financial asset is classified as held for trading if acquired or incurred principally for the purpose of selling or repur chasing it in the near term derivatives are classified as held for trading unless they are designated as hedges regular way purchases and sales of financial investments are recognised at the trade date in the statement of financial position during the year the above mentioned trans actions were recognised at the settlement date in the state ment of financial postion this change in the accounting policies did not have any effect on the statement of financial position as there were no transactions as at which trade date was before balance sheet date and settle ment date after the balance sheet date financial instru ments included in this category are initially recognised at fair value transaction costs are taken directly to the profit or loss after initial recognition financial assets in this cate gory are measured at fair value interest income is calcu lated based on financial assets amortised cost using the effective interest rate method and is recorded within interest income in the statement of profit or loss changes in fair values of these financial assets are recognised consistently either as a profit or loss in the statement of profit or loss of the reporting period under net gains losses from financial assets measured at fair value in accounting for financial assets at fair value through profit or loss the change in the value of assets between the trade date and the settlement date are taken to profit or loss similarly to the recognition of similar assets in the ownership of the group dividend income on financial assets measured at fair value through profit or loss are recognised in the line dividend income of the statement of profit or loss when the right to receive dividends by the group is established in case of listed securities i e the securities which have an active market the current bid price is considered as the fair value of investments to estimate the fair value of invest ments not actively traded in the market alternative methods such as the price of recent transactions under market conditions the discounted cash flow method or option valuation models are used derivative financial instruments futures forward swap and option contracts are initially recognised in the statement of financial position at fair value at the trade date and are subsequently valued at fair value through profit or loss if derivatives are quoted on an active market market value is used as the fair value otherwise the valuation techniques are used to estimate the fair value gains and losses from derivatives are recognised as income or expense of the period in the statement of profit or loss under net gains losses from financial assets measured at fair value deriva tives are carried in statement of financial position as assets if their fair value is positive and as liabilities if their fair value is negative the fair values of derivative assets and liabilities recorded in the statement of financial position are not netted the group does not use hedge accounting to account for its derivative financial instruments financial assets designated at fair value through profit or loss at inception securities are classified into this group if the group has upon initial recognition designated the secu rities to be recorded as at fair value through profit or loss and as a result the changes in the fair value of these secu rities are consistently recognised in the profit or loss of the reporting period in the statement of profit or loss new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee financial assets and financial liabilities are designated at fair value through profit or loss when certain investments such as debt or equity investments are managed and evaluated on a fair value basis in accordance with a documented risk management or investment strategy and reported to key management personnel on that basis refer to note for fair value of financial assets b held to maturity financial investments held to maturity financial investments are financial instru ments quoted in an active market with a fixed due date and which the management board of the group has an inten tion and opportunity to hold until their maturity they do not include investments designated as fair value through profit or loss upon their initial recognition investments classified as available for sale assets investments which meet the criteria of loans and receivables these investments are initially recognised at fair value plus all directly attributable transaction costs incremental to their acquisitions and they are subsequently measured at amor tised cost using the effective interest rate method interest income on held to maturity investments is included within interest income in the statement of profit or loss impairment losses are deducted from the carrying amount of the invest ments and the impairment charge is recorded in the state ment of profit or loss c available for sale financial assets available for sale financial assets are financial instruments which the group intends to hold for an indefinite period of time and which may be sold in response to needs for liquidity or changes in interest rates exchange rates or equity prices investment securities available for sale are carried at fair value interest income on available for sale debt securities is calculated using the effective interest method and recognised in profit or loss other changes in fair values of these financial assets are recognised in other comprehensive income d loans and receivables loans and receivables are non derivative financial assets with fixed or determinable payments that are not quoted in an active market loans and receivables to customers are recorded when the group advances money to purchase or originate an unquoted non derivative receivable from a customer due on fixed or determinable dates and has no intention of trading the receivable loans and receivables are initially recognised at fair value plus transaction costs and are derecognised only when they are repaid or writ ten off after initial recognition the group recognises loans and receivables at amortised cost less principal repayments and any potential impairment losses if necessary and calculates interest income on the receivable in subsequent periods using the effective interest method in case of credit cards the borrowers actual use of limit is recorded in the statement of financial position finance lease transactions are lease transactions under which all significant risks and rewards from using the assets are immediately transferred from the group to the lessee legal ownership of assets is transferred to the customer at the end of the lease term finance leases are capitalised at the inception of the lease at the lower of the fair value of the leased property and the present value of minimum lease payments plus any unguaranteed residual value of the leased asset each lease payment received from the lessee is allocated between the receivable and finance income lease income is recognised over the term of the lease using the net investment method which reflects a constant peri odic rate of return when there are service fees and initial direct costs attributable to the conclusion of lease agree ments they are included in the calculation of the effective interest rate of the lease and calculation of a finance lease receivable the basis for assessing receivables is the timely fulfilment of contractual obligations the estimated market price of the leased asset and excess collateral the financial position and reliability of the customer impairment of receiv ables reduces the respective asset category in the state ment of financial position loans restructured because of significant changes in contract terms are recognised as new loans when a new loan has been issued incl refinancing an existing loan or increasing an existing loan for payment of the loan principal interest overdue penalties or other fees related to it loans which include a change in other contractual terms as a result of deterioration of the customers credit quality do not qualify for derecognition trade receivables arise from provision of services to the customers and are initially recognised at fair value plus transaction costs and subsequently measured at amortised cost using the effective interest method less repayments and a provision for incurred impairment losses the impairment of loans and advances is assessed in accordance with the principles described in clause impairment of financial assets carried at amortised cost impairment losses are recognised in profit or loss for the year when incurred as a result of one or more events loss events that occurred after the initial recognition of the finan new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee cial asset and which have an impact on the amount or timing of the estimated future cash flows of the financial asset or group of financial assets that can be reliably estimated if the group determines that no objective evidence exists that impairment was incurred for an individually assessed finan cial asset whether significant or not it includes the asset in a group of financial assets with similar credit risk characteris tics and collectively assesses them for impairment the group assesses the risks taking into consideration all known information about the solvency of the debtor and whether there are any objective circumstances indicating impairment the buyers financial difficulties bankruptcy or inability to meet its obligations to the group corporate loans overdrafts and leases are assessed indi vidually primarily based on the overdue status of loan and realisable of related collateral if any other criteria used to determine whether there is objective evidence that an impairment loss has occurred are financial position of the company industry situation reliability of the borrower the competence of its management and other factors margin and repo loans secured by debt or equity securities both to legal as well as private individuals are assessed individ ually using primarily the market value of collateral as the basis consumer loans mortgage loans credit card loans and hire purchase loans are assessed as a group private individuals are assessed in terms of their timely fulfilment of obligations solvency value of the collateral age educa tion length of employment savings habits and other factors impacting credit risk for the purposes of a collective evaluation of impairment financial assets are grouped on the basis of similar credit risk characteristics those characteristics are relevant to the estimation of future cash flows for groups of such assets by being indicative of the debtors ability to pay all amounts due according to the contractual terms of the assets being evaluated future cash flows in a group of financial assets that are collectively evaluated for impairment are estimated on the basis of the contractual cash flows of the assets and the experience of management in respect of the extent to which amounts will become overdue as a result of past loss events and the success of recovery of overdue amounts past experience is adjusted on the basis of current observ able data to reflect the effects of current conditions that did not affect past periods and to remove the effects of past conditions that do not exist currently the group regularly reviews future cash flow assessment methods and assump tions in order to reduce potential differences between loss estimates and actual losses if the terms of an impaired financial asset held at amor tised cost are renegotiated or otherwise modified because of financial difficulties of the borrower or issuer impair ment is measured using the original effective interest rate before the modification of terms the renegotiated asset is then derecognized and a new asset is recognized at its fair value only if the risks and rewards of the asset substantially changed this is normally evidenced by a substantial differ ence between the present values of the original cash flows and the new expected cash flows impairment losses are always recognised through an allow ance account to write down the assets carrying amount to the present value of expected cash flows which exclude future credit losses that have not been incurred discounted at the original effective interest rate of the asset the calcu lation of the present value of the estimated future cash flows of a collateralised financial asset reflects the cash flows that may result from foreclosure less costs for obtaining and selling the collateral whether or not foreclosure is probable any impairment losses are charged to statement of profit or loss line impairment losses on loans and advances if in a subsequent period the amount of the impairment loss decreases and the decrease can be related objectively to an event occurring after the impairment was recognised such as an improvement in the debtors credit rating the previ ously recognised impairment loss is reversed by adjusting the allowance account through profit or loss for the year the reversal of the impairment loss is recognised as income in the line impairment losses on loans and advances in the statement of profit or loss uncollectible assets are written off against the related impairment loss provision after all the necessary procedures to recover the asset have been completed and the amount of the loss has been determined subsequent recoveries of amounts previously written off are credited to impairment loss account in profit or loss for the year interest income on loans is presented on the statement of profit or loss under interest income discontinued operations a discontinued operation is a component of the group that is classified as held for sale and represents a separate major line of business or geographical area of operations and is part of a single co ordinated plan to dispose of a separate major line of business or geographical area of operations non current asset or disposal group is classified as held for sale when their carrying amount is to be covered princi pally through a sale transaction rather than through contin uing use and the sale is considered highly probable within months the disposal group is recognized at carrying amount or fair value less costs to sell depending on which is lower earnings and cash flows of discontinued operations if any are disclosed separately from continuing operations with comparatives being re presented intragroup trans actions between discontinued and continuing operations are eliminated depending on whether the transactions will continue after the sale transaction new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee tangible fixed assets tangible fixed assets are non current assets used in the operating activities of the group with a useful life of over one year an item of tangible fixed assets is initially recognised at its cost which consists of the purchase price incl customs duties and other non refundable taxes and other expendi tures directly related to the acquisition that are necessary for bringing the asset to its operating condition and loca tion subsequent costs are included in the assets carrying amount or are recognised as a separate asset as appro priate only when it is probable that future economic benefits associated with the item will flow to the group and the cost of the item can be measured reliably all other repairs and maintenance are charged to operating expenses during the reporting period in which they are incurred an item of tangible fixed assets is carried in the statement of financial position at cost less any accumulated deprecia tion and any accumulated impairment losses depreciation is calculated on a straight line basis the annual depreci ation rate for computers furniture and fixtures is for improvements of rental space either p a or amorti zation throughout the rental period depending on which is shorter depreciation is calculated starting from the month of acquisition until the carrying amount reaches the residual value of the asset in case the residual value is greater than the carrying amount of the asset no further depreciation expense is calculated the appropriateness of the assets residual values depre ciation methods used and useful lives are reviewed and adjusted if appropriate at each balance sheet date the group performs an impairment test when there are circum stances indicating impairment where an assets carrying amount is greater than its estimated recoverable amount higher of an assets fair value less costs to sell or its value in use it is written down immediately to its recoverable amount recognising an impairment loss in the statement of profit or loss for the reporting period gains and losses on disposals are determined by comparing proceeds with carrying amount these are included in oper ating expenses or other income in the statement of profit or loss for the reporting period intangible assets intangible assets are identifiable non monetary assets without physical substance and currently comprise of acquired software licences an intangible asset is initially measured at cost comprising of its purchase price and any directly attributable expenditure on preparing the asset for its intended use after initial recognition an intangible asset is carried at its cost less any accumulated amortisation and any accumulated impairment losses amortisation is calcu lated on a straight line basis the annual amortisation rate for purchased licenses is the amortisation method for client agreements is dimishing balance method the annual amortisation rate for purchased client agreements is of the residual value of those assets at each balance sheet date the appropriateness of amortization rates methods and residual values is assessed the group reviews intan gible assets for impairment losses whenever events or changes in circumstances indicate that the carrying amount may not be recoverable an impairment loss is recognised for the amount by which the carrying amount of the asset exceeds its recoverable amount which is the higher of an assets fair value less costs to sell and value in use financial guarantees and performance guarantees the group issues financial guarantees and commitments to provide loans financial guarantees represent irrevocable assurances to make payments in the event that a customer cannot meet its obligations to third parties and carry the same credit risk as loans financial guarantees and commit ments to provide a loan are initially recognised at their fair value which is normally evidenced by the amount of fees received this amount is amortised on a straight line basis over the life of the commitment except for commitments to originate loans if it is probable that the group will enter into a specific lending arrangement and does not expect to sell the resulting loan shortly after origination such loan commit ment fees are deferred and included in the carrying value of the loan on initial recognition at the end of each reporting period the commitments are measured at the higher of i the remaining unamortised balance of the amount at initial recognition and ii the best estimate of expenditure required to settle the commitment at the end of each reporting period performance guarantees are contracts that provide compen sation if another party fails to perform a contractual obliga tion such contracts do not transfer credit risk performance guarantees are initially recognised at their fair value which is normally evidenced by the amount of fees received this amount is amortised on a straight line basis over the life of the contract at the end of each reporting period the perfor mance guarantee contracts are measured at the higher of i the unamortised balance of the amount at initial recognition and ii the best estimate of expenditure required to settle the contract at the end of each reporting period discounted to present value where the group has the contractual right to revert to its customer for recovering amounts paid to settle the performance guarantee contracts such amounts will be recognised as loans and receivables upon transfer of the loss compensation to the guarantees beneficiary financial liabilities the group classifies financial liabilities to categories a financial liabilities measured at fair value through profit or loss b financial liabilities measured at amortised cost new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee recognition of a financial liability derivative at fair value through profit or loss is disclosed under accounting policy a all other financial liabilities are subsequently meas ured at amortised cost using the effective interest method transaction costs are included in the calculation of the effective interest rate the difference between the proceeds less transaction costs and redemption value are recog nised in the statement of profit or loss during the term of the instrument using the effective interest rate interest costs are included in the statement of profit or loss line interest expenses deposits from customers are initially recorded on their settlement date at their fair value less transaction costs and are subsequently measured at amortised cost using effec tive interest method in the statement of financial position line deposits from customers and loans received accrued interest liabilities are included in the same line interest expense is recognised in the statement of profit or loss line interest expense on the accrual basis loans received debt securities issued and similar subordi nated debts are initially recognised at fair value less trans action costs cash received less transaction costs the subordinated debts are those liabilities which in case of a liquidation of a credit institution or declaration of bankruptcy are settled after the satisfaction of the justifiable claims of other creditors other financial liabilities trade payables accrued expenses and other borrowings are initially recog nised at fair value less transaction costs and are subse quently carried at amortised cost payables to employees payables to employees include unpaid salary accruals and a vacation pay accrual calculated in accordance with employ ment contracts and the laws in force in employee residential country as at the balance sheet date the liability related to the payment of a vacation pay accrual together with social security and unemployment insurance payments is included within current liabilities in the statement of financial position and as staff costs in the statement of profit or loss social security tax includes payments to the state pension fund group has neither existing legal nor constructive obli gations to make pension or similar payments supplementary to social security tax share based payments as lhv group operates a share based compensation plan under which the company receives services from group employees as consideration for equity instruments options of lhv group the fair value of the employee services received in exchange for the grant of the options is recog nised during the shared based compensation program as groups staff expense and as an increase in the equity other reserves the total amount to be expensed is determined by the fair value at the time of issuing the options the fair value of the options granted including any market performance conditions influencing the share price e g lhv group share price excluding the impact of any service and non market performance vesting conditions for example profitability sales growth targets and remaining an employee of the entity over a specified time period at the end of each reporting period the group revises its estimates of the number of options that are expected to vest based on the vesting conditions other than market condi tions it recognises the impact of the revision to original estimates if any in the statement of profit or loss with a corresponding adjustment to equity when the options are exercised lhv group issues new shares the grant by lhv group of options over its equity instru ments to the employees of subsidiary undertakings in the group is treated as a capital contribution the fair value of employee services received measured by reference to the grant date fair value is recognised over the vesting period as an increase to investment in subsidiary undertakings with a corresponding credit to equity in the parent entity accounts according to the contractual conditions of the share options there are no social security tax charges when exercising the options after the year period provisions and contingent liabilities liabilities arising from an obligating event before the balance sheet date that have either a legal basis or that have arisen from the groups current operating practice legal or constructive obligation that require probable outflow of resources the realisation of which is probable it is more likely than not that an outflow of resources will be required to settle the obligation and the amount of which can reliably be estimated but for which the timing or amount are not defi nitely known are recognised as provisions in the statement of financial position the provisions are recognised based on the managements estimates regarding the amount and timing of the expected outflows a provision is recog nised in the statement of financial position in the amount which according to the management is necessary as at the balance sheet date for the meeting of the obligation arising from the provision or transfer to the third party the provision expense and an expense from the change in the carrying amount of provisions are included within expenses in the reporting period provisions are not set up to cover future operating losses when it is probable that a provision will be realised later than months after the balance sheet date it is recorded at discounted value present value of expected outflows new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee unless the discounting effect is immaterial other contingent liabilities guarantees except financial guarantees and other commitments the realisation of which is less probable than non realisation or related costs cannot be determined reliably that in certain circumstances may become obligations are disclosed in the notes to the finan cial statements as contingent liabilities contingent liabilities may develop in a way not initially expected therefore they are assessed continually to determine whether an outflow of resources embodying economic benefits has become probable on a portfolio basis if it becomes probable that an outflow of future economic benefits will be required for an item or portfolio of items previously dealt with as a contingent liability a provision is recognised in the financial statements of the period in which the change in probability occurs except in the extremely rare circumstances where no reliable estimate can be made distinction between short and long term assets and liabilities assets from which resources are expected to flow to the group within months are recognised as current assets the portion of assets with expected inflows later than months after the balance sheet date is recognised as non current assets see note liabilities are classified as current when they are due within twelve months after the balance sheet date or if the group does not have an unconditional right to defer the payment for later than months after the balance sheet date loans received with due date within months after the balance sheet date which are refinanced as non current after the balance sheet date but before the financial statements are authorised for issue are recognised as current loans received that the lender has the right to recall at the balance sheet date due to breaches of contractual terms are also classified as current for all long term assets and liabilities the long term portion is separately disclosed in respective disclosure to these financial statements see note revenues and expenses revenues and expenses are recorded on an accrual basis revenue is recognised when it is probable that the economic benefits associated with the transaction will be received by the group the amount of the revenue can be measured reliably and services were rendered by the group revenue earned in the ordinary course of business is recognised at fair value of the fee received or receivable expenses are recognised when the group has obligation to pay for the expense and or the group has received goods or services and the latter occurs at an earlier date fee and commission income incl custody and portfolio management fees are recognised on an accrual basis when the service has been provided and the group has a right of claim to the receivablecommissions and fees arising from provision of advisory services to third parties purchase or sale of businesses acquisitions etc are generally recognised upon completion of the underlying transaction or when service is provided over a longer period of time based on the stage of completion method portfolio management other consultation fees and management fees related to investment funds are recognised according to the agreement usually over the period during which the service is provided the same principle is applied to wealth management financial planning and custody services that are continuously provided over an extended period of time performance linked fees or fee components are recognised when the performance criteria are fulfilled or based on the stage of completion other one off service revenues and other revenues are recognised on an accrual basis at the moment of executing the respective transaction fee and commission expenses are recognised after the service has been received and when the liability has incurred interest income and expense is recognised in the state ment of profit or loss for all financial instruments carried at amortised cost using the effective interest method or for debt securities measured at fair value through profit or loss transaction costs of debt securities measured at fair value through profit or loss is recognised as part of effec tive interest method loan commitment fees that are directly related with the undrawn portion of the loan commitment and which changes based on the portion of the unused commitment are recognised as interest income during the period when draw down is available the effective interest method is a method of calculating the amortised cost of a financial asset or a financial liability and allocating the interest income or interest expense over the relevant period the effective interest rate is the rate that exactly discounts estimated future cash payments or receipts through the expected life of the financial instru ment to the net carrying amount of the financial asset or financial liability when calculating the effective interest rate the group estimates cash flows considering all contractual terms of the financial instrument but does not consider future impairment losses the calculation includes all signif icant fees paid or received between parties to the contract that are an integral part of the effective interest rate transac tion costs and all other premiums or discounts when a receivable is impaired the group reduces the carrying amount to its recoverable amount being the esti mated future cash flows discounted at the original effective interest rate of the instrument and continues unwinding the discount as interest income once a financial asset or a group of similar financial assets has been written down as a new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee result of an impairment loss interest income is recognised using the rate of interest used to discount the future cash flows for the purpose of measuring the impairment loss dividend income is recognised when the legal right to receive dividends is established and inflow of economic benefits are probable asset management services the group is engaged in providing asset management services note such assets have been given to the group to be managed by third parties and these assets are recorded off balance sheet service fees are derived from management of such assets and no associated credit and market risks arise for the group leases group as the lessee leases of tangible fixed assets where the lessee acquires substantially all the risks and rewards of ownership are clas sified as finance leases other leases are classified as oper ating leases payments made under operating leases are charged to the statement of profit or loss on a straight line basis over the lease term the group primarily uses an operating lease for renting the premises a rental expense is recognised in the statement of profit or loss as administrative and other oper ating expenses taxation and deferred income tax corporate income tax in estonia according to the income tax act the annual profit earned by entities is not taxed in estonia and thus there are no temporary differences between the tax bases and carrying amounts of assets and liabilities and no deferred tax assets or liabilities arise instead of the income tax payable on profits the estonian entities pay corporate income tax on dividends fringe benefits gifts donations costs of entertaining guests non business related disbursements and adjustments of the transfer price the tax rate from is until on net dividends paid the corporate income tax arising from the payment of dividends is recog nised as a liability and an income tax expense in the period in which dividends are declared regardless of the period for which the dividends are paid or the actual payment date an income tax liability is due on the day of the month following the payment of dividends corporate income tax in other countries in accordance with the local income tax laws the net profit of companies located in latvia lithuania and finland that has been adjusted for the permanent and temporary differ ences as stipulated by law is subject to corporate income tax corporate income tax rates latvia lithuania finland deferred income tax is calculated on all significant tempo rary differences between the tax bases of assets and liabili ties and their carrying values in the consolidated statement of financial position the main temporary differences arise from depreciation and tax loss carry forwards deferred tax balances are measured at tax rates enacted or substan tively enacted at the balance sheet date which are expected to apply to the period when the temporary differences will reverse or the tax loss carry forwards will be utilised deferred tax assets and liabilities are netted only within the individual companies of the group deferred tax assets for deductible temporary differences and tax loss carry for wards are recorded only to the extent that it is probable that future taxable profit will be available against which the deductions can be utilised deferred income tax is provided on temporary differences arising on investments in subsidi aries and associates except where the timing of the reversal of the temporary difference is controlled by the group and it is probable that the temporary difference will not reverse in the foreseeable future a deferred income tax liability in respect of the groups avail able equity which would accompany the payment of avail able equity as dividends is not reported in the statement of financial position the maximum amount of income tax payable which would arise from paying out the retained earnings as dividends is disclosed in the note to the financial statements offsetting financial assets and liabilities are offset and the net amount reported in the consolidated statement of financial position only when there is a legally enforceable right to offset the recognised amounts and there is an intention to either settle on a net basis or to realise the asset and settle the liability simultaneously more detailed information in note statutory reserve capital statutory reserve capital is formed from annual net profit allocations to comply with the requirements of the commer cial code during each financial year at least one twentieth of the net profit shall be transferred to the statutory reserve until the reserve reaches one tenth of share capital stat utory reserve may be used to cover a loss or to increase share capital payments to shareholders from statutory reserve are not allowed new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee risk is defined as a potential negative deviation from expected financial results the group encounters several risks in its day to day operations the objective of risk management at the group is to recognise these risks to measure them appropriately and to manage them more broadly the aim of risk management is to increase the value of the group by minimising losses and reducing the volatility of results risk management at the group is based on a strong risk culture and is built on the principle of three lines of defence where the first line of defence the business units are responsible for taking risk and for day to day manage ment the second line of defence the risk management division is responsible for the development of risk manage ment methodologies and for the reporting of risks the third line of defence the internal audit conducts independent supervision over the entire group risk management princi ples requirements and areas of responsibility are described in policies and procedures in accordance with the capital management principles that the group has adopted it must have adequate capital to support risks capital management the goal of the groups capital management is to ensure continuity of the groups business and ability to generate return for its shareholders maintain a strong capital base supporting the development of business comply with capital requirements as established by supervision authorities the amount of capital that the group managed as of was eur thousand eur thousand the goals of the groups capital management are set based on both the regulative require ments and additional internal buffer the group follows the general principles in its capital management the group must be adequately capitalized at all times having necessary capital to ensure economic preservation and enabling financing of new profitable growth opportunities the main focus of the capital management is on tier own funds because only tier own funds can absorb losses all other capital layers in use are dependent of tier own funds volume capital of the group can be divided into regu lated minimum capital and capital buffer held by the group in order to reach its long term economic goals the group must on one hand strive towards propor tional lowering of the regulated minimum capital through minimizing risks and high transparency on the other hand the group must strive towards sufficient and conservative capital reserve which will ensure economic preservation even in the event of severe negative risk scenario the risk appetite set by the group is an impor tant input to capital management planning and capital goal setting higher risk appetite requires maintaining higher capital buffer note risk management paid in share capital share premium statutory reserves transferred from net profit other reserves accumulated profit deficit intangible assets subtracted net profit for accounting period non controlling interests total tier own funds subordinated debt total tier own funds total net own funds own funds in thousands of euros new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the net capital of a credit institution at any time shall be equal to or exceed the minimum amount of share capital prescribed in the credit institutions act eur million in the capital of banks and investment firms in the euro pean union became subject to a new legal framework crd iv crr largely based on the basel iii framework that was agreed in the basel committee on banking supervision the objective of the new legal framework is to strengthen the resilience of the financial sector to adverse economic shocks and thereby ensure an adequate and sustainable financing of the economy significant changes implemented by the new requirements include the requirement for credit institutions to maintain a higher level and quality of capital than before and a unified framework for designing liquidity buffers the new capital requirements directive also define measures for macro financial supervision that member states can use to control the behaviour of credit institutions in amplifying the cycles and to alleviate risks arising from market structure the regulation which is directly applicable in the member states obligates all credit institutions operating in the euro pean union including their consolidating holding compa nies and investment firms to maintain of common equity tier cet and of tier capital against risk assets the overall capital adequacy requirement cad including both tier and tier capital remains at the existing level in addition to the main requirements based on uniform rules the directive defines the principles for forming capital buffers in estonia in addition to the baseline capital requirements credit institutions have been subjected to capital mainte nance and systemic risk buffers which are imposed by the financial supervisory authority and imposed by estonian central bank respectively as the aforemen tioned buffers are incremental to the tier and the total baseline capital requirements the minimum tier require ment in estonia is and the total capital requirement is the latter is also subject to the specific supple mentary pillar requirement applicable to credit institutions an overview of capital requirements is provided in the table below capital requirements cad base capital requirement capital conservation buffer systemic risk buffer capital requirements total capital reqirements valid till august are provided in the table below capital requirements cad base capital requirement capital conservation buffer systemic risk buffer capital requirements total each year the groups supervisory board confirms the goals of capitalization and the target level of capital adequacy to cover potential risks arising from fast growth each year an internal capital adequacy assessment process icaap is performed the goal of which is to identify potential capital needs in addition to regulatory capital requirements internal capital adequacy targets for are next tier capital adequacy total capital adequacy capital adequacy and the use of regulatory capital are monitored by the finance department reports about the compliance with prudential and capital requirements for covering the risks are submitted regularly to supervision authorities lhv uses standard method for calculating capital requirements for credit and market risk and basic indicator approach calculating operational risk capital requirement the own funds of lhv varahaldus as the fund manager need at any given time exceed the minimum amount of share capital laid down in the investment funds act eur million the net own funds of a fund manager who managed pension funds with the market value of over eur million had to be till january at least eur million and of the market value of managed pension funds which exceeds eur million in addition the fund manager had to maintain additional own funds equal to of the portion of the market value of all managed funds which exceeds eur million starting from january the net own funds of a fund manager have to be of the market value of managed pension funds with the market value less than eur million and of the market value of managed pension funds with the market value more than eur million the group has complied with all capital requirements during the financial year and in previous years credit risk credit risk is the potential loss which would arise from the counterpartys inability to meet its obligations to the group credit risk arises from cash and cash equivalents deriva tives and deposits with banks and other financial institutions debt securities but mostly credit exposures to customers including outstanding loans given guarantees other receiv ables and commitments in order to mitigate credit risk group analyses the operations and financial position of its customers and business partners after authorising the loan the solvency of the customer and the value of the collateral are regularly monitored new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee distribution of credit risks the group classifies the financial assets exposed to credit risk in the following key categories a debt securities and derivatives b loans and advances to central bank and credit institutions referred to as banks in the tables and investment companies c leveraged loans loans secured by debt or equity securities incl repo loans d corporate loans and overdraft e retail loans f consumer loans without collateral g credit cards and overdraft to individuals h leasing i hire purchase j mortgage loans k financial guarantees l unused loan commitments a debt securities and derivatives the credit committee sets limits for taking credit risk asso ciated with debt securities considering the issuers rating the risk capital committee or authorised employees make decisions regarding investments within the limits set no principal and accrued interest receivables arising from debt securities are overdue the group s debt securities at fair value through profit or loss fvtpl and available for sale afs according to ratings given by standard poors or equivalent maximum exposure to credit risk in thousands of euros loans and advances to banks and investment companies note available for sale financial assets note financial assets at fair value debt securities note loans and advances to customers note receivables from customers note other financial assets note total assets exposures related to off balance sheet items note excluding performance guarantees total maximum exposure to credit risk ratings distribution in thousands of euros aaa a to a bbb to bbb non rated total note investment portfolio foreign exchange forwards afs total liquidity portfolio trading portfolio fvtpl new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee ratings distribution in thousands of euros aaa aa to aa a to a bbb to bbb b to bb total note rating in thousands of euros central bank the bank of estonia aa to aa a to a bbb to bbb non rated total note investment portfolio total total credit institutions credit institutions afs total liquidity portfolio investment companies investment companies trading portfolio fvtpl b loans and advances to banks and investment companies management estimates that the credit risk exposure from cash and cash equivalents held at the central bank other correspondent banks and investment institutions has inher ently low credit risk no loans and advances to central bank credit institutions and investments companies are overdue the funds of the group according to ratings given by standard poors or equivalent central bank without a rating in case there are ratings available from more than one rating agency the most conservative rating is used the positions are held as follows c leveraged loans the bank issues margin loans secured by debt or equity securities to its customers i e financial leverage the maximum amount of the loan depends on the market value of the assets held as collateral in the investment account and on the general limit set by the bank which is currently eur thousand or an equivalent in a foreign currency per customer granting loans above the threshold assumes an analysis of the portfolio offered as collateral by the customer and the decision is within the competence of the credit committee the list of acceptable marketable finan cial instruments and the levels of the required collateral are published on the banks website www lhv ee the risks arising from financial leverage are mitigated by constant monitoring of the market values of the financial instru ments required as collateral the group has set sufficiently conservative limits to the ratio of the loan and the collat eral assets value when the value of collateral assets falls below the established limit the bank is entitled to demand a transfer of additional collateral to the account or pay off the loan in the customers account without the customers trade order liquidating the collateral asset for this purpose see note for more detailed information on the credit quality of loans stress tests are carried out for evaluating the credit risk of leveraged loans in order to determine potential losses in case of changes in the value of collateral and for evaluating the credit risk of other loans the probability of the lender becoming insolvent the amount of loans not covered by collateral at the time of insolvency and the amount of the resulting related expected loss is assessed if necessary decisions are made in respect of allocating additional risk capital in order to cover credit risk the bank had no impaired leveraged loans as at and d corporate loans and overdraft corporate loans are evaluated on individual basis each customer is assigned a credit rating between where means the lowest probability of default and indicates insolvency credit rating is assigned to a customer consid ering combination of financial status and business risk new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee financial status is the basis for financial rating focusing on the companys assets and liabilities structure profitability cash flows and estimated results the emphasis of financial analysis is focused on the following period to the loan issu ance in parallel with financial analysis the companys busi ness risk including management and owners market situa tion and competition diversification and history of activities the quality of information and previous payment behaviour is analysed where there is consolidation group involved credit risk for the whole group is evaluated credit rating is assigned by credit analyst and confirmed by head of credit but final decision of risk taking is the unanimous decision by the credit committee the requirements for loan collateral are established in the credit policy of group and specified in credit risk manage ment rules of the bank the preferred collateral is where there is no strong correlation between the clients default risk and value of the collateral in general the pledged assets need to be secured the life of the collateral needs to be longer than the due date of the loan and the market value of the collateral needs to exceed the outstanding loan balance in addition to individual impairment assessment corpo rate loans provisions for potential credit loss is calculated based on historical performance of these loans and applied to current portfolio at the balance sheet date for the purpose of recognition of group based impairment losses the receivables are grouped into subclasses on the basis of homogeneous credit risk features considering customers payment practice and past due time for the receivables that have been grouped the amount of the impairment loss is the multiple of the carrying amount of the receivables in the group and the groups percentage rate of impairment loss the framework is based on the method of calculating the probability of default pd loss given default lgd and exposure at default ead ead consists of two compo nents the amount of the existing receivable and the amount of use of an estimated additional limit for calculating the latter the model includes the credit conversion factor ccf after issuing the loan follow up monitoring of each custom ers financial position is performed at least once a quarter all client ratings are reviewed at least once a year see note for more detailed information on the credit quality of loans as at the group based impairment reserve makes up of corporate loans and overdraft and the related interest receivables e retail loans in group started offering micro loans the loan is aimed to micro enterprices in growth stage and the maximum loan amount is up to eur during the year the existing corporate loan portfolio was divided into two credits below thousand euros were given over to retail banking and all credits exceeding thousand euros remained in corporate banking shifting smaller loans to retail banking was impor tant to keep corporate banking and credit analyses focus on larger and more individual approach requiring credit clients credits below thousand euros are analysed with more cost efficient scoring process the scoring is carried out at the request of the loan and it is one of the criterias for issuing the loan financial data and information on payment behav iour of the company is used when calculating the proba bility of default pd the structure of assets and liabilities are taken into account also the profitability and cash flow ratios regarding the guarantees the same principles apply as for business loans and overdrafts thus the essence of it does not constitute a new product but only retail loans separa tion from corporate loans in distinction then there is no comparative information as at december the retail loans are included inside loans to legal entities and overdrafts to legal entities lines in the report f consumer loans without collateral as consumer loans are homogeneous loans provisions for potential credit loss level is calculated based on historical performance of these homogeneous loans and applied to current portfolio at the balance sheet date for the purpose of recognition of group based impairment losses the receiv ables are grouped into subclasses on the basis of homoge neous credit risk features considering customers payment practice past due time and the time passed from initiation of proceedings by the bailiff in estonia the group has entered into agreements where overdue consumer loans are sold usually when loans reaching overdue of days similar contract was signed also in mokilizingas in lithuania in third quarter for the receivables that have been grouped the amount of the impairment loss is the multiple of the carrying amount of the receivables in the group and the groups percentage rate of impairment loss the framework is based on the method of calculating the probability of default pd loss given default lgd and exposure at default ead ead consists of two components the amount of the existing receivable and the amount of use of an estimated additional limit for calculating the latter the model includes the credit conversion factor ccf consumer loans are classified as performing less than days overdue and are not terminated non performing more than days overdue and or terminated and doubtful orig inal loan schedule has expired and there has been no cash flows received for continuous period of months fraud case etc loss rates are calculated from the past histor ical data for defaults and full history of recoveries in lithu new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee ania due to default rate volatility and uncertain data quality a conservative buffer of shall be imposed for portfolio as at buffer of was imposed in the new methodology valid since buffer for lithuanian branch will be and only for latvian branch and credit cards buffer will remain pd and lgd values are updated at least once a year with new historical data in order to maintain adequate provision levels on portfolio if necessary additional impairments will be made in the statement of financial position if non per forming or doubtful loan recovery is neither economical nor practical to continue the loan is written off as irrecoverable as at the group based impairment reserve makes up of consumer loans and the related interest receivables g credit cards the bank issues credit cards and partner credit cards in cooperation with tallinna kaubamaja similarly with consumer loans the credit card portfolio uses the credit scoring model to assess the customers credit behaviour the scoring is made at the time the application is filed and is one of the criteria for issuing a loan provisions for potential credit losses are made based on the same framework as the consumer loans the framework is based on pd lgd and ead indicators ead consists of two components the amount of the existing receivable and the amount of use of an estimated additional limit for calcu lating the latter the model includes the credit conversion factor ccf as of december the group based impairment reserve amounted to of credit card loans and related receivables h leasing the bank offers leasing products for individuals and legal entities the creditworthiness of customers is assessed by using scoring models which mainly consists of charac teristics described for consumer loans the leasing provi sioning is done based on leasing scoring models as of december the group based impairment reserve amounted to of leasing portfolio also in lithuania leasing products are offered to individuals as well as legal entities the creditworthiness of customers is assessed by using scoring models which mainly consists of characteristics described for consumer loans provision for finance lease is calculated using following methodology net book value of portfolio minus net present value of portfolio net present value is calculated from future cash flows discounted by effective interest rate and adjusted for coefficient of possible losses in addition to homogenous impairment some problematic loans are assessed individu ally based on the market distressed sale value of the under lying assets as of december the group based impairment reserve amounted to of leasing portfolio i hire purchase group offers hire purchase service for merchants through its subsidiary lhv finance in estonia the creditworthiness of customers is assessed by using scoring models which mainly consist of characteristics described for consumer loans similarly to other homogenous portfolios provisions are made based on the same framework as consumer loans as of december the group based impair ment reserve amounted to of hire purchase portfolio the group offers hire purchase to private individuals in lithuania the creditworthiness of customers is assessed and provisions are made based on the same framework as for lithuanian consumer loans as of december the group based impairment reserve amounted to of hire purchase portfolio j mortgage loans in group started to offer on a limited basis mortgage loans to customers in the tallinn and tartu region only in autumn group started offering the mortgage loan to a large scale of retail clients the creditworthiness of customers is assessed by using scoring models and maximum loan amount is in line with the regulations set by central bank of estonia the provisions are made based on the same framework as for consumer loans as of december the group based impair ment reserve amounted to of mortgage portfolio new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee credit quality of loans and advances to customers and off balance sheet liabilities loans to legal entities corporate loans retail loans overdraft hire purchase leveraged loans leasing credit card loans loans to individuals consumer loans hire purchase leveraged loans leasing mortgage loans credit card loans overdraft total note loans to legal entities corporate loans overdraft hire purchase leveraged loans leasing credit card loans loans to individuals consumer loans hire purchase leveraged loans leasing mortgage loans credit card loans overdraft total note loans and advances to customers in thousands of euros loans and advances to customers in thousands of euros neither past due nor impaired neither past due nor impaired past due but not impaired past due but not impaired collective impairment collective impairment individual impairment individual impairment net net individually impaired individually impaired total total there are several criteria for the loan to be considered indi vidually impaired groups internal default definition is in line with the generally accepted definition of default criteria such as payment delay more than days on any material credit obligation distressed restructuring of the credit obliga tion specific credit adjustment due to significant perceived decline in credit quality since inception significant financial difficulties of the obligor or likely bankruptcy process etc as part of risk analysis the group is regularly performing stress tests and sensitivity analysis regarding credit risk and its components such as pd lgd collective impairment new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee credit assessment of the group is based on historical loss rate and credit rating the group has performed stress test scenarios when pd and lgd estimations will both increase by percent for example from to the impact of the described stress test to impairments is aggregated in the table below the table includes loans which have collec tive impairment therefore excluding leveraged loans and which have material balances and potential impact loans to legal entities corporate loans incl overdraft leasing loans to individuals consumer loans hire purchase mortgage loan leasing credit card loans total impact to impairment as at in case pd and lgd levels will increase by in thousands of euros balance as at impairment with increased pds and lgds impact to impairment booked loans to legal entities corporate loans incl overdraft leasing loans to individuals consumer loans hire purchase mortgage loan leasing credit card loans total impact to impairment as at in case pd and lgd levels will increase by in thousands of euros balance as at impairment with increased pds and lgds impact to impairment booked the rating scale used for evaluating the pd of corporate loans consists of a total of credit ratings where stands for the lowest and for the highest credit risk ratings and are attributed only on the basis of average ratings of rating agencies fitch moodys and standard poors and are subject to the groups financial analysis as neces sary thus ratings and are attributed only to international enterprises organisations local governments and countries whose solvency has been confirmed by the rating agencies with their investment grade rating also rating classes are partly based on the evaluation of rating agencies although this category includes mainly estonian enterprises that do not have an international rating distribution of internal ratings of corporate loans minimum credit risk the rating is only assigned on the basis of the ratings of rating agencies the average of the ratings assigned by rating agencies must be the equivalent of at least aaa moodys aaa minimum credit risk the rating is only assigned on the basis of the ratings of rating agencies the average of the ratings assigned by rating agencies must be the equivalent of at least aa moodys low credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least aa moodys low credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least a moodys low credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least bbb moodys low credit risk the rating is assigned on the new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least bbb moodys average credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least bbb moodys average credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least bb moodys heightened credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equiv alent of at least bb moodys high credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least b moodys high credit risk the rating is assigned on the basis of the assessments of rating agencies or lhv the average rating of rating agencies must be the equivalent of at least b moodys non satisfactory rating the rating is assigned on the basis of the assessments of rating agencies or lhv the credit analyst deems the companys finan cial position to be sufficiently weak indicating a very high probability of default the average rating of rating agencies must be the equivalent of at least c moodys the obligor is in default low credit risk low credit risk low credit risk medium credit risk medium credit risk heightened credit risk high credit risk high credit risk non satisfactory rating insolvent total low credit risk low credit risk low credit risk medium credit risk medium credit risk heightened credit risk high credit risk high credit risk non satisfactory rating insolvent total distribution of corporate loans and overdraft by internal ratings in thousands of euros distribution of corporate loans and overdraft by internal ratings in thousands of euros neither past due nor impaired neither past due nor impaired past due but not impaired past due but not impaired individually impaired individually impaired total total the group considers the loan as non performing and assesses impairment when the loan payments have not been collected by the due date and or the expected cash flows from the realization of collateral are not sufficient for covering the carrying amount of the loan principal and interest payments the group evaluates these loans indi vidually in case of individual evaluation the amount of the impairment loss is the difference between the carrying value of the receivable and expected cash flows discounted at the effective interest rate in the group provisioned new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee corporate loans in the total amount of eur thousand eur thousand loan customers with ratings of are subject to more frequent monitoring during which the solvency of the borrower is evaluated in case of a deterioration of payment behaviour the requirement for individual provisioning is eval uated considering among other factors the collateral of the loan therefore some loans are not individually impaired the ratings based rate of the collective provisioning is applied to those loans in addition to loans issued loan agreements have been concluded and signed for the unused loan commit ment amount of thousand euros at thousand euros see note in the following table there are presented unused portions of corporate loans and financial guarantee limits no provisions have been made for off balance sheet liabilities because the value of collateral exceeds the value of potential liabil ities in addition to that as at the group has also committed to grant mortgage loans hire purchase loans consumer loans overdraft loans and credit card loans to individuals in total amount of eur thousand eur thousand all uncommissioned loans available to individuals have at least good internal rating as at the end of each reporting period distribution of internal ratings for other loan products excellent leveraged loans secured by debt or equity securities and loans with very low business risk very good loans with lower business risks and no past due payments good loans with lower business risks and past due payments up to days satisfactory loans with average business risk and up to days past due weak or doubtful loans carrying higher busi ness risk past due more than days and portfolio in proceedings by the bailiff low credit risk low credit risk medium credit risk medium credit risk heightened credit risk high credit risk high credit risk total credit quality of off balance sheet liabilities unused loan commitments for corporate loans and financial guarantees in thousands of euros neither past due nor impaired excellent good and very good past due but not impaired good satisfactory weak or doubtful individually impaired good satisfactory weak or doubtful total as of in thousands of euros retail loans leveraged loans consumer loans hire purc hase over draft mortgage loans total credit cards leasing new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee neither past due nor impaired excellent good and very good past due but not impaired good satisfactory weak or doubtful individually impaired good satisfactory weak or doubtful total listed securities unlisted equity securities mortgages real estate guarantee of kredex and rural development foundation pledges of rights of claim deposits leased assets others unsecured loans total listed securities unlisted equity securities mortgages real estate guarantee of kredex and rural development foundation pledges of rights of claim deposits leased assets others unsecured loans total as of in thousands of euros loans against collateral as at in thousands of euros loans against collateral as at in thousands of euros leveraged loans consumer loans credit cards consumer loans overd raft to private individuals credit cards consumer loans overd raft to private individuals hire purc hase leasing leasing over draft hire purchase hire purchase mortgage loans mortgage loans mortgage loans total total total credit cards corporate and retail loans including overdraft corpo rate loans including overdraft leasing leveraged loans leveraged loans in the table below collateral information of loans and advances to customers are disclosed based on the collat eral type and the fair value of collateral held the under col lateralised amount is presented as unsecured loans new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee unsecured loans in the years of and include credit card loans and lithuanian consumer loans collaterals for leveraged loans are monitored on daily basis and if collateral value is falling immediate measures are taken to avoid credit losses as of december and as of december all leveraged loans and repur chase loans are over collateralized consumer loans and credit card loans are issued without collateral and risk miti gation is done by regular monitoring of clients payment behaviour leasing hire purchase mortgage loans and overdraft to private individuals are all over collateralized the group monitors customers in arrears of leasing hire pur chase mortgage loans and overdraft to private individuals on regular basis in relation to under collateralized corporate loans it should be taken into consideration that the group has assessed the market value of certain collaterals conservatively personal sureties commercial pledges under collateralized loans are mainly considered to bear higher risk for which the group carries out monthly monitoring in credit committee in order to mitigate potential credit losses under collateralized loans include contracts more than days overdue totalling eur thousand eur thousand euros with a collateral value of thousand eur thou sand euros corporate loans and corporate credit lines in thousands of euros as at as at fair value of collateralcarrying value fair value of collateral over collateralized loans under collateralized loans carrying value structure of past due but not impaired loans according to past due time loans which have overdue interest or principal payments as at the balance sheet date are divided in past due categories according to the past due time from the earliest outstanding payment structure of individually impaired loans according to past due time loans which have overdue interest or principal payments as at the balance sheet date are divided in past due categories according to the past due time from the earliest outstanding payment past due receivables days days days days days more than days total past due receivables days days days days days more than days total as at in thousands of euros as at in thousands of euros corporate loans incl overdraft corporate loans incl overdraft credit cards credit cards leasing leasing hire purchase hire purchase consumer loans consumer loans total total retail loans leveraged loans leveraged loans new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee no past due payments past due receivables days days days days days more than days total no past due payments past due receivables days days days days days more than days total credit quality of other receivables in thousands of euros receivables neither past due nor impaired receivables past due but not impaired incl receivables from individuals incl receivables from legal entities total note as at in thousands of euros as at in thousands of euros corporate loans incl overdraft corporate loans incl overdraft credit cards credit cards leasing leasing hire purchase hire purchase consumer loans consumer loans total total retail loans leveraged loans leveraged loans as of february other receivables of eur thou sand euros are uncollected and in previous years the group has not written off any receivables other financial assets see note in amount eur thousand eur thousand are guarantee deposits on the baltic stock exchanges held to guarantee securities trading activity on the stock exchanges of tallinn riga and vilnius and the deposits of visa and mastercard to guarantee credit card transactions the group considers the counterparties credit risk as very low market risk market risk arises from the groups trading and investment activities in the financial markets from interest rate products foreign exchange and stock markets as well as lending activities and collecting financial resources market risk is a potential loss which may arise from unfavourable changes in foreign exchange rates prices of securities or interest rates internal judgement is used to assess potential losses in order to mitigate market risk conservative limits have been established for the trading portfolio and open foreign currency exposures lhv varahaldus invests most of its available resources into his own managed pension funds the management of lhv varahaldus is responsible for monitoring of the market risk foreign currency risk foreign currency risk may arise from acquisition of secu rities mostly denominated in foreign currencies or foreign currency receivables and liabilities the treasury of the bank is responsible for daily monitoring of open foreign currency positions the groups foreign currency risk management is based on risk policies limits and internal procedures new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the internal limits for open foreign currency positions are proposed by risk control unit and treasury and the limits are set internally by risk and capital committee the limits set by the committee are maximum nominal limits in euro equivalent with a respect to the groups net own funds limits imposed on individual currencies are in line with the boundaries proposed by the bank of estonia if the open currency position exceeds the limits set by the committee measures are immediately implemented to close or reduce such positions hedging the risk with relevant instruments such as foreign currency forwards or futures a sensitivity analysis has been performed for the effect of possible reasonable changes attributable to open currency positions on statement of profit or loss with the assumption of other conditions remaining constant impact on statement of profit or loss in thousands of euros usd exchange rate sek exchange rate gbp exchange rate chf exchange rate open currency exposures the following tables present the risks arising from open currency exposures assets and liabilities denominated in foreign currencies have been presented in euro equiva lent in respective columns according to the exchange rate prevailing at the balance sheet date derivatives reported at fair value in the statement of financial position have been included at contractual amounts under off balance sheet assets and liabilities open currency exposure and the volume of financial assets and liabilities of the group at the balance sheet date do not significantly differ from the average exposure during the year the balances of total assets and total liabilities bearing currency risk do not include currency futures at their fair value but they are shown here at their full contractual cash flow amounts as off balance sheet assets and liabilities also the table does not include the assets tangible and intangible assets and liabilities provisions not bearing currency risk and equity in thousands of euros eur chf gbp sek usd other total assets bearing currency risk due from banks and investment companies note financial assets at fair value note loans and advances to customers note receivables from customers note other financial assets note total assets bearing currency risk liabilities bearing currency risk deposits from customers and loans received note financial liabilities at fair value note accounts payable and other financial liabilities note subordinated debt note total liabilities bearing currency risk open gross position derivative assets at contractual value open gross position derivative liabilities at contractual value open foreign currency position new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros eur chf gbp sek usd other total assets bearing currency risk due from banks and investment companies note financial assets at fair value note loans and advances to customers note receivables from customers note other financial assets note total assets bearing currency risk liabilities bearing currency risk deposits from customers and loans received note financial liabilities at fair value note accounts payable and other financial liabilities note subordinated debt note total liabilities bearing currency risk open gross position derivative assets at contractual value open gross position derivative liabilities at contractual value open foreign currency position price risk financial instruments bearing price risk at group are secu rities held in the trading portfolio and investment portfolio note at the group limits are set for the size of the trading portfolio and acceptable credit quality ratings are specified for debt securities in the investment portfolio the risk management unit monitors the compliance with limits pursuant to the investment funds act the mandatory shares of lhv varahaldus as the management company is of the number of units in each of the mandatory pension fund managed by it based on historical performance and volatility of the pension fund unit nav changes the price changes should remain in the next month period with the most likely change and the impact to state ment of profit or loss is presented in the next table lhv varahaldus invests the remaining available assets in euros to ensure liquidity and to not bear investment risk bank does not hold significant amounts of equity securities in its position see note due to which the sensitivity to change in the market value of these positions is marginal banks debt securities portfolio recognised at the market price is short term and of high quality therefore the effect of the changes in market risk premiums on the market value of the debt security portfolio should remain at around sensitivity analysis of the impact to net result from the risk exposures of the groups largest entity as lhv pank against reasonable possible change in thousands euros impact on statement of profit or loss in thousands of euros equity securities mandatory pension fund units debt securities fvtpl impact on other comprehensive income debt securities afs interest rate risk interest rate risk reflects the mismatch in the statement of financial position items and the off balance sheet items when interest rate repricing periods volumes or the under lying interest rate of assets liabilities and derivatives do not correspond exactly groups interest rate risk manage ment is based on risk policies limits and internal proce dures resulting in identification of all significant sources of interest rate risk and balanced risk taking interest rate risk is measured through various scenario analyses by calculating several net interest income scenarios and comparing the difference between these scenarios the basic measures of interest rate risk uses two scenarios an increase and new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee a decrease of interest rates to measure the effect on the groups net interest income for a month period internal limits for interest rate risk management are set by as lhv panks risk and capital committee and the operational risk management is the treasurys responsibility the goal of monitoring measuring and managing interest rate risk is to evaluate the profitability of the groups inter est bearing products to forecast interest income and to set limits for risk management in order to prevent signifi cant reduction of income through limitation of risks in three aspects cash flow interest rate risk whereby in case of a change in market interest rates a change in annual net interest income is limited as compared to the esti mated actual income based on the term structure of instru ments currently in the statement of financial position fair value interest rate risk whereby in case of a change in market interest rates a change in the groups economic value is limited fair value of assets and liabilities is determined by discounting the future cash flows at the market rate of interest the balance sheet and margins on assets and deposits are assumed to be constant over time the deposits interest rates did not change in remaining at the level of up to up to in the interest rates for leveraged loans granted are changed at most once a month according to fluctuations in market interest rates in the interest rate on loans received for specific purposes was the effec tive interest rate of subordinated debts entered into in was and the effective interest rate of subordinated debts entered into in and entered into in was the information about subordinated debt contractual interest rates is provided in note as at stress tests were performed to the biggest interest sensitive subsidiary of the group as lhv pank an increase of percentage point in interest rates would affect the banks annual net interest income and profit by eur thousand eur thou sand in the same time a decrease of percentage point in interest rates would affect the banks annual net interest income profit by eur thousand eur thousand a percentage point increase in market interest rates would raise the banks economic value i e equity by eur thousand eur thousand a percentage point decrease in market interest rates would decrease the banks economic value equity by eur thousand eur thousand effect on the banks economic value is positive due to the fact that the bank has majorly invested in current assets and because of the prolonged nature of demand deposits hence the average duration of assets exposures to interest rate risk is shorter than the average duration of liabilities exposures to interest rate risk the table below shows the structure of the financial assets and liabilities exposures to interest rate risk of the group according to the recalculation dates of interest rates at the principal amounts of receivables and liabilities leveraged loans are treated as a one month product maturing at the next interest fixing date demand deposits are divided as follows liquidity accounts with the duration of months and other demand deposits with the duration of years based on their behav ioural nature the interest rate of demand deposits is not sensitive to market rate fluctuations the prices of derivative contracts on the market are the bases for the assumption that the interest rates of the time deposits will not change drastically during the next years which would cause a significant amount of the demand deposits to transfer to term deposits financial assets due from banks and investment companies note financial assets at fair value debt securities note loans and advances to customers note total financial liabilities deposits from customers and loans received note subordinated debt note total net interest sensitivity gap exposures to interest rate risk in thousands of euros up to months over year subtotal accrued interest impair ments total months years new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee financial assets due from banks and investment companies note financial assets at fair value debt securities note loans and advances to customers note total financial liabilities deposits from customers and loans received note subordinated debt note total net interest sensitivity gap up to months over year subtotal accrued interest impair ments total months years liquidity risk liquidity risk relates to the solvency of the group to meet its contractual obligations on time and it arises from differences between maturities of assets and liabilities the groups liquidity management and strategy is based on risk policies resulting in various liquidity risk measures limits and internal procedures as per policy statements the groups liquidity management reflects a conservative approach towards liquidity risk the liquidity risk management includes stress testing and business continuity plan for liquidity manage ment stress testing framework includes a survival period metrics which represents a combined liquidity risk scenario including both idiosyncratic and market wide stress internal metrics are complemented by the basel iii metrics the liquidity coverage ratio lcr and the net stable funding ratio nsfr which the group is fully compliant with as of and the aim of the lcr standard is to ensure that the group has an adequate amount of unencumbered assets of high quality and liquidity that could be monetised without incurring material losses to meet a liquidity requirement in a day stress scenario the output of the standard is the ratio of liquid assets to stress scenario liquidity requirement which has a limit of the groups liquidity coverage ratio level as at was the objective of the nsfr is to ensure that the group has adequate stable funding sources to finance longer term assets according to the proposals published in the banking regulation and the banking committee of basel in october the groups nsfr level as at was the net stable funding ratio will be imposed as minimum requirement from the year the treasury of the bank is responsible for the manage ment of liquidity risk in order to hedge liquidity risk the probable net position of receivables and liabilities by matur ities is regularly monitored and adequate amounts of liquid assets are kept in each time period also the concentration of the groups liabilities by maturities is monitored to enable covering unexpected monetary outflows the group holds a liquidity buffer the liquidity buffer consists of cash and deposits with the central bank and liquid securities held by the treasury which can be readily sold or used as a collat eral in funding operations with the central bank the group has a sufficient supply of liquid resources to enable issuing standby loans as at and the group does not have any debts past due short term loans can be obtained from the central bank secured by the majority of instruments from the debt secu rities portfolio the following tables present the distribution of financial assets and liabilities excl derivatives by due dates and by future contractual undiscounted cash flows and in a sepa rate column the statement of financial position balance is disclosed in the maturity analysis the cash flows are split into the maturity buckets in which the cash flows occur including interest cash flows explanation of the fair value of these financial assets and liabilities is presented in note exposures to interest rate risk in thousands of euros the contractual term of subordinated debts received in and in is years and the interest rate is fixed at the contractual term of subordinated debts received in is years and the interest rate is fixed at new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee liabilities by contractual maturity dates deposits from customers and loans received note subordinated debt note accounts payable and other financial liabilities note unused loan commitments note financial guarantees by contractual amounts note foreign exchange derivatives gross settled financial liabilities at fair value note total liabilities assets held for managing liquidity risk by contractual maturity dates due from banks and investment companies note financial assets at fair value debt securities note loans and advances to customers note receivables from customers note other financial assets note foreign exchange derivatives gross settled total assets held for managing liquidity risk maturity gap from assets and liabilities liabilities by contractual maturity dates deposits from customers and loans received note subordinated debt note accounts payable and other financial liabilities note unused loan commitments note financial guarantees by contractual amounts note foreign exchange derivatives gross settled financial liabilities at fair value note total liabilities assets held for managing liquidity risk by contractual maturity dates due from banks and investment companies note financial assets at fair value debt securities note loans and advances to customers note receivables from customers note other financial assets note foreign exchange derivatives gross settled total assets held for managing liquidity risk maturity gap from assets and liabilities in thousands of euros in thousands of euros on demand on demand up to months up to months months months total total carrying amount carrying amount years years over years over years new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros current assets due from central bank note due from credit institutions note due from investment companies note available for sale financial assets note financial assets at fair value through profit or loss note loans and advances to customers note receivables from customers note other assets note total current assets non current assets available for sale financial assets note financial assets at fair value through profit or loss note loans and advances to customers note other financial assets note tangible assets note intangible assets note goodwill note total non current assets total assets note liabilities current liabilities deposits from customers and loans received note financial liabilities at fair value through profit or loss note accounts payable and other liabilities note total current liabilities non current liabilities deposits from customers and loans received note subordinated debt note total non current liabilities total liabilities note the following table presents the distribution of assets and liabilities by classification of current and non current distribution of assets and liabilities by geographic region is presented below as at the loans issued to customers total had a large risk exposure i e more than of the net own funds nof of group either individually or via group risk totalling of nof the group has invested in the debt securities of issuers with a large risk exposure totalling of nof unused loan commitments in amount of eur thousand are for estonian residents and in the amount of eur thousand for lithuanian residents eur thousand for estonian residents new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros in thousands of euros estonia estonia latvia latvia the nether lands the nether lands germany germany other eu other eu usa usa other other total total lithuania lithuania finland finland due from banks and investment companies note financial assets at fair value note loans and advances to customers note receivables from customers note other financial assets note total financial assets deposits from customers and loans received note subordinated debt note accounts payable and other financial liabilities note financial liabilities at fair value note total financial liabilities due from banks and investment companies note financial assets at fair value note loans and advances to customers note receivables from customers note other financial assets note total financial assets deposits from customers and loans received note subordinated debt note accounts payable and other financial liabilities note financial liabilities at fair value note total financial liabilities new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee distribution of loans granted by industry gross in thousands of euros individuals real estate manufacturing art and entertainment financial services wholesale and retail administrative activities transport and logistics agriculture other servicing activities construction information and communication professional scientific and technical activities education other areas at activities total note level level level financial assets at fair value through profit or loss shares and fund units note available for sale debt securities note debt securities at fair value through profit or loss note interest rate swaps and foreign exchange forwards note total financial assets financial liabilities at fair value through profit or loss interest rate swaps and foreign exchange forwards note total financial liabilities in thousands of euros level level level financial assets at fair value through profit or loss shares and fund units note available for sale debt securities note debt securities at fair value through profit or loss note total financial assets financial liabilities at fair value through profit or loss interest rate swaps and foreign exchange forwards note total financial liabilities fair value of financial assets and financial liabilities in thousands of euros shares and fund units include the groups subsidiary as lhv varahaldus investment into pension fund units in the amount of eur thousand eur thousand euros pursuant to the investment funds act the mandatory shares of lhv varahaldus as the management company is of the number of units in each of the mandatory pension fund managed by it the management board of the group has assessed the fair value of assets and liabilities carried at amortised cost in the statement of financial position for estimating fair value the future cash flows are discounted on the basis of the market interest yield curve levels used in hierarchy level quoted prices in active market level valuation technique based on observable market data as inputs rates and interest curves from similar transactions level other valuation techniques e g discounted cash flow method with judgemental inputs new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee interest rate swaps and foreign exchange forwards are instruments where active markets supply observable inputs to the valuation model which is used for establishing the fair value the fair value for such over the counter derivatives is calculated as a theoretical net present value npv based on independently sourced market parameters assuming no risk and uncertainties market observable zero coupon yield curve is used for discounting in npv calculations as lhv varahaldus investments into pension fund units have quoted prices from the market however the market is inactive and therefore disclosed as level as at the fair value of corporate loans is eur thousand lower than their carrying amount eur thousand higher the loans have been issued at market conditions in the segment that the group operates in and therefore their fair value is not much different from their carrying amount as at and the fair value level of corporate loans and overdraft is as significant judgmental assumptions are used for the valuation process discounted cash flow method with current market interest interest rate used is the average interest rate of corporate loans issued in last months of the reporting period and eur thousand carrying value fair value difference level lease interest rates offered to customers generally corre spond to interest rates prevailing in the market for such products considering that the interest rate environment has been relatively stable since the group started to provide leasing consequently the fair value of lease agreements does not materially differ from their carrying amount as significant management judgment is required to determine fair value leases are classified as level in the fair value hierarchy mortgage loans interest rates offered to customers corre spond to interest rates prevailing in the market for such prod ucts as mortgage product is long term product then bank has set up separate process monitoring the rates offeref by competitiors considering that bank has actively started offering the product only in october the interest rate environment has not changed consequently the fair value of mortgage agreements does not differ from their carrying amount as significant management judgment is required to determine fair value mortgage loans are classified as level in the fair value hierarchy leveraged loans hire purchase and credit cards granted to customers are of sufficiently short term nature and they have been issued at market terms therefore the fair market rate of interest and also the fair value of loans do not change significantly during the loan term the effective interest rate of consumer loans issued is at the same level as the interest rate of the loan product offered in the market and it can be stated that the carrying amount of loans does not signifi cantly differ from their fair value the fair value level of lever aged loans hire purchase credit cards and consumer loans is as significant judgmental assumptions are used for the valuation process trade receivables other than the receivables related to loans and advances to customers and accrued expenses and other financial liabilities have been incurred in the course of ordinary business and are payable in the short term there fore the management estimates that their fair value does not significantly differ from their carrying amount these receivables and liabilities are interest free the fair value level of those receivables accrued expenses and other financial liabilities is customer deposits and loans received with fixed interest rates are primarily short term and pricing of the deposits is subject to market conditions as a result the fair value of deposits determined using the discounted future cash flows do not significantly differ from their carrying amount the fair value level of customer deposits is as significant judgmental assumptions are used for the valuation process the majority of subordinated debts were received in and in and the remainder in which was redeemed in these loans carry approximately equal interest rates and and considering the short term between the loan received in october and the balance sheet date it can be concluded that no material changes have occurred in interest rates as of the balance sheet date therefore the fair value of loans does not differ significantly from their carrying amount as significant management judgment is required to determine fair value loans are classified as level in the fair value hierarchy for the term structure of financial assets and financial liabili ties refer to note operating risk operating risk is a potential loss caused by human process or information system flaws when completing transac tions transaction limits and competence systems are used to minimise potential losses and the segregation of duties principle is used in the groups working procedures according to which there should be an approval by at least new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee two employees or units in order to carry out a transaction or procedure the information received from monitoring operating risk helps to gather initial information to secure capital adequacy of the group and to evaluate capital requirements the anal ysis of cases collected into the database enables the group to identify the flaws in rules of procedure avoid making mistakes in the future and mitigate possible risks or define the terms of their acceptance the risk control manager of group is responsible for collecting information compliance control and internal audit department have an important role in evaluating monitoring and mitigating the operating risk pursuant to the credit institutions act and securities market act the main task of compliance officer is to define the risks of non compliance of the activities of the group with legislation recommended guidelines of the financial supervision authority and procedure rules of the group considering the nature range and complexity of busi ness and services rendered and arrangement of mitigating or avoiding those risks internal audit is an independent and objective function assuring and consulting activity that is targeted at improving the groups performance and adding value internal audit helps achieving the goals of the group using a systematic and disciplined approach to assess and increase the efficiency of the risk management control and organisational management process gross amounts before offsetting in the statement of financial position offsetting cash collateral received net amounts net amount of exposure presented in the statement of financial position related arrangements not set off in the statement of financial position financial instruments assets prepayments to merchants for registered customer contacts client receivables leveraged loans incl repo loans note liabilities payables to merchants assets prepayments to merchants for registered customer contacts client receivables leveraged loans incl repo loans note liabilities payables to merchants offsetting assets and liabilities the group has offset the following assets and liabilities new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note significant management estimates and assumptions note business segments in accordance with ifrs several financial figures presented in the financial statements are strictly based on critical accounting estimates and assumptions made by manage ment which affect the reported amounts of the assets and liabilities and disclosure of contingent assets and liabilities presented in the financial statements at the balance sheet date and the reported amounts of income and expenses of subsequent reporting periods although these estimates have been made to the best of managements knowledge and their judgement of current events the actual outcome may ultimately not coincide with them and may significantly differ from these estimates managements estimates have been applied to valuation of loans receivables and invest ments notes and estimates and judgments are continually evaluated based on historical experience and other factors including expec tations of future events that are believed to be reasonable in the circumstances changes in managements estimates are reported prospectively bank divides its business activities by main business segments retail banking private banking and corpo rate banking the business segments form a part of the group with a separate access to financial data and which are subject to regular monitoring of operating result by the groups decision maker the full planning and kpi setting is done on segments level for each of the segment full balance sheet and profit and loss statement is prepared in addition to these high level segments group has planning measure ment on department level grouping is done based on the client product base where similar departments are grouped under one segment private banking covers high net worth customers who main product is investments retail banking segment covers all private individuals and small legal enti ties with credit exposure under eur thousand this is the regular universal banking segment offering payments cards credits etc corporate banking segment covers all large corporates and other legal entities with credit exposure over eur thousand the main products are credits and payments asset management is the pension managemet segment covering pension second and third pillar hire pur chase units both in estonia and in lithuania are separate legal entities covering small loan business to private individ uals these portfolio are relatively small but due to clients different payment history these segments have to be looked separately the management board of as lhv group has been designated as the decision maker responsible for allocation of funds and assessment of the profitability of the business activities the result posted by a segment includes revenue and expenditure directly related to the segment the revenue of a reported segment includes gains from transactions between the segments i e loans granted by as lhv pank to other group companies the division of interest income and fee and commission income by customer loca tion has been presented in notes and the group does not have any customers whose income would account for more than of the corresponding type of revenue new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros in thousands of euros retail banking private banking hire purc hase and consumer finance in estonia hire purc hase and consumer finance in lithuania other activities intra seg ment eliminations total corporate banking asset manage ment retail banking private banking hire purc hase and consumer finance in estonia hire purc hase and consumer finance in lithuania other activities intra seg ment eliminations total corporate banking asset manage ment interest income interest expense net interest income fee and commission income fee and commission expense net fee and commission income net income net gains from financial assets administrative and other operating expenses staff costs operating profit income loss from associates impairment losses on loans and advances income tax net profit total assets total liabilities interest income interest expense net interest income fee and commission income fee and commission expense net fee and commission income net income net gains from financial assets administrative and other operating expenses staff costs operating profit income loss from associates impairment losses on loans and advances income tax discontinued operations net profit total assets total liabilities new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note subsidiaries and associated companies goodwill as at the groups subsidiaries which have been consolidated in these financial statements include as lhv pank estonia ownership interest as lhv varahaldus estonia ownership interest cuber technology oü estonia ownership interest uab mokilizingas lithuania ownership interest share as lhv finance estonia ownership interest through as lhv pank as lhv pank paid eur thousand euros of monetary contribution for of ownership in lhv finance and a non controlling interest paid eur thousand euros of monetary contribution for of ownership the contri bution of the non controlling interest has been reflected in the consolidated statement of financial position on the line non controlling interest as at the group does not have any associates as at the group had ownership interest in oü svipe which was sold in the beginning of the for eur thousand total book value of associates as of was eur thousand as in investment of oü svipe was impaired by eur thousand in january subsidiary as lhv pank disposed its finnish branch for the amount of thousand euros the transfer included all assets and rights connected with the business and all obligations liabilities and undertakings dormant subsidiary lhv finance oy was liquidated in as at goodwill in amount eur thousand in the consolidated financial statements of as lhv group consists of positive goodwill which had arisen on the acquisi tion of the ownership interests in as lhv varahaldus in the amount of eur thousand positive goodwill which had arisen after the conclusion of a purchase contract of as lhv varahaldus entered into in in the amount of eur thousand positive goodwill which had arisen on the acqui sition of the ownership interests in danske capital as by as lhv varahaldus in the amount of eur thousand impairment tests were performed as at and as at the cash generating unit of goodwill is as lhv varahaldus the calculation of the value in use is based on the following assumptions the volume of assets under management is expected to increase per annum increase of income of fund manager is expected to be average of per annum due to the economic environment growth of in indirect costs is expected per annum the discount rate used is when using the main assumptions the management used previous years experience and its best estimate in respect of probable expectations expectations are more conservative due to the changes in the market situation and legislation based on the results of the impairment test performed as at and the recoverable amount of cash generating unit is higher than its carrying amount as a result of which no impairment losses have been recognized on may as lhv varahaldus acquired shares of danske capital as and its managed funds this purchase enabled lhv varahaldus to increase significantly market share and also profitability immediately after conclusion of the transaction of purchase and sale of shares on may lhv varahaldus launched the merger of the two manage ment companies the merger took effect on july with the balance sheet date of may the impact of revenue in is eur thousand and the impact of profit and loss is eur thousand as lhv varahaldus recognised the acquisition of danske capital as in accordance with requirements of ifrs by carrying out purchase price allocation in the course of the purchase price allocation the value of assets of the danske capital as was assessed and the assets were recognised in fair value on the transaction date date of the financial information used for the purchase price allocation was which is the date closest to the transaction date with reliable financial information available no signifi cant transactions occurred between the financial information date and transaction date that had a significant impact on the value of net assets acquired the goodwill is related with the expected synergies of managing acquired pension funds acquisition related costs of eur thousand have been charged to administrative expenses in the consolidated statement of profit or loss for the year ended december the total fair value of assets cash clients agreements pension fund units and other receivables was thou sand euros the total fair value of payables payables to employees tax payables and other payables was thou sand euros new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in thousands of euros fair values recorded on acquisition cash and cash equivalents pension fund units other financial assets client agreements other financial liabilities total net assets identified amount paid by lhv varahaldus as goodwill acquired by the group outflow of cash and cash equivalents on acquisition set out below are the summarised financial information for each subsidiary that has non controlling interests that are material to the group the information disclosed is the amount before inter company eliminations uab mokilizingas as lhv finance summarised statement of financial position loans and advances to customers and other current assets non current assets current liabilities non current liabilities total net assets summarised statement of profit or loss and other comprehensive income total net interest and fee income profit before income tax income tax expense net profit total comprehensive income profit and other comprehensive income allocated to non controlling interests summarised statement of cash flows cash generated from operations interest paid income tax paid net cash generated from used in operating activities net cash generated from used in investing activities net cash generated from used in financing activities net increase decrease in cash and cash equivalents cash and cash equivalents at beginning of year cash and cash equivalents at end of year uab mokilizingas as lhv finance uab mokilizingas as lhv finance new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note net interest income interest income corporate loans incl retail loans incl loans to related parties note hire purchase consumer loans leasing incl loans to related parties note leveraged loans and lending of securities from debt securities incl debt securities available for sale incl debt securities at fair value through profit or loss credit card loans mortgage loans from balances with credit institutions and investment companies other loans total interest expense deposits from customers and loans received incl loans from related parties note from balances with central bank subordinated debt note incl loans from related parties note total net interest income interest income of loans by customer location interests from bank balances and debt securities not included estonia latvia lithuania total interests calculated on impaired loans in is eur eur thousand new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note net fee and commission income note operating expenses fee and commission income security brokerage and commission fees asset management and related fees incl funds managed by the group currency exchange fees fees from cards and settlements fee for snoras portfolio administration other fee and commission income total fee and commission expense security brokerage and commission fees paid expenses related to cards expenses related to acquiring other fee expense total net fee and commission income commission fees from pension and investment funds are calculated as a fixed percentage of total assets of the fund and varies between p a mokilizingas is providing portfolio administration services to snoras bank to whom the portfolio was sold in fee and commission income by customer location estonia latvia lithuania sweden other total in thousands of euros wages salaries and bonuses social security and other taxes total staff costs it expenses information services and banking services marketing expenses office expenses transportation and communication costs training and travelling expenses of employees other outsourced services other administrative expenses depreciation operating lease payments other operating expenses total other operating expenses total operating expenses lump sum payment of social health and other insurances the average number of employees working for lhv group in was new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note due from central bank credit institutions and investment companies note available for sale financial assets note financial assets and liabilities at fair value through profit or loss in thousands of euros demand and term deposits with maturity less than months statutory reserve capital at central bank demand deposit from central bank total cash and cash equivalents in the statement of cash flows securities held for trading shares and fund units debt securities designated as at fair value through profit or loss upon initial recognition fund units incl investments in managed pension funds foreign exchange forwards total financial assets interest rate swaps and foreign exchange forwards total financial liabilities financial assets at fair value through profit or loss net changes of investment securities at fair value through profit or loss interest income note revaluation financial assets at fair value through profit or loss net changes of investment securities at fair value through profit or loss interest income note revaluation financial assets at fair value through profit or loss available for sale financial assets proceeds from disposal and maturities of assets available for sale interest income note revaluation of available for sale assets available for sale financial assets proceeds from disposal and maturities of assets available for sale interest income note revaluation of available for sale assets revaluation reclassified through profit or loss available for sale financial assets distribution of receivables by countries is presented in note mandatory banking reserve as at was of all financial resources collected deposits from customers and loans received the reserve requirement is to be fulfilled as a monthly average in euros or in the foreign securities preapproved by the central bank the group has available for sale debt and equity securi ties portfolio the balance of other reserve in equity as of is eur thousand eur thousand see also note in and no gains or losses arose from the sales of debt securities in visa share was revalued to eur thousand which profit on disposal was recycled to profit and loss statement during new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee bid price is the fair value of investments in the case of publicly listed securities note in a gain of eur thousand was recognised on the revaluation of debt securities eur thousand no profit or loss arose on the revaluation of interest rate swaps in loss eur thousand the volume of pension and investment fund assets managed by the group as at was eur million eur million note discontinued operations note loans and advances to customers discontiniued operations presents the groups disposed business of the branch in finland which was sold on january in thousands of euros loans to legal entities incl corporate loans incl retail loans incl leasing incl overdraft incl leveraged loans incl hire purchase incl credit card loans loans to individuals incl hire purchase incl mortgage loans incl consumer loans incl leasing incl leveraged loans incl credit card loans incl overdraft total incl related parties note impairment provisions total net income from discontinued operations other financial income total expenses net profit for the year from discontinued operations new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee balance as at january impairment provisions set up during the year written off during the year balance as at december balance as at january impairment provisions set up during the year written off during the year balance as at december changes in impairments in changes in impairments in net and gross investments on finance leases according to remaining maturity corporate loans incl overdraft retail loans hire purchase leasing leve raged loans other loans incl mortgage total consumer loans credit cards corporate loans incl overdraft hire purchase leasing gross investment leve raged loans unearned future interest income other loans incl mortgage impair ment loss provision total present value of lease payments receivable consumer loans credit cards up to year years over years total as at up to year years over years total as at note receivables from customers asset management fees from customers incl related parties note other fees for providing services to customers payments in transit other receivables total for credit risk exposures and loan collateral see note distribution of loans granted by currencies is disclosed in note distribution of loans granted by maturity is disclosed in note the geographical distribution of loans granted is disclosed in note for interest income on loans granted see note all fees other than receivables related to collection of receivables are collected within months of the balance sheet date and are considered as current asset new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note other assets note tangible and intangible assets financial assets guarantee deposits of baltic stock exchanges guarantee deposits of visa and mastercard subtotal non financial assets prepayments to financial supervision authority tax prepayments repossessed assets prepayments to merchants for registered customer contracts other prepayments subtotal total prepayments include office rent insurance communica tion services periodicals and training prepayments are expected to be received or used within months of the balance sheet date and are therefore consid ered current assets guarantee deposits on the baltic stock exchanges are held to guarantee securities trading activity on the stock exchanges of tallinn riga and vilnius and the deposits of visa and mastercard to guarantee credit card transactions and should therefore both be considered non current assets tangible assets intangible assets total balance as at cost accumulated depreciation and amortisation assets of discontinued operations note carrying amount changes in purchase of non current assets depreciation amortisation charge write off of non current assets balance as at cost accumulated depreciation and amortisation carrying amount purchase of non current assets assets acquired on business combination disposal of non current assets depreciation amortisation charge write off of non current assets balance as at cost accumulated depreciation and amortisation carrying amount tangible assets include computer technology and office equipment furniture capitalized costs of office renovation intangible assets include licences and development costs in and there was no indication of impairment of tangible and intangible assets new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note deposits from customers and loans received note accounts payable and other liabilities individuals legal entities public sector total demand deposits term deposits loans received accrued interest liability total incl related parties note individuals legal entities public sector total demand deposits term deposits loans received accrued interest liability total incl related parties note financial liabilities trade payables payables to merchants other short term financial liabilities accrued interest on subordinated loans note payments in transit financial guarantee contracts issued subtotal non financial liabilities performance guarantee contracts issued tax liabilities payables to employees incl related parties note other short term liabilities subtotal total loans received from public sector includes loans from maaelu edendamise sihtasutus rural development foun dation in the amount of eur thousand eur thousand with an intended purpose to finance loans to small enterprises operating in rural areas in loans received included also loan from the european central bank secured with the debt securities portfolio in the amount of eur thousand and interest rate the nominal interest rates of most deposits from customers and loans received equal their effective interest rates as no other significant fees have been paid distribution of deposits from customers and loans received by currency is presented in note distribution of deposits from customers and loans received by maturity is presented in note distribution of deposits from customers and loans received by geography is presented in note new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee payables to employees consist of unpaid salaries bonus accruals and vacation pay accrual for the reporting period and the increase in liabilities is caused by the increase in the number of employees during the year payments in transit consist of foreign payments and payables to customers related to intermediation of securities transactions for which the customers current account have been debited all liabil ities except for financial guarantees are payable within months and are therefore recognised as current liabilities the group has received subordinated debts in order to increase long term capital in the case of default of the group the subordinated debts are repayable after all other debts have been paid but before debts to shareholders are paid the balances of subordinated debt as at the end of each reporting period is disclosed in the table below in october subordinated bonds were issued totalling eur thousand carrying a rate of interest at the due date of the bonds is october after five years the issuer has the right to prematurely redeem the bonds in june subordinated bonds were issued totalling eur thousand carrying a rate of interest at the due date of the bonds is june in december subordinated bonds were issued totalling eur thousand most of these bonds have been converted to share capital refer to note and the remaining bonds of eur thousand were redeemed in october interest expenses on subordinated bonds for each reporting period and accrued interest liabilities as at the end of each reporting period is disclosed in the table below interest liabilities are accounted in the statement of financial position using the effective interest rate subordinated debt in thousands of euros subordinated debt as at subordinated bonds redeemed subordinated bonds issued subordinated debt as at subordinated debt as at the group leases office premises under the operating lease terms all lease agreements are cancellable upon the consent of both parties the minimum unilaterally non can cellable lease payable in the next period are disclosed in the table below in the operating lease payments for office premises in the amount of eur thousand eur thousand note operating lease note subordinated debts up to year to year total non cancellable lease payables as of non cancellable lease payables as of accrued interest on subordinated debt in thousands of euros accrued interest on subordinated debts as at note interest calculated for note paid out during accrued interest on subordinated debts as at note interest calculated for note paid out during accrued interest on subordinated debts as at note new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note shareholders equity in the public limited company number of shares share premium share capital as at share capital as at paid in share capital may share capital as at share capital is paid in in full through cash contributions the nominal value of the shares is euro and as at the number of shares amounted to according to as lhv groups articles of association the minimum share capital is eur million and the maximum share capital is eur million eur and eur million rain lõhmus who owns of the voting rights and andres viisemann who owns of the voting rights in as lhv group have significant influence over the company and as at the retained earnings of the group totalled eur thousand eur thousand as of it is possible to pay out divi dends in amount thousand euros and the related income tax charge would be thousand euros statutory reserve capital in equity is as follows in thousands of euros statutory reserve as at transferred from net profit statutory reserve as at transferred from net profit statutory reserve as at other reserves in the consolidated statement of changes in equity consist of in thousands of euros revaluation reserve of available for sale securities reserve of share options granted to staff total the group is granting share options to the management board members and equivalent employees and department managers and equivalent employees of group companies that are part of as lhv group number of shares strike price per share eur exercise period number of people to whom the share options were granted outstanding amount of share options at granted amount during the period outstanding amount of share options at granted amount during the period outstanding amount of share options at granted amount during the period cancelled amount during the period outstanding amount of share options at new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee the group will most likely issue the above mentioned share options also for the period the vesting period for all the share options program is years vesting date is the first day of the respective exercise period no options expired during the periods covered by the above tables members of the management board and employees were unable to opt for the determined amount in cash instead of share options share options cannot be exchanged sold pledged or encumbered share options can be inher ited the share option agreement will become void upon premature termination of the employment contract at the employees initiative exemptions to which can be made by the supervisory board of as lhv group and due to termination of employment contract on an extraordinary basis by the employer for a cause related to the employee pursuant to the credit institutions act the supervisory board of as lhv group may reduce the number of share options issued or cancel share options if the overall financial results of the group have materially deteriorated compared to the previous period a member of the management board or employee of the group no longer meets award criteria the group no longer meets the prudential ratios the risks of the group are not adequately covered with own funds or deter mination of the performance pay was based on information which was inaccurate or incorrect to a material extent total expenses arising from share based payment transactions recognised during the period as part of staff costs was eur thousand asset management fees for the management of these assets have been in the range of p a for respective income see note the monetary funds of the customers who use the platform of an active securities trader or the trading system lhv trader offered by the bank have been recognised as off balance sheet assets due to the nature of the system the bank has deposited these funds in personalised accounts with its partner and as the monetary funds of these customers are not used for business purposes by the bank they cannot be lent to other customers or used as collateral therefore the monetary funds are recognised as off balance sheet assets the bank earns commission and interest income on intermediation of transactions in these accounts similarly to customers accounts reflected in the groups financial statement the bank has provided contractual guarantees to its partner in respect of lhv trader accounts guaran teeing potential losses to be incurred from financing of the transactions executed by customers intermediated by itself leveraging with the primary collateral being the securities used as collateral for leveraged loans the customer has similar guarantee to the bank and therefore the bank has in substance transferred the guarantee to end customer to avoid potential losses the bank has a daily practice to monitor the potential decrease of collateral and deficiency of customer accounts in its current practice no such losses in respect of which the guarantee provided by the bank would apply have incurred and the bank has not paid any compen sation relating to the guarantee note assets under management as lhv pank operating as an account manager for its customers has custody of or intermediates the following customer assets cash balance of customers securities of customers incl shareholders of the parent company and related entities note total new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee tax authorities have the right to review the companys tax records for up to years after submitting the tax decla ration and upon finding errors impose additional taxes interest and fines the tax authorities have not performed any tax audits at the group during the groups management estimates that in there are no such circumstances which may lead the tax authorities to impose significant additional taxes on the group performance guarantees are contracts that provide compensation if another party fails to perform a contractual obligation such contracts do not transfer credit risk the risk under performance guarantee contracts is the possibility that the insured event i e the failure to perform the contrac tual obligation by another party occurs the key risks the group faces are significant fluctuations in the frequency and severity of payments incurred on such contracts relative to expectations the group uses historical data and statistical techniques to predict levels of such payments claims must be made before the contract matures and most claims are settled within short term this allows the group to achieve a high degree of certainty about the estimated payments and therefore future cash flows the group manages such risks by constantly monitoring the level of payments for such products and has the ability to adjust its fees in the future to reflect any change in claim payments experi ence the group has a claim payment requests handling process which includes the right to review the claim and reject fraudulent or non compliant requests the exposure and concentration of performance guarantees expressed at the amounts guaranteed is as follows performance guarantees are over collateralised as at each reporting date therefore the group has not borne any losses from performance guarantee contracts neither in nor in previous period according to sectors construction water supplies manufacturing professional scientific and technical activities other areas at activities total according to internal ratings low credit risk low credit risk medium credit risk medium credit risk heightened credit risk high credit risk high credit risk non satisfactory rating total note contingent assets and liabilities irrevocable transactions liability in contractual amount liability in contractual amount performance guarantees financial guarantees unused loan commitments total new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note transactions with related parties in preparing the financial statements of the group the following entities have been considered related parties owners that have significant impact on the group and the entities related to them members of the management board and legal entities controlled by them together referred to as management members of the supervisory board close relatives of the persons mentioned above and the entities related to them transactions note interest income incl management incl shareholders their related entities and close relatives that have significant influence fee and commission income incl management incl shareholders their related entities and close relatives that have significant influence interest expenses from deposits incl management incl shareholders their related entities and close relatives that have significant influence interest expenses from subordinated debt incl management incl shareholders their related entities and close relatives that have significant influence balances note loans and receivables as at the year end incl management incl shareholders their related entities and close relatives that have significant influence deposits as at the year end incl management incl shareholders their related entities and close relatives that have significant influence subordinated debt as at the year end incl management incl shareholders their related entities and close relatives that have significant influence the table provides an overview of the material balances and transactions involving related parties all other transac tions involving the close relatives and the entities related to members of the management board and supervisory board and the minority shareholders of the parent company as lhv group have occurred in the course and on the terms of ordinary business loans granted to related parties are issued at market conditions as at and the management did not have term deposits and the interest rate on demand deposits corresponds to the terms applicable to customers the subordinated debts received in october have the interest rate of the subordinated debts received in june have the interest rate of refer to note the subordinated debts received in december had the interest rate of during the first three years these loans were redeemed in october new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee in salaries and other compensations paid to the management of the parent as lhv group and its subsid iaries totalled eur thousand eur thousand including all taxes as at remuner ation for december and accrued holiday pay in the amount of eur thousand eur thousand is reported as a payable to management note the group did not have any long term payables or commitments to the members of the management board and the supervisory board as at and pension liabilities termination benefits etc in the remuneration paid to the members of the groups supervisory board totalled eur thousand eur thousand the group has signed contracts with the members of the management board which do not provide for severance benefits upon termination of the contract in any matters not regulated by the contract the parties adhere to the proce dure specified in the legislation of the republic of estonia management is involved with share option program in the share options were granted to the members of the management board in the amount of eur thousand eur thousand information on assets of related parties held as an account manager is presented in note in order to calculate basic earnings per share net profit has been divided by the weighted average number of shares issued for calculation of diluted earnings per share for the annual average potential shares from subordi nated bonds is used although the last convertible bonds were redeemed in october see note and as at december no convertible bonds were outstanding note basic earnings and diluted earnings per share total profit incl discontinued operations attributable to owners of the parent eur thousand profit attributable to owners of the parent from continuing operations eur thousand weighted average number of shares in thousands of units basic earnings per share eur basic earnings per share from continuing operations eur basic earnings per share from discontinued operations eur weighted average number of shares used for calculating the diluted earnings per shares in thousands of units diluted earnings per share eur diluted earnings per share eur from continuing operations diluted earnings per share eur from discontinued operations weighted average number of shares used as the denominator in thousands of shares weighted average number of ordinary shares used as the denominator in calculating basic earnings per share adjustments for calculation of diluted earnings per share convertible subordinated bonds share options weighted average number of ordinary shares and potential ordinary shares used as the denominator in calculating diluted earnings per share new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee note separate financial statements of parent company in accordance with the estonian accounting act information on the separate primary financial statements of the parent of the consolidation group shall be disclosed in the notes to the financial statements statement of profit or loss and other comprehensive income of the parent in thousands of euros interest income interest expense net interest expense net gains losses from investments to associates net gains losses from financial assets operating expenses net loss for the year total loss and other comprehensive loss for the year new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee statement of financial position of the parent in thousands of euros assets due from banks and investment companies loans granted other receivables and assets investments in subsidiaries total assets liabilities accrued expenses and other liabilities subordinated debt total liabilities equity share capital share premium statutory reserve capital other reserves accumulated deficit total shareholders equity total liabilities and equity new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee statement of cash flows of the parent in thousands of euros cash flows from operating activities interest received interest paid administrative and other operating expenses paid cash flows from operating activities before change in operating assets and liabilities adjustments investments in subsidiaries from share options net increase decrease in operating assets and liabilities net increase decrease of other receivables net increase decrease of other liabilities net cash from used in operating activities cash flows from investing activities loans granted disposals of subsidiaries and associates acquisitions of subsidiaries and associates net cash from used in investing activities cash flows from financing activities paid in to share capital incl share premium repayment of subordinated debt proceeds from subordinated debt net cash from financing activities increase decrease in cash and cash equivalents cash and cash equivalents at the beginning of the financial year cash and cash equivalents at the end of the financial year new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee statement of changes in shareholders equity in thousands of euros balance as at transfer to statutory reserve capital share options total loss and other comprehensive loss for balance as at carrying amount of holdings under control and significant influence value of holdings under control and significant influence under equity method adjusted unconsolidated equity as at balance as at paid in share capital transfer to statutory reserve capital share options total loss and other comprehensive loss for balance as at carrying amount of holdings under control and significant influence value of holdings under control and significant influence under equity method adjusted unconsolidated equity as at statutory reserve capital share premium share capital other reserves accumulated deficit retained earnings total adjusted unconsolidated equity is the maximum amount that can be distributed to shareholders according to estonian legislation new stamp as lhv group tartu mnt tallinn info lhv ee lhv ee signatures of the management board to the consolidated annual report the management board has prepared the management report and the consolidated financial statements of as lhv group for the financial year ended december the management board confirms that the management report on pages provides a true and fair view of the business operations financial results and financial condition of the parent company and the entities included in consolidation as a whole the management board confirms that according to their best knowledge the consolidated financial report on pages presents a fair view of the assets liabilities financial position and profit or loss of the issuer and the entities involved in the consolidation as a whole according to the international financial reporting standards as they are adopted by the european union and contains a description of the main risks and doubts madis toomsalu stamp as pricewaterhousecoopers pärnu mnt tallinn estonia license no registry code t f www pwc ee independent auditors report translation of the estonian original to the shareholders of lhv group as our opinion in our opinion the consolidated financial statements present fairly in all material respects the consolidated financial position of lhv group as the company and its subsidiaries together the group as at december and its consolidated financial performance and its consolidated cash flows for the year then ended in accordance with international financial reporting standards as adopted by the european union what we have audited the groups consolidated financial statements comprise the consolidated statement of financial position as at december the consolidated statement of profit or loss and other comprehensive income for the year then ended the consolidated statement of cash flows for the year then ended the consolidated statement of changes in equity for the year then ended and the notes to the consolidated financial statements which include a summary of significant accounting policies and other explanatory information basis for opinion we conducted our audit in accordance with international standards on auditing isas our responsibilities under those standards are further described in the auditors responsibilities for the audit of the consolidated financial statements section of our report we believe that the audit evidence we have obtained is sufficient and appropriate to provide a basis for our opinion independence we are independent of the group in accordance with the international ethics standards board for accountants code of ethics for professional accountants iesba code and the ethical requirements of the auditors activities act of the republic of estonia we have fulfilled our other ethical responsibilities in accordance with the iesba code and the ethical requirements of the auditors activities act of the republic of estonia our audit approach overview materiality overall group materiality is eur thousand which represents of profit before tax adjusted for any significant one off gains or losses audit scope a full scope audit was performed by pwc network firms for group entities covering substantially all of the groups consolidated statements of financial position and profit or loss key audit matters impairment of loans and advances to customers accounting for the acquisition of danske capital as as part of designing our audit we determined materiality and assessed the risks of material misstatement in the consolidated financial statements in particular we considered where the management board made subjective judgments for example in respect of significant accounting estimates that involved making assumptions and considering future events that are inherently uncertain as in all of our audits we also addressed the risk of management override of internal controls including among other matters consideration of whether there was evidence of bias that represented a risk of material misstatement due to fraud materiality the scope of our audit was influenced by our application of materiality an audit is designed to obtain reasonable assurance whether the consolidated financial statements are free from material misstatement misstatements may arise due to fraud or error they are considered material if individually or in aggregate they could reasonably be expected to influence the economic decisions of users taken on the basis of the consolidated financial statements based on our professional judgment we determined certain quantitative thresholds for materiality including the overall group materiality for the consolidated financial statements as a whole as set out in the table below these together with qualitative considerations helped us to determine the scope of our audit and the nature timing and extent of our audit procedures and to evaluate the effect of misstatements both individually and in aggregate on the financial statements as a whole overall group materiality eur thousand how we determined it of profit before tax adjusted for any significant one off gains or losses rationale for the materiality benchmark applied the use of profit before tax is considered appropriate as in our view profit before tax is stakeholders primary measurement benchmark and key performance indicator for management and supervisory board the adjusting items are excluded on the basis that they do not reflect the day to day operational performance of the group materiality audit scope key audit matters we agreed with the audit committee that we would report to them misstatements identified during our audit above eur thousand as well as misstatements below that amount that in our view warranted reporting for qualitative reasons key audit matters key audit matters are those matters that in our professional judgment were of most significance in our audit of the consolidated financial statements of the current period these matters were addressed in the context of our audit of the consolidated financial statements as a whole and in forming our opinion thereon and we do not provide a separate opinion on these matters key audit matter how our audit addressed the key audit matter impairment of loans and advances to customers refer to note summary of significant accounting policies note credit risk and note loans and advances to customers for further details as at december loans and advances to customers amounted to eur million and related impairment loss for amounted to eur million we focused on this area because management makes complex and subjective judgments over both timing of recognition of impairment and the estimation of the size of any such impairment for corporate loans a portion of impairment is individually assessed by taking into consideration the credit rating of the customer and other known heightened risk factors credit rating is assigned considering a combination of financial and business risks associated with the customer individual impairment is calculated based on the exposure and realisable value of the collateral at the balance sheet date in addition to individual assessment and impairment calculation of corporate loans a significant portion of their impairment is calculated on a collective basis taking into account customer rating historical performance the probability of default loss given default and other factors loans to individuals and loans to legal entities other than corporate loans are grouped for impairment calculation purposes into sub classes on the basis of we assessed whether the groups accounting policies in relation to the impairment of loans and advances to customers are in compliance with ifrs we assessed the design and operating effectiveness of the controls over impairment data and calculations these controls included those over the identification of which loans and advances were individually impaired overdue loans monitoring and the calculation of the collective impairment provisions including annual validation process of collective impairment model inputs in addition we tested the design and operating effectiveness of the credit file periodic review and rating assessment and monitoring of collateral controls for corporate loans we determined that we could rely on these controls for the purposes of our audit we performed detailed testing over the completeness and accuracy of data in the impairment model for collective impairment calculations the internal assignment of credit ratings for corporate loan customers the assumptions used for critical inputs in the collective impairment model such as probability of default and loss given default we have checked the annual calculations of probability of default and loss given default performed by management for material loan products by evaluating the correctness of the calculation methodology and historical data used the correctness of information on collaterals homogeneous credit risk features the major part of impairment for those sub classes is calculated on a collective basis taking into account historical performance the probability of default loss given default and other factors areas that require the most significant judgments and estimates are the identification and monitoring process of corporate customers with ratings below investment grade and other known heightened risk factors and individual impairment calculations of such loans with special attention on the valuation of collaterals the key assumptions and judgments made by management that underlie the calculation of collective impairment key assumptions and judgments include the probability of default calculation the loss given default calculation and credit conversion factor calculation the assessment of annual internal validation process of group based valuation models taking into account the repayment behavior of different customer groups and the completeness of the customer accounts that are included in the impairment calculation and their values in the loan systems and the completeness of loans subject to individual impairment assessment and related calculations for customers operating in currently low performing industry sectors e g agriculture and oil industry we checked the total exposure of customer loans in those sectors and assessed the performance of those customers based on internal and external information in the case of some impairment provisions we formed a different view from that of the management but in our view the differences were within a reasonable range of outcomes in the context of the overall loans and advances and the uncertainties disclosed in the financial statements as a result of our work we noted no material exceptions accounting for the acquisition of danske capital as refer to note summary of significant accounting policies and note subsidiaries and associated companies goodwill for further details in may the group acquired the asset management company danske capital as together with pension and investment funds managed by it for a total consideration of eur million as further described in note accounting for business combination involves compiling the purchase price allocation during which the fair value of the purchase consideration paid is allocated to the fair values of the identifiable assets acquired liabilities and contingent liabilities assumed with any remaining difference recorded as goodwill these fair values were assessed by the management and it required judgment due to the size of the acquired business and related estimation uncertainty it is considered a key audit matter we assessed whether the groups accounting policy in relation to accounting for the business combinations is in compliance with ifrs we evaluated and performed detailed testing of the purchase price allocation compiled by the management including assessing the list of separately identified assets and liabilities for their reasonableness and compliance with ifrs requirements evaluating the managements assessment of the fair values of identifiable assets acquired and liabilities assumed involving pwc valuation experts to evaluate the methodology applied for determination of fair values deploying our pwc valuation experts we tested the reasonableness of key assumptions including cash flow forecasts customer churn rate and discount rates and checking the correctness of calculation of arising goodwill furthermore we assessed the adequacy of the disclosures related to the acquisition of danske capital as as a result of our work we noted no material exceptions how we tailored our audit scope we tailored the scope of our audit in order to perform sufficient work to enable us to provide an opinion on the consolidated financial statements as a whole taking into account the structure of the group the accounting processes and controls and the financial services industry in which the group operates the group comprises a number of subsidiaries that mostly operate in estonia and lithuania refer to note a full scope audit was performed by pwc estonia or other pwc network firms for the following group entities covering substantially all of the groups consolidated statements of financial position and profit or loss lhv group as lhv pank as including its subsidiary lhv finance as lhv varahaldus as and uab mokilizingas where the work was performed by other pwc network firms we determined the level of involvement we needed to have to be able to conclude whether sufficient appropriate audit evidence had been obtained as a basis for our opinion on the group financial statements as a whole the nature timing and extent of the work performed by other pwc network firms impacting the group audit opinion is set and monitored in estonia other information the management board is responsible for the other information contained in the consolidated annual report in addition to the consolidated financial statements and our auditors report thereon our opinion on the consolidated financial statements does not cover the other information and we do not express any form of assurance conclusion thereon in connection with our audit of the consolidated financial statements our responsibility is to read the other information identified above and in doing so consider whether the other information is materially inconsistent with the consolidated financial statements or our knowledge obtained in the audit or otherwise appears to be materially misstated if based on the work we have performed we conclude that there is a material misstatement of this other information we are required to report that fact we have nothing to report in this regard responsibilities of the management board and those charged with governance for the consolidated financial statements the management board is responsible for the preparation and fair presentation of the consolidated financial statements in accordance with international financial reporting standards as adopted by the european union and for such internal control as the management board determines is necessary to enable the preparation of consolidated financial statements that are free from material misstatement whether due to fraud or error in preparing the consolidated financial statements the management board is responsible for assessing the groups ability to continue as a going concern disclosing as applicable matters related to going concern and using the going concern basis of accounting unless the management board either intends to liquidate the group or to cease operations or has no realistic alternative but to do so those charged with governance are responsible for overseeing the groups financial reporting process auditors responsibilities for the audit of the consolidated financial statements our objectives are to obtain reasonable assurance about whether the consolidated financial statements as a whole are free from material misstatement whether due to fraud or error and to issue an auditors report that includes our opinion reasonable assurance is a high level of assurance but is not a guarantee that an audit conducted in accordance with isas will always detect a material misstatement when it exists misstatements can arise from fraud or error and are considered material if individually or in the aggregate they could reasonably be expected to influence the economic decisions of users taken on the basis of these consolidated financial statements as part of an audit in accordance with isas we exercise professional judgment and maintain professional scepticism throughout the audit we also identify and assess the risks of material misstatement of the consolidated financial statements whether due to fraud or error design and perform audit procedures responsive to those risks and obtain audit evidence that is sufficient and appropriate to provide a basis for our opinion the risk of not detecting a material misstatement resulting from fraud is higher than for one resulting from error as fraud may involve collusion forgery intentional omissions misrepresentations or the override of internal control obtain an understanding of internal control relevant to the audit in order to design audit procedures that are appropriate in the circumstances but not for the purpose of expressing an opinion on the effectiveness of the groups internal control evaluate the appropriateness of accounting policies used and the reasonableness of accounting estimates and related disclosures made by the management board conclude on the appropriateness of the management boards use of the going concern basis of accounting and based on the audit evidence obtained whether a material uncertainty exists related to events or conditions that may cast significant doubt on the groups ability to continue as a going concern if we conclude that a material uncertainty exists we are required to draw attention in our auditors report to the related disclosures in the consolidated financial statements or if such disclosures are inadequate to modify our opinion our conclusions are based on the audit evidence obtained up to the date of our auditors report however future events or conditions may cause the group to cease to continue as a going concern evaluate the overall presentation structure and content of the consolidated financial statements including the disclosures and whether the consolidated financial statements represent the underlying transactions and events in a manner that achieves fair presentation obtain sufficient appropriate audit evidence regarding the financial information of the entities or business activities within the group to express an opinion on the consolidated financial statements we are responsible for the direction supervision and performance of the group audit we remain solely responsible for our audit opinion we communicate with those charged with governance regarding among other matters the planned scope and timing of the audit and significant audit findings including any significant deficiencies in internal control that we identify during our audit we also provide those charged with governance with a statement that we have complied with relevant ethical requirements regarding independence and to communicate with them all relationships and other matters that may reasonably be thought to bear on our independence and where applicable related safeguards from the matters communicated with those charged with governance we determine those matters that were of most significance in the audit of the consolidated financial statements of the current period and are therefore the key audit matters we describe these matters in our auditors report unless law or regulation precludes public disclosure about the matter or when in extremely rare circumstances we determine that a matter should not be communicated in our report because the adverse consequences of doing so would reasonably be expected to outweigh the public interest benefits of such communication as pricewaterhousecoopers ago vilu verner uibo auditors certificate no auditors certificate no february this version of our report is a translation from the original which was prepared in estonian all possible care has been taken to ensure that the translation is an accurate representation of the original however in all matters of interpretation of information views or opinions the original language version of our report takes precedence over this translation as lhv group tartu mnt tallinn info lhv ee lhv ee proposal for profit distribution the management board of lhv group proposes to the general meeting of shareholders to distribute the profit of the financial year as follows transfer eur thousand to statutory reserve capital pay dividends eur per share in the total amount of thousand transfer the profit for reporting period attributable to shareholders of the parent in the amount of eur thousand to retained earnings as lhv group tartu mnt tallinn info lhv ee lhv ee signatures of the supervisory board to the annual report chairman of the supervisory board rain lõhmus members of the supervisory board raivo hein heldur meerits tiina mõis sten tamkivi tauno tats andres viisemann as lhv group tartu mnt tallinn info lhv ee lhv ee allocation of income according to emta classificators consolidated emtak activity security and commodity contracts brokerage credit institutions banks granting loans finance lease fund management total income separate emtak activity activities of holding companies total income '.lower().split()



nltk.download('stopwords', quiet=True)
stop_words = stopwords.words('english')
sentence_obama = [w for w in sentence_obama if w not in stop_words]
sentence_president = [w for w in sentence_president if w not in stop_words]


corpus = [sentence_obama, sentence_president]
dictionary = Dictionary(corpus)
tfidf = TfidfModel(dictionary=dictionary)

sentence_obama = tfidf[dictionary.doc2bow(sentence_obama)]
sentence_president = tfidf[dictionary.doc2bow(sentence_president)]

cossim(sentence_obama, sentence_president)
#glove = api.load("glove-wiki-gigaword-50")
#similarity_index = WordEmbeddingSimilarityIndex(glove)
#similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

#interpret_soft_cosine_measure(sentence_obama, sentence_president, dictionary, similarity_matrix)
similarity_matrix.matrix.nnz
s = len(dictionary)**2)
s

SyntaxError: unmatched ')' (<ipython-input-103-83f385709a1e>, line 39)